In [ ]:
import json
import base64
import ipaddress
import datetime
import requests
import dns.message
import pyasn
import sys
import ast
import socket
import statistics

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.font_manager as font_manager
from matplotlib import rcParams

import ipaddress
import sys
from tqdm import tqdm
import os, glob
from matplotlib import colors as pltColors
import geopandas
import cartopy.crs as ccrs
from matplotlib.lines import Line2D
import copy
import collections
import sqlite3
import haversine as hs
from matplotlib.patches import Patch
import matplotlib.patches as patches
from scipy import stats
from matplotlib import ticker
import operator

In [ ]:
# Set formatting parameters and file paths
rcParams["font.family"] = "CMU Sans Serif"
rcParams["font.size"] = 10

plt.rc('text', usetex=True)

# We are using the tab10 colormap
cols = plt.get_cmap("tab10").colors

asndb = pyasn.pyasn("../dataset/ipasn_25082025.dat")
cloudflare_aim_path = "../dataset/late-2025/cloudflare-AIM"
top_level_folder_path = "../dataset/late-2025/ripe_atlas"
old_top_level_folder_path = "../dataset/early-2025/ripe_atlas"
old_controlled_top_level_folder_path = "../dataset/early-2025/controlled_nodes"
controlled_top_level_folder_path = "../dataset/late-2025/controlled_nodes"
figs_top_level_folder_path = "../figs"
ip_to_loc_map_filename = "../dataset/ip_to_location.json"
ipinfo_ip_to_country_filename = "../dataset/ip-to-country-Aug2025.csv"
prb_map_dict = {}
prb_map_filename = "../dataset/ripe_probe_date_archive_list.json"
# ip_country_map_filename = "../dataset/cdn_ip_to_country_map.json"

In [ ]:
## Helper functions

def ip_to_int(ip):
    return int(ipaddress.ip_address(ip))

def find_ip(ip,df):
    ip_int = ip_to_int(ip)

    result = df[(df['start_ip_int'] <= ip_int) & (df['end_ip_int'] >= ip_int)]

    if len(result) == 1:
        country = result['country_name'].iloc[0]
        country_code = result['country'].iloc[0]
        continent = result['continent_name'].iloc[0]
        continent_code = result['continent'].iloc[0]
        return {
            'country': country,
            'country_code': country_code,
            'continent': continent,
            'continent_code': continent_code
        }

    return None

def reverse_dns_lookup(ip_address):
    """
    Performs a reverse DNS lookup for a given IP address.
    
    Args:
        ip_address (str): The IP address to look up.
        
    Returns:
        str or None: The domain name if found, otherwise None.
    """
    try:
        # gethostbyaddr returns a tuple: (hostname, aliaslist, ipaddrlist)
        hostname, _, _ = socket.gethostbyaddr(ip_address)
        return hostname
    except socket.herror:
        # Handle cases where no reverse DNS entry exists
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def check_ip_version(ip):
    try:
        ip_obj = ipaddress.ip_address(ip)
        if isinstance(ip_obj, ipaddress.IPv4Address):
            return "IPv4"
        elif isinstance(ip_obj, ipaddress.IPv6Address):
            return "IPv6"
    except ValueError:
        return "Invalid IP address"
    
def is_valid_ipv4(ip):
    try:
        ipaddress.IPv4Address(ip)
        return True
    except ipaddress.AddressValueError:
        return False

In [ ]:
ip_to_loc = {}
with open(ip_to_loc_map_filename, "r") as json_file:
    ip_to_loc = json.load(json_file)
    print("IP to Loc file loaded successfully")

In [ ]:
## Loading IPInfo IP to country database
ip_to_country_df = pd.read_csv(filepath_or_buffer=ipinfo_ip_to_country_filename,delimiter=',',keep_default_na=False)
ip_to_country_df['start_ip_int'] = ip_to_country_df['start_ip'].apply(ip_to_int)
ip_to_country_df['end_ip_int'] = ip_to_country_df['end_ip'].apply(ip_to_int)

# DNS Measurements - Cloudflare

## CHAOS queries

### Starlink

In [ ]:
network_type = "Starlink"
dns_server = "cloudflare"
msm_type = "dns"

acc_result_cf_filepath = os.path.join(top_level_folder_path,network_type,msm_type,dns_server,"CHAOS","acc_result",f"acc_dns_{dns_server}_CHAOS_final_result.json")
print(acc_result_cf_filepath)
with open(acc_result_cf_filepath,"r") as file:
    try:
        cf_data = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_cf_filepath, "Error: ", err)

In [ ]:
# Check for duplicate probes with different country 
cf_prb_country_map = {}
for prb_tuple_str, results in cf_data.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    
    if prb_id not in cf_prb_country_map:
        cf_prb_country_map[prb_id] = {}
    if country_code not in cf_prb_country_map[prb_id]:
        cf_prb_country_map[prb_id][country_code] = 0
    cf_prb_country_map[prb_id][country_code] += len(results['response_time'])

In [ ]:
cf_country_wise_dns_CHAOS_response_time_dict = {}
cf_country_prb_resolver_responsetime_dns_CHAOS_dict = {}
for prb_tuple_str, results in cf_data.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    countries_dict = cf_prb_country_map[prb_id]
    max_key_country = max(countries_dict,key=countries_dict.get)
    if country_code == max_key_country:
        if country_code not in cf_country_wise_dns_CHAOS_response_time_dict:
            cf_country_wise_dns_CHAOS_response_time_dict[country_code] = []
        cf_country_wise_dns_CHAOS_response_time_dict[country_code] += results['response_time']

        if country_code not in cf_country_prb_resolver_responsetime_dns_CHAOS_dict:
            cf_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code] = {}
        if prb_id not in cf_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code]:
            cf_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id] = {}
        for idx, resolver_loc in enumerate(results['resolver_location']):
                if resolver_loc not in cf_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id]:
                    cf_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id][resolver_loc] = []
                cf_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id][resolver_loc].append(results['response_time'][idx])


In [ ]:
del cf_data

# DNS Measurement - Google

## CHAOS queries

### Starlink

In [ ]:
network_type = "Starlink"
dns_server = "google"

acc_result_google_filepath = os.path.join(top_level_folder_path,network_type,msm_type,dns_server,"CHAOS","acc_result",f"acc_dns_{dns_server}_CHAOS_final_result.json")
print(acc_result_google_filepath)
with open(acc_result_google_filepath,"r") as file:
    try:
        google_data = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_google_filepath, "Error: ", err)

In [ ]:
# Check for duplicate probes with different country 
google_prb_country_map = {}
for prb_tuple_str, results in google_data.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    
    if prb_id not in google_prb_country_map:
        google_prb_country_map[prb_id] = {}
    if country_code not in google_prb_country_map[prb_id]:
        google_prb_country_map[prb_id][country_code] = 0
    google_prb_country_map[prb_id][country_code] += len(results['response_time'])

In [ ]:
google_country_wise_dns_CHAOS_response_time_dict = {}
google_country_prb_resolver_responsetime_dns_CHAOS_dict = {}
for prb_tuple_str, results in google_data.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    countries_dict = google_prb_country_map[prb_id]
    max_key_country = max(countries_dict,key=countries_dict.get)
    if country_code == max_key_country:
        if country_code not in google_country_wise_dns_CHAOS_response_time_dict:
            google_country_wise_dns_CHAOS_response_time_dict[country_code] = []
        google_country_wise_dns_CHAOS_response_time_dict[country_code] += results['response_time']

        if country_code not in google_country_prb_resolver_responsetime_dns_CHAOS_dict:
            google_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code] = {}
        if prb_id not in google_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code]:
            google_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id] = {}
        for idx, resolver_loc_str in enumerate(results['resolver_location']):
            if resolver_loc_str:
                resolver_loc = resolver_loc_str.split("-")[1]
                if resolver_loc not in google_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id]:
                    google_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id][resolver_loc] = []
                google_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id][resolver_loc].append(results['response_time'][idx])


In [ ]:
del google_data

# DNS Measurement - Quad9

## CHAOS queries

### Starlink

In [ ]:
network_type = "Starlink"
dns_server = "quad9"
msm_type = "dns"

acc_result_quad9_filepath = os.path.join(top_level_folder_path,network_type,msm_type,dns_server,"CHAOS","acc_result",f"acc_dns_{dns_server}_CHAOS_final_result.json")
print(acc_result_quad9_filepath)
with open(acc_result_quad9_filepath,"r") as file:
    try:
        quad9_data = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_quad9_filepath, "Error: ", err)

In [ ]:
# Check for duplicate probes with different country 
quad9_prb_country_map = {}
for prb_tuple_str, results in quad9_data.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]

    if prb_id not in quad9_prb_country_map:
        quad9_prb_country_map[prb_id] = {}
    if country_code not in quad9_prb_country_map[prb_id]:
        quad9_prb_country_map[prb_id][country_code] = 0
    quad9_prb_country_map[prb_id][country_code] += len(results['response_time'])

In [ ]:
quad9_country_wise_dns_CHAOS_response_time_dict = {}
quad9_country_prb_resolver_responsetime_dns_CHAOS_dict = {}
for prb_tuple_str, results in quad9_data.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    countries_dict = quad9_prb_country_map[prb_id]
    max_key_country = max(countries_dict,key=countries_dict.get)
    if country_code == max_key_country:
        if country_code not in quad9_country_wise_dns_CHAOS_response_time_dict:
            quad9_country_wise_dns_CHAOS_response_time_dict[country_code] = []
        quad9_country_wise_dns_CHAOS_response_time_dict[country_code] += results['response_time']

        if country_code not in quad9_country_prb_resolver_responsetime_dns_CHAOS_dict:
            quad9_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code] = {}
        if prb_id not in quad9_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code]:
            quad9_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id] = {}
        for idx, resolver_loc_str in enumerate(results['resolver_location']):
            resolver_loc = resolver_loc_str.split(".")[1]
            if resolver_loc not in quad9_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id]:
                quad9_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id][resolver_loc] = []
            quad9_country_prb_resolver_responsetime_dns_CHAOS_dict[country_code][prb_id][resolver_loc].append(results['response_time'][idx])


In [ ]:
del quad9_data

## Controlled Probes

### CHAOS queries

In [ ]:
chaos_controlled_nodes_filepath = f"{controlled_top_level_folder_path}/dns/CHAOS/08-26-2025-10-59-20_files_list"
controlled_CHAOS_msms = {
       "Cloudflare" : {},
       "Google" : {},
       "Quad9" : {},
}
for filename in glob.glob(os.path.join(chaos_controlled_nodes_filepath, '*.json')):

        count_new_items = 0
        f = open(filename)
        print(filename)
        data = json.load(f)

        if "user_details" in data:
                if "dns_measurements" in data:
                        user_details = data["user_details"]
                        dns_measurements =  data["dns_measurements"]

                        isp = user_details["ISP_AS"]
                        if not "14593" in isp:
                                continue
                        country_code = user_details["Country"]

                        
                        for dns_resolver in dns_measurements:
                                resolver_msm = dns_measurements[dns_resolver]
                                if country_code not in controlled_CHAOS_msms[dns_resolver]:
                                        for dns_resolver in controlled_CHAOS_msms:
                                                controlled_CHAOS_msms[dns_resolver][country_code] = {
                                                        'response_time' : [],
                                                        'resolver_location' : []
                                                }
                                if 'query_time' in resolver_msm:
                                        controlled_CHAOS_msms[dns_resolver][country_code]['response_time'].append(resolver_msm['query_time'])
                                        if 'resolver_id' in resolver_msm:
                                                if not resolver_msm['resolver_id'] == "":
                                                        controlled_CHAOS_msms[dns_resolver][country_code]['resolver_location'].append(resolver_msm['resolver_id'])
                                                elif not resolver_msm['nsid'] == "":
                                                        controlled_CHAOS_msms[dns_resolver][country_code]['resolver_location'].append(resolver_msm['nsid'])
                                                else:
                                                        controlled_CHAOS_msms[dns_resolver][country_code]['resolver_location'].append("")
                        

### Add controlled node measurements to CHAOS ripe measurements

In [ ]:
# Cloudflare
for country in controlled_CHAOS_msms['Cloudflare']:
    if country not in cf_country_wise_dns_CHAOS_response_time_dict:
        cf_country_wise_dns_CHAOS_response_time_dict[country] = []
    cf_country_wise_dns_CHAOS_response_time_dict[country] += controlled_CHAOS_msms['Cloudflare'][country]['response_time']

In [ ]:
# Google
for country in controlled_CHAOS_msms['Google']:
    if country not in google_country_wise_dns_CHAOS_response_time_dict:
        google_country_wise_dns_CHAOS_response_time_dict[country] = []
    google_country_wise_dns_CHAOS_response_time_dict[country] += controlled_CHAOS_msms['Google'][country]['response_time']

In [ ]:
# Quad9
for country in controlled_CHAOS_msms['Quad9']:
    if country not in quad9_country_wise_dns_CHAOS_response_time_dict:
        quad9_country_wise_dns_CHAOS_response_time_dict[country] = []
    quad9_country_wise_dns_CHAOS_response_time_dict[country] += controlled_CHAOS_msms['Quad9'][country]['response_time']

# CHAOS DNS Response Time

In [ ]:
def adjust_box(plot):
    for i in range(2):
        plt.setp(plot['boxes'][i], facecolor=cols[i], linewidth=1)
        plt.setp(plot['medians'][i], color='yellow')


common_countries = ["US", "CA" ,"DE", "HU", "CL", "CO", "AU", "NZ", "PH", "GU", "BJ", "ZW" , "ZM", "MG"]
fig, ax = plt.subplots(figsize=(4.66, 2))

pos1 = np.arange(0, 4 * len(common_countries), 4)
pos2 = np.arange(1, 4 * len(common_countries) + 1, 4)
pos3 = np.arange(2, 4 * len(common_countries) + 2, 4)

ax.boxplot([cf_country_wise_dns_CHAOS_response_time_dict[country] for country in common_countries], positions=pos1, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[1], lw=1), medianprops=dict(color="yellow"))
ax.boxplot([google_country_wise_dns_CHAOS_response_time_dict[country] for country in common_countries], positions=pos2, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[0], lw=1), medianprops=dict(color="yellow"))
ax.boxplot([quad9_country_wise_dns_CHAOS_response_time_dict[country] for country in common_countries], positions=pos3, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[3], lw=1), medianprops=dict(color="yellow"))
xspan = 4
[ax.axvspan(i * xspan - 1, i * xspan - 1 + xspan, facecolor="k", alpha=0.2)
    for i in range(len(common_countries))
    if i % 2 == 1]

ax.set_xticks(np.arange(1.0, 4 * len(common_countries), 4))
ax.set_xticklabels(common_countries)
ax.set_ylabel("DNS Response Time [ms]")
ax.set_yticks(np.arange(0, 260, 50))
# ax.set_ylim(-5,90)
ax.xaxis.get_major_formatter()._usetex = False
ax.yaxis.get_major_formatter()._usetex = False

handles = [Patch(facecolor=cols[1]),Patch(facecolor=cols[0]),Patch(facecolor=cols[3])]
labels = ["Cloudflare", "Google", "Quad9"]
ax.set_xlim(-1,55)
ax.legend(handles, labels, loc="upper left", fontsize="small", edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4)
plt.grid(True, axis="y", linestyle="--")
# plt.ylim(0,100)
fig.tight_layout()

plt.savefig(f"{figs_top_level_folder_path}/dns_comparison_CHAOS_rt_selected_countries.pdf", bbox_inches = "tight", pad_inches = 0.1)
plt.savefig(f"{figs_top_level_folder_path}/dns_comparison_CHAOS_rt_selected_countries.svg", bbox_inches = "tight", pad_inches = 0.1)
plt.show()

# DNS Measurements - Cloudflare

In [ ]:
cloudflare_dns_domains = ["www.broadcom.com.","www.comodoca.com.","www.apnic.net.","www.riskified.com.","www.wiley.com.",]
akamai_dns_domains = ["www.microsoft.com.", "www.apple.com.","www.bing.com.", "www.icloud.com.", "www.intuit.com.",]
cloudfront_dns_domains = [ "www.soundcloud.com.","www.zynga.com.","www.doi.org.","www.booking.com.","www.brave.com.",]

check_domains = ["www.microsoft.com.", "www.apple.com.","www.bing.com.", "www.icloud.com.",]

# Check top 500 domains
check_domains_top500 = []
tranco_df = pd.read_csv('../dataset/tranco_Sep2025.csv',header=0)
for full_domain in cloudflare_dns_domains + akamai_dns_domains + cloudfront_dns_domains:
    domain_split = full_domain.split(".")
    tld_part = domain_split[1] + "." + domain_split[2]
    domain_ranking = tranco_df.loc[(tranco_df['domain'] == tld_part), 'ranking'].iloc[0]
    if domain_ranking <= 500:
        check_domains_top500.append(full_domain)

## RD True

In [ ]:
network_type = "Starlink"
dns_server = "cloudflare"
msm_type = "dns"
rd_bit = True

acc_result_cf_filepath = os.path.join(top_level_folder_path,network_type,msm_type,dns_server,f"RD-{str(rd_bit)}","acc_result",f"acc_dns_{dns_server}_final_result.json")
with open(acc_result_cf_filepath,"r") as file:
    try:
        cf_data_resolution = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_cf_filepath, "Error: ", err)

### Add controlled nodes measurement

In [ ]:
controlled_WebMsms_folder_path = f"{controlled_top_level_folder_path}/dns/WebMsms"
network_type = "Starlink"
dns_server = "cf"
msm_type = "dns"
rd_bit = True

acc_result_google_filepath = os.path.join(controlled_WebMsms_folder_path,"acc_result",f"{dns_server}_controlled_DNS_acc_msms.json")
with open(acc_result_google_filepath,"r") as file:
    try:
        cf_controlled_data_resolution = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS controlled nodes accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_google_filepath, "Error: ", err)

In [ ]:

cf_country_wise_dns_resolution_time = {}
for prb_tuple_str, results in cf_data_resolution.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    if country_code not in cf_country_wise_dns_resolution_time:
        cf_country_wise_dns_resolution_time[country_code] = {}
    for domain, domain_results in results.items():
        
        if domain not in cf_country_wise_dns_resolution_time[country_code]:
            cf_country_wise_dns_resolution_time[country_code][domain] = {
                'resolved_ips': [],
                'resolver_loc' : [],
                'resolution_time_all' : [],
                'resolution_time_hit' : [],
                'resolution_time_miss' : [],
            }

        check_ttl = None
        if domain in cloudflare_dns_domains:
            check_ttl = 300
        elif domain in akamai_dns_domains:
            check_ttl = 20
        elif domain in cloudfront_dns_domains:
            check_ttl = 60
        else:
            continue
        
        for idx, each_ans in enumerate(domain_results['dns_ans']):
            if domain_results['response_time'][idx] > 15:
                if each_ans['ip_addr_dict']:
                    for _, ip_addr_list in each_ans['ip_addr_dict'].items():
                        cf_country_wise_dns_resolution_time[country_code][domain]['resolved_ips'].append(ip_addr_list[0])
                    cf_country_wise_dns_resolution_time[country_code][domain]['resolution_time_all'].append(domain_results['response_time'][idx])
                    
                    
                    ttl_val = domain_results['ttls'][idx]

                    if ttl_val == 0:
                        continue
                    elif ttl_val == check_ttl:
                        cf_country_wise_dns_resolution_time[country_code][domain]['resolution_time_miss'].append(domain_results['response_time'][idx])
                    elif ttl_val < check_ttl:
                        cf_country_wise_dns_resolution_time[country_code][domain]['resolution_time_hit'].append(domain_results['response_time'][idx])
                    


In [ ]:
cf_all_country_wise_dns_rt = {}

for country_code in cf_country_wise_dns_resolution_time:
    msm_results = cf_country_wise_dns_resolution_time[country_code]
    if country_code not in cf_all_country_wise_dns_rt:
        cf_all_country_wise_dns_rt[country_code] = {}
    for domain , domain_results in msm_results.items():
        if domain not in cf_all_country_wise_dns_rt[country_code]:
            cf_all_country_wise_dns_rt[country_code][domain] = [] 
        cf_all_country_wise_dns_rt[country_code][domain] += domain_results['resolution_time_all']

for country_code in cf_controlled_data_resolution:
    msm_results = cf_controlled_data_resolution[country_code]
    if country_code not in cf_all_country_wise_dns_rt:
        cf_all_country_wise_dns_rt[country_code] = {}
    for domain , domain_results in msm_results.items():
        if domain not in cf_all_country_wise_dns_rt[country_code]:
            cf_all_country_wise_dns_rt[country_code][domain] = []
        cf_all_country_wise_dns_rt[country_code][domain] += domain_results['response_time']

# DNS Measurements - Google

## RD True

In [ ]:
network_type = "Starlink"
dns_server = "google"
msm_type = "dns"
rd_bit = True

acc_result_google_filepath = os.path.join(top_level_folder_path,network_type,msm_type,dns_server,f"RD-{str(rd_bit)}","acc_result",f"acc_dns_{dns_server}_final_result.json")
print(acc_result_google_filepath)
with open(acc_result_google_filepath,"r") as file:
    try:
        google_data_resolution = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_google_filepath, "Error: ", err)

### Add controlled nodes measurement

In [ ]:
controlled_WebMsms_folder_path = f"{controlled_top_level_folder_path}/dns/WebMsms"
network_type = "Starlink"
dns_server = "google"
msm_type = "dns"
rd_bit = True

acc_result_google_filepath = os.path.join(controlled_WebMsms_folder_path,"acc_result",f"{dns_server}_controlled_DNS_acc_msms.json")
with open(acc_result_google_filepath,"r") as file:
    try:
        google_controlled_data_resolution = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS controlled nodes accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_google_filepath, "Error: ", err)

In [ ]:

google_country_wise_dns_resolution_time = {}
for prb_tuple_str, results in google_data_resolution.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    if country_code not in google_country_wise_dns_resolution_time:
        google_country_wise_dns_resolution_time[country_code] = {}
    for domain, domain_results in results.items():
        if domain not in google_country_wise_dns_resolution_time[country_code]:
            google_country_wise_dns_resolution_time[country_code][domain] = {
                'resolved_ips': [],
                'resolver_loc': [],
                'resolution_time_all' : [],
                'resolution_time_hit' : [],
                'resolution_time_miss' : [],

            }

        check_ttl = None
        if domain in cloudflare_dns_domains:
            check_ttl = 300
        elif domain in akamai_dns_domains:
            check_ttl = 20
        elif domain in cloudfront_dns_domains:
            check_ttl = 60
        else:
            continue
        
        for idx, each_ans in enumerate(domain_results['dns_ans']):
            if domain_results['response_time'][idx] > 15:
                if each_ans['ip_addr_dict']:
                    for _, ip_addr_list in each_ans['ip_addr_dict'].items():
                        google_country_wise_dns_resolution_time[country_code][domain]['resolved_ips'].append(ip_addr_list[0])
                    google_country_wise_dns_resolution_time[country_code][domain]['resolution_time_all'].append(domain_results['response_time'][idx])

                    resolver_loc_split = each_ans['nsid'].split('-')
                    resolver_loc = ""
                    if len(resolver_loc_split) == 2:
                        resolver_loc = resolver_loc_split[1]
                    google_country_wise_dns_resolution_time[country_code][domain]['resolver_loc'].append(resolver_loc)

                    ttl_val = domain_results['ttls'][idx]

                    if ttl_val == 0:
                        continue
                    elif ttl_val == check_ttl:
                        google_country_wise_dns_resolution_time[country_code][domain]['resolution_time_miss'].append(domain_results['response_time'][idx])
                    elif ttl_val < check_ttl:
                        google_country_wise_dns_resolution_time[country_code][domain]['resolution_time_hit'].append(domain_results['response_time'][idx])
                    



In [ ]:
google_all_country_wise_dns_rt = {}

for country_code in google_country_wise_dns_resolution_time:
    msm_results = google_country_wise_dns_resolution_time[country_code]
    if country_code not in google_all_country_wise_dns_rt:
        google_all_country_wise_dns_rt[country_code] = {}
    for domain , domain_results in msm_results.items():
        if domain not in google_all_country_wise_dns_rt[country_code]:
            google_all_country_wise_dns_rt[country_code][domain] = [] 
        google_all_country_wise_dns_rt[country_code][domain] += domain_results['resolution_time_all']

for country_code in google_controlled_data_resolution:
    msm_results = google_controlled_data_resolution[country_code]
    if country_code not in google_all_country_wise_dns_rt:
        google_all_country_wise_dns_rt[country_code] = {}
    for domain , domain_results in msm_results.items():
        if domain not in google_all_country_wise_dns_rt[country_code]:
            google_all_country_wise_dns_rt[country_code][domain] = []
        google_all_country_wise_dns_rt[country_code][domain] += domain_results['response_time']

# DNS Measurements - Quad9

## RD True

In [ ]:
network_type = "Starlink"
dns_server = "quad9"
msm_type = "dns"
rd_bit = True

acc_result_quad9_filepath = os.path.join(top_level_folder_path,network_type,msm_type,dns_server,f"RD-{str(rd_bit)}","acc_result",f"acc_dns_{dns_server}_final_result.json")
with open(acc_result_quad9_filepath,"r") as file:
    try:
        quad9_data_resolution = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_quad9_filepath, "Error: ", err)

### Add controlled nodes measurement

In [ ]:
controlled_WebMsms_folder_path = f"{controlled_top_level_folder_path}/dns/WebMsms"
network_type = "Starlink"
dns_server = "quad9"
msm_type = "dns"
rd_bit = True

acc_result_quad9_filepath = os.path.join(controlled_WebMsms_folder_path,"acc_result",f"{dns_server}_controlled_DNS_acc_msms.json")
with open(acc_result_quad9_filepath,"r") as file:
    try:
        quad9_controlled_data_resolution = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS controlled nodes accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_quad9_filepath, "Error: ", err)

In [ ]:

quad9_country_wise_dns_resolution_time = {}
for prb_tuple_str, results in quad9_data_resolution.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    if country_code not in quad9_country_wise_dns_resolution_time:
        quad9_country_wise_dns_resolution_time[country_code] = {}
    for domain, domain_results in results.items():
        if domain not in quad9_country_wise_dns_resolution_time[country_code]:
            quad9_country_wise_dns_resolution_time[country_code][domain] = {
                'resolved_ips': [],
                'resolver_loc': [],
                'resolution_time_all' : [],
                'resolution_time_hit' : [],
                'resolution_time_miss' : [],
            }

        check_ttl = None
        if domain in cloudflare_dns_domains:
            check_ttl = 300
        elif domain in akamai_dns_domains:
            check_ttl = 20
        elif domain in cloudfront_dns_domains:
            check_ttl = 60
        else:
            continue
        
        
        for idx, each_ans in enumerate(domain_results['dns_ans']):
            if domain_results['response_time'][idx] > 15:
                if each_ans['ip_addr_dict']:
                    for _, ip_addr_list in each_ans['ip_addr_dict'].items():
                        quad9_country_wise_dns_resolution_time[country_code][domain]['resolved_ips'].append(ip_addr_list[0])
                    quad9_country_wise_dns_resolution_time[country_code][domain]['resolution_time_all'].append(domain_results['response_time'][idx])

                    resolver_loc_split = each_ans['nsid'].split('.')
                    resolver_loc = ""
                    if len(resolver_loc_split) >= 2:
                        resolver_loc = resolver_loc_split[1]
                    quad9_country_wise_dns_resolution_time[country_code][domain]['resolver_loc'].append(resolver_loc)

                    ttl_val = domain_results['ttls'][idx]

                    if ttl_val == 0:
                        continue
                    elif ttl_val == check_ttl:
                        quad9_country_wise_dns_resolution_time[country_code][domain]['resolution_time_miss'].append(domain_results['response_time'][idx])
                    elif ttl_val < check_ttl:
                        quad9_country_wise_dns_resolution_time[country_code][domain]['resolution_time_hit'].append(domain_results['response_time'][idx])

In [ ]:
quad9_all_country_wise_dns_rt = {}

for country_code in quad9_country_wise_dns_resolution_time:
    msm_results = quad9_country_wise_dns_resolution_time[country_code]
    if country_code not in quad9_all_country_wise_dns_rt:
        quad9_all_country_wise_dns_rt[country_code] = {}
    for domain , domain_results in msm_results.items():
        if domain not in quad9_all_country_wise_dns_rt[country_code]:
            quad9_all_country_wise_dns_rt[country_code][domain] = [] 
        quad9_all_country_wise_dns_rt[country_code][domain] += domain_results['resolution_time_all']

for country_code in quad9_controlled_data_resolution:
    msm_results = quad9_controlled_data_resolution[country_code]
    if country_code not in quad9_all_country_wise_dns_rt:
        quad9_all_country_wise_dns_rt[country_code] = {}
    for domain , domain_results in msm_results.items():
        if domain not in quad9_all_country_wise_dns_rt[country_code]:
            quad9_all_country_wise_dns_rt[country_code][domain] = []
        quad9_all_country_wise_dns_rt[country_code][domain] += domain_results['response_time']

### All results

In [ ]:
def adjust_box(plot):
    for i in range(2):
        plt.setp(plot['boxes'][i], facecolor=cols[i], linewidth=1)
        plt.setp(plot['medians'][i], color='yellow')


common_countries = ["US", "CA" ,"DE", "HU", "CL", "CO", "AU", "NZ", "PH", "GU", "BJ", "ZM", "MG"]

fig, ax = plt.subplots(figsize=(4.66, 2))

pos1 = np.arange(0, 4 * len(common_countries), 4)
pos2 = np.arange(1, 4 * len(common_countries) + 1, 4)
pos3 = np.arange(2, 4 * len(common_countries) + 2, 4)

cf_dns_resolution_time_all = {}
google_dns_resolution_time_all = {}
quad9_dns_resolution_time_all = {}

domain_list = check_domains_top500

for country in common_countries:
    for domain in domain_list:
        if country not in cf_dns_resolution_time_all:
            cf_dns_resolution_time_all[country] = []
            google_dns_resolution_time_all[country] = []
            quad9_dns_resolution_time_all[country] = []

        if country in cf_all_country_wise_dns_rt:
            cf_dns_resolution_time_all[country] += cf_all_country_wise_dns_rt[country][domain]
        if country in google_all_country_wise_dns_rt:
            google_dns_resolution_time_all[country] += google_all_country_wise_dns_rt[country][domain]
        if country in quad9_all_country_wise_dns_rt:
            quad9_dns_resolution_time_all[country] += quad9_all_country_wise_dns_rt[country][domain]


ax.boxplot([cf_dns_resolution_time_all[country] for country in common_countries], positions=pos1, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[1], lw=1), medianprops=dict(color="yellow"))
ax.boxplot([google_dns_resolution_time_all[country]  for country in common_countries], positions=pos2, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[0], lw=1), medianprops=dict(color="yellow"))
ax.boxplot([quad9_dns_resolution_time_all[country]  for country in common_countries], positions=pos3, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[3], lw=1), medianprops=dict(color="yellow"))
xspan = 4
[ax.axvspan(i * xspan - 1, i * xspan - 1 + xspan, facecolor="k", alpha=0.2)
    for i in range(len(common_countries))
    if i % 2 == 1]

ax.set_xticks(np.arange(1.0, 4 * len(common_countries), 4))
ax.set_xticklabels(common_countries)
ax.set_ylabel("DNS Resolution Time [ms]")
ax.set_yticks(np.arange(0, 600, 100))
# ax.set_ylim(-5,90)
ax.xaxis.get_major_formatter()._usetex = False
ax.yaxis.get_major_formatter()._usetex = False

handles = [Patch(facecolor=cols[1]),Patch(facecolor=cols[0]),Patch(facecolor=cols[3])]
labels = ["Cloudflare", "Google", "Quad9"]
ax.set_xlim(-1,51)
ax.legend(handles, labels, loc="upper left", fontsize="small", edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4)
plt.grid(True, axis="y", linestyle="--")
# plt.ylim(0,100)
fig.tight_layout()

plt.savefig(f"{figs_top_level_folder_path}/dns_resolution_time_comparison_selected_countries_all.pdf", bbox_inches = "tight", pad_inches = 0.1)
plt.savefig(f"{figs_top_level_folder_path}/dns_resolution_time_comparison_selected_countries_all.svg", bbox_inches = "tight", pad_inches = 0.1)
plt.show()

# CDN Measurements

# Traceroute Measurements - Cloudflare CDN

## Starlink

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "cloudflare"
cf_acc_tracert_json = None
full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)
print(full_msms_path)
acc_cf_tracert_filename = "acc_trace_cloudflare_final_result.json"
acc_cf_tracert_filepath = os.path.join(full_msms_path,'acc_result',acc_cf_tracert_filename)
with open(acc_cf_tracert_filepath, "r") as json_file:
    cf_acc_tracert_json = json.load(json_file)

In [ ]:
cf_country_wise_cdn_rtts_stl = {}
cf_country_wise_pop_rtts_stl = {}

idx = 0
for each_msm in cf_acc_tracert_json:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']

    if country_code not in cf_country_wise_cdn_rtts_stl:
        cf_country_wise_cdn_rtts_stl[country_code] = []
        cf_country_wise_pop_rtts_stl[country_code] = []


    dst_addr = each_msm['dst_addr']
    
    total_dst_rtts = []
    for idx, hop_ip in enumerate(each_msm['all_hops'][-1]['hop_ips']):
        if hop_ip == dst_addr:
            total_dst_rtts.append(each_msm['all_hops'][-1]['hop_rtts'][idx])
    
    if len(total_dst_rtts):
        cf_country_wise_cdn_rtts_stl[country_code].append(min(total_dst_rtts))
    
    if len(total_dst_rtts) and len(each_msm['pop_rtts']):
        if statistics.mean(total_dst_rtts) > statistics.mean(each_msm['pop_rtts']):
            cf_country_wise_pop_rtts_stl[country_code].append(min(each_msm['pop_rtts']))



In [ ]:
del cf_acc_tracert_json

# Traceroute Measurements - Akamai CDN

## Starlink

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "akamai"

full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)
print(full_msms_path)
acc_akamai_tracert_filename = "acc_trace_akamai_final_result.json"
acc_akamai_tracert_filepath = os.path.join(full_msms_path,'acc_result',acc_akamai_tracert_filename)
with open(acc_akamai_tracert_filepath, "r") as json_file:
    akamai_acc_tracert_json = json.load(json_file)

In [ ]:
akamai_country_wise_cdn_rtts_stl = {}
akamai_country_wise_pop_rtts_stl = {}

idx = 0
for each_msm in akamai_acc_tracert_json:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    dst_addr = each_msm['dst_addr']
    
    total_dst_rtts = []
    for idx, hop_ip in enumerate(each_msm['all_hops'][-1]['hop_ips']):
        if hop_ip == dst_addr:
            total_dst_rtts.append(each_msm['all_hops'][-1]['hop_rtts'][idx])

    if country_code not in akamai_country_wise_cdn_rtts_stl:
        akamai_country_wise_cdn_rtts_stl[country_code] = []
        akamai_country_wise_pop_rtts_stl[country_code] = []

    if len(total_dst_rtts):
        akamai_country_wise_cdn_rtts_stl[country_code].append(min(total_dst_rtts))
    
    if len(total_dst_rtts) and len(each_msm['pop_rtts']):
        if statistics.mean(total_dst_rtts) > statistics.mean(each_msm['pop_rtts']):
            akamai_country_wise_pop_rtts_stl[country_code].append(min(each_msm['pop_rtts']))


In [ ]:
print(akamai_country_wise_cdn_rtts_stl.keys())
print(akamai_country_wise_pop_rtts_stl.keys())

In [ ]:
## Add ZM measurements which were filtered out because of intermediate hops not being visible - but still got response from destination CDN server 
user_type = "Starlink"
msm_type = "traceroute"
server = "akamai"
country_to_add = 'ZM'
akamai_country_cdn_rtts = {}
country_total_msms = 0

if os.path.exists(prb_map_filename):
    with open(prb_map_filename, "r") as json_file:
        prb_map_dict = json.load(json_file)
    print("Prb Map JSON file loaded successfully")
else:
    print("Error: No Prb Map JSON file found")

full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)
print(full_msms_path)


ip_to_asn = {}

first_hop_filtered_json_list = []
non_starlink_ip_set = set ()

bad_probe_data_set = set()


file_index = 1
num_files = len(glob.glob(os.path.join(full_msms_path, '*.json')))
print(num_files)

for filename in glob.glob(os.path.join(full_msms_path, '*.json')):

    count_new_items = 0
    f = open(filename)
    print(filename)
    data = json.load(f)
    
    i=0
    rows=0
    for idx in tqdm(range(0,len(data))):
        test = data[idx]

        if 'result' in test:
            # check probe id and server address in same country
            # by checking set1_ripe_atlas_probe_dict ,or
            # by querying RIPE Atlas API
            prb_id = str(test['prb_id'])
            msm_date = datetime.datetime.fromtimestamp(test['timestamp']).strftime('%Y-%m-%d')
            client_ip = test['from']
            if not is_valid_ipv4(client_ip):
                continue
            ip_version = check_ip_version(client_ip)
            country_code = None
            in_net_hops = []
            gs_rtts = []
            gw_rtts = []
            last_hop_rtts = []
            pop_rtts = []
            ixp_rtts = []
            gs_hop_id = 0
            gs_hop_ip = ''
            gw_hop_id = 0
            gw_hop_ips = []
            pop_hop_id = 0
            pop_hop_ips = []
            ixp_hop_id = 0
            ixp_hop_ips = []
            all_hops = []
            if 'dst_addr' not in test:
                continue
            dst_addr = test['dst_addr']
            prb_in_map =  False
            asn_info = None
            country_code = None

            if prb_id in prb_map_dict:
                if msm_date in prb_map_dict[prb_id]:
                    if client_ip in prb_map_dict[prb_id][msm_date]:
                        prb_in_map = True
                        asn_info = prb_map_dict[prb_id][msm_date][client_ip]['asn']
                        country_code = prb_map_dict[prb_id][msm_date][client_ip]['country_code']
                        # print(f"Found probe details for {prb_id} from prb_map for {msm_date} with ip {client_ip}")
            
            # Request probe details
            if not prb_in_map:
                print(f"Requesting probe details for {prb_id} from ripe atlas api for {msm_date} with ip {client_ip}")
                r = requests.get(f'https://atlas.ripe.net/api/v2/probes/archive/?probe={prb_id}&date={msm_date}', allow_redirects=True)

                if not r.ok:
                    # print(f"Not getting appropriate probe response. Status Code: {r.status_code}")
                    continue
                probe_data = r.json()

                if 'results' not in probe_data:
                    # print('No results key in probe_data. Trying alternative sources')
                    asn_info, _ = asndb.lookup(client_ip)
                    ip_geo_details = find_ip(client_ip,ip_to_country_df)
                    if asn_info and ip_geo_details['country_code']:
                        if prb_id not in prb_map_dict:
                                prb_map_dict[prb_id] = {}
                        if msm_date not in prb_map_dict[prb_id]:
                            prb_map_dict[prb_id][msm_date] = {}
                        if client_ip not in prb_map_dict[prb_id][msm_date]:
                            prb_map_dict[prb_id][msm_date][client_ip] = {}
                        prb_map_dict[prb_id][msm_date][client_ip]['country_code'] = ip_geo_details['country_code']
                        prb_map_dict[prb_id][msm_date][client_ip]['asn'] = asn_info
                        country_code = ip_geo_details['country_code']
                    else:
                        bad_probe_data_set.add(prb_id)
                        continue
                if len(probe_data['results']) <= 0:
                    # print("Empty results list in probe_data. Trying alternative sources")
                    asn_info, _ = asndb.lookup(client_ip)
                    ip_geo_details = find_ip(client_ip,ip_to_country_df)
                    if asn_info and ip_geo_details['country_code']:
                        if prb_id not in prb_map_dict:
                                prb_map_dict[prb_id] = {}
                        if msm_date not in prb_map_dict[prb_id]:
                            prb_map_dict[prb_id][msm_date] = {}
                        if client_ip not in prb_map_dict[prb_id][msm_date]:
                            prb_map_dict[prb_id][msm_date][client_ip] = {}
                        prb_map_dict[prb_id][msm_date][client_ip]['country_code'] = ip_geo_details['country_code']
                        prb_map_dict[prb_id][msm_date][client_ip]['asn'] = asn_info
                        country_code = ip_geo_details['country_code']
                    else:
                        bad_probe_data_set.add(prb_id)
                        continue

                if 'results' in probe_data:    
                    for probe_details in probe_data['results']:
                        # Find appropriate ASN based on client ip address seen in measurement
                        # print(probe_details)
                        asn_info = 0
                        if ip_version == "IPv4":
                            if probe_details['address_v4'] is not None and probe_details['address_v4'] == client_ip:
                                asn_info = probe_details['asn_v4']
                        elif ip_version == "IPv6":
                            if probe_details['address_v6'] is not None and probe_details['address_v6'] == client_ip:
                                asn_info = probe_details['asn_v6']

                        
                        if asn_info == 0:
                            asn_info, _ = asndb.lookup(client_ip)
                            # print(f"ASN info blank in RIPE Atlas database for prb_id: {prb_id}. ASNDB answer -> asn: {asn_info}")


                        if asn_info is not None and asn_info != 0:
                            country_code = probe_details['country_code']
                            lat  = probe_details['geometry']['coordinates'][1]
                            lon  = probe_details['geometry']['coordinates'][0]

                            if prb_id not in prb_map_dict:
                                prb_map_dict[prb_id] = {}
                            if msm_date not in prb_map_dict[prb_id]:
                                prb_map_dict[prb_id][msm_date] = {}
                            if client_ip not in prb_map_dict[prb_id][msm_date]:
                                prb_map_dict[prb_id][msm_date][client_ip] = {}
                            prb_map_dict[prb_id][msm_date][client_ip]['country_code'] = country_code
                            prb_map_dict[prb_id][msm_date][client_ip]['asn'] = asn_info
                            prb_map_dict[prb_id][msm_date][client_ip]['latitude'] = lat
                            prb_map_dict[prb_id][msm_date][client_ip]['longitude'] = lon

                            break

            asn_info, _ = asndb.lookup(client_ip)
            if asn_info not in [14593,45700]:
                # print(f"Not a Starlink measurement. asndb is {asn_info}. Client IP is {client_ip}")
                non_starlink_ip_set.add(client_ip)
                continue
            
            if country_code != country_to_add:
                continue
            
            # print("Checking ZM msm")
            # add all hops
            for hop in test['result']:
                this_hop_ips = []
                this_hop_rtts = []

                if 'result' in hop:
                    for each_result in hop['result']:
                        if 'from' in each_result:
                            if 'rtt' in each_result:
                                this_hop_ips.append(each_result['from'])
                                this_hop_rtts.append(each_result['rtt'])
                    if len(this_hop_rtts):
                        all_hops.append({
                            "hop_id" : hop['hop'],
                            "hop_ips" : this_hop_ips,
                            "hop_rtts" : this_hop_rtts
                        })
            
            total_dst_rtts = []
            if len(all_hops):
                for idx, hop_ip in enumerate(all_hops[-1]['hop_ips']):
                    if hop_ip == dst_addr:
                        total_dst_rtts.append(all_hops[-1]['hop_rtts'][idx])
                if country_code not in akamai_country_cdn_rtts:
                    akamai_country_cdn_rtts[country_code] = []
                if len(total_dst_rtts):
                    akamai_country_cdn_rtts[country_code].append(min(total_dst_rtts))
                    country_total_msms +=1

                
    print(f'{file_index} out of {num_files} files')
    file_index = file_index + 1

    with open(prb_map_filename, "w+") as json_file:
                json.dump(prb_map_dict, json_file, indent=4)

print(f"Traceroute filtering successfully completed")
print(f"{country_to_add} msms to be added = {country_total_msms}")   



In [ ]:
akamai_country_wise_cdn_rtts_stl['ZM'] = akamai_country_cdn_rtts['ZM']

In [ ]:
del akamai_acc_tracert_json

# Traceroute Measurements - Cloudfront CDN

## Starlink

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "cloudfront"

full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)
print(full_msms_path)
acc_cloudfront_tracert_filename = "acc_trace_cloudfront_final_result.json"
acc_cloudfront_tracert_filepath = os.path.join(full_msms_path,'acc_result',acc_cloudfront_tracert_filename)
with open(acc_cloudfront_tracert_filepath, "r") as json_file:
    cloudfront_acc_tracert_json = json.load(json_file)

In [ ]:
cloudfront_country_wise_cdn_rtts_stl = {}
cloudfront_country_wise_pop_rtts_stl = {}

idx = 0
for each_msm in cloudfront_acc_tracert_json:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    dst_addr = each_msm['dst_addr']
    
    total_dst_rtts = []
    for idx, hop_ip in enumerate(each_msm['all_hops'][-1]['hop_ips']):
        if hop_ip == dst_addr:
            total_dst_rtts.append(each_msm['all_hops'][-1]['hop_rtts'][idx])


    if country_code not in cloudfront_country_wise_cdn_rtts_stl:
        cloudfront_country_wise_cdn_rtts_stl[country_code] = []
        cloudfront_country_wise_pop_rtts_stl[country_code] = []

    if len(total_dst_rtts):
        cloudfront_country_wise_cdn_rtts_stl[country_code].append(min(total_dst_rtts))
    
    if len(total_dst_rtts) and len(each_msm['pop_rtts']):
        if statistics.mean(total_dst_rtts) > statistics.mean(each_msm['pop_rtts']):
       
            cloudfront_country_wise_pop_rtts_stl[country_code].append(min(each_msm['pop_rtts']))


In [ ]:
## Add ZM measurements which were filtered out because of intermediate hops not being visible - but still got response from destination CDN server 
user_type = "Starlink"
msm_type = "traceroute"
server = "cloudfront"
country_to_add = 'ZM'
cloudfront_country_cdn_rtts = {}
country_total_msms = 0

if os.path.exists(prb_map_filename):
    with open(prb_map_filename, "r") as json_file:
        prb_map_dict = json.load(json_file)
    print("Prb Map JSON file loaded successfully")
else:
    print("Error: No Prb Map JSON file found")

full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)
print(full_msms_path)


ip_to_asn = {}

first_hop_filtered_json_list = []
non_starlink_ip_set = set ()

bad_probe_data_set = set()


file_index = 1
num_files = len(glob.glob(os.path.join(full_msms_path, '*.json')))
try: 
    for filename in glob.glob(os.path.join(full_msms_path, '*.json')):

        count_new_items = 0
        f = open(filename)
        print(filename)
        data = json.load(f)
        
        i=0
        rows=0
        for idx in tqdm(range(0,len(data))):
            test = data[idx]

            if 'result' in test:
                # check probe id and server address in same country
                # by checking set1_ripe_atlas_probe_dict ,or
                # by querying RIPE Atlas API
                prb_id = str(test['prb_id'])
                msm_date = datetime.datetime.fromtimestamp(test['timestamp']).strftime('%Y-%m-%d')
                client_ip = test['from']
                if not is_valid_ipv4(client_ip):
                    continue
                ip_version = check_ip_version(client_ip)
                country_code = None
                in_net_hops = []
                gs_rtts = []
                gw_rtts = []
                last_hop_rtts = []
                pop_rtts = []
                ixp_rtts = []
                gs_hop_id = 0
                gs_hop_ip = ''
                gw_hop_id = 0
                gw_hop_ips = []
                pop_hop_id = 0
                pop_hop_ips = []
                ixp_hop_id = 0
                ixp_hop_ips = []
                all_hops = []
                if 'dst_addr' not in test:
                    continue
                dst_addr = test['dst_addr']
                prb_in_map =  False
                asn_info = None
                country_code = None

                if prb_id in prb_map_dict:
                    if msm_date in prb_map_dict[prb_id]:
                        if client_ip in prb_map_dict[prb_id][msm_date]:
                            prb_in_map = True
                            asn_info = prb_map_dict[prb_id][msm_date][client_ip]['asn']
                            country_code = prb_map_dict[prb_id][msm_date][client_ip]['country_code']
                            # print(f"Found probe details for {prb_id} from prb_map for {msm_date} with ip {client_ip}")
                
                # Request probe details
                if not prb_in_map:
                    print(f"Requesting probe details for {prb_id} from ripe atlas api for {msm_date} with ip {client_ip}")
                    r = requests.get(f'https://atlas.ripe.net/api/v2/probes/archive/?probe={prb_id}&date={msm_date}', allow_redirects=True)

                    if not r.ok:
                        # print(f"Not getting appropriate probe response. Status Code: {r.status_code}")
                        continue
                    probe_data = r.json()

                    if 'results' not in probe_data:
                        # print('No results key in probe_data. Trying alternative sources')
                        asn_info, _ = asndb.lookup(client_ip)
                        ip_geo_details = find_ip(client_ip,ip_to_country_df)
                        if asn_info and ip_geo_details['country_code']:
                            if prb_id not in prb_map_dict:
                                    prb_map_dict[prb_id] = {}
                            if msm_date not in prb_map_dict[prb_id]:
                                prb_map_dict[prb_id][msm_date] = {}
                            if client_ip not in prb_map_dict[prb_id][msm_date]:
                                prb_map_dict[prb_id][msm_date][client_ip] = {}
                            prb_map_dict[prb_id][msm_date][client_ip]['country_code'] = ip_geo_details['country_code']
                            prb_map_dict[prb_id][msm_date][client_ip]['asn'] = asn_info
                            country_code = ip_geo_details['country_code']
                        else:
                            bad_probe_data_set.add(prb_id)
                            continue
                    if len(probe_data['results']) <= 0:
                        # print("Empty results list in probe_data. Trying alternative sources")
                        asn_info, _ = asndb.lookup(client_ip)
                        ip_geo_details = find_ip(client_ip,ip_to_country_df)
                        if asn_info and ip_geo_details['country_code']:
                            if prb_id not in prb_map_dict:
                                    prb_map_dict[prb_id] = {}
                            if msm_date not in prb_map_dict[prb_id]:
                                prb_map_dict[prb_id][msm_date] = {}
                            if client_ip not in prb_map_dict[prb_id][msm_date]:
                                prb_map_dict[prb_id][msm_date][client_ip] = {}
                            prb_map_dict[prb_id][msm_date][client_ip]['country_code'] = ip_geo_details['country_code']
                            prb_map_dict[prb_id][msm_date][client_ip]['asn'] = asn_info
                            country_code = ip_geo_details['country_code']
                        else:
                            bad_probe_data_set.add(prb_id)
                            continue

                    if 'results' in probe_data:    
                        for probe_details in probe_data['results']:
                            # Find appropriate ASN based on client ip address seen in measurement
                            # print(probe_details)
                            asn_info = 0
                            if ip_version == "IPv4":
                                if probe_details['address_v4'] is not None and probe_details['address_v4'] == client_ip:
                                    asn_info = probe_details['asn_v4']
                            elif ip_version == "IPv6":
                                if probe_details['address_v6'] is not None and probe_details['address_v6'] == client_ip:
                                    asn_info = probe_details['asn_v6']

                            
                            if asn_info == 0:
                                asn_info, _ = asndb.lookup(client_ip)
                                # print(f"ASN info blank in RIPE Atlas database for prb_id: {prb_id}. ASNDB answer -> asn: {asn_info}")


                            if asn_info is not None and asn_info != 0:
                                country_code = probe_details['country_code']
                                lat  = probe_details['geometry']['coordinates'][1]
                                lon  = probe_details['geometry']['coordinates'][0]

                                if prb_id not in prb_map_dict:
                                    prb_map_dict[prb_id] = {}
                                if msm_date not in prb_map_dict[prb_id]:
                                    prb_map_dict[prb_id][msm_date] = {}
                                if client_ip not in prb_map_dict[prb_id][msm_date]:
                                    prb_map_dict[prb_id][msm_date][client_ip] = {}
                                prb_map_dict[prb_id][msm_date][client_ip]['country_code'] = country_code
                                prb_map_dict[prb_id][msm_date][client_ip]['asn'] = asn_info
                                prb_map_dict[prb_id][msm_date][client_ip]['latitude'] = lat
                                prb_map_dict[prb_id][msm_date][client_ip]['longitude'] = lon

                                break

                asn_info, _ = asndb.lookup(client_ip)
                if asn_info not in [14593,45700]:
                    # print(f"Not a Starlink measurement. asndb is {asn_info}. Client IP is {client_ip}")
                    non_starlink_ip_set.add(client_ip)
                    continue
                

                if country_code != country_to_add:
                    continue
                
                # print("Checking ZM msm")
                # add all hops
                for hop in test['result']:
                    this_hop_ips = []
                    this_hop_rtts = []

                    if 'result' in hop:
                        for each_result in hop['result']:
                            if 'from' in each_result:
                                if 'rtt' in each_result:
                                    this_hop_ips.append(each_result['from'])
                                    this_hop_rtts.append(each_result['rtt'])
                        if len(this_hop_rtts):
                            all_hops.append({
                                "hop_id" : hop['hop'],
                                "hop_ips" : this_hop_ips,
                                "hop_rtts" : this_hop_rtts
                            })
                
                total_dst_rtts = []
                for idx, hop_ip in enumerate(all_hops[-1]['hop_ips']):
                    if hop_ip == dst_addr:
                        total_dst_rtts.append(all_hops[-1]['hop_rtts'][idx])
                if country_code not in cloudfront_country_cdn_rtts:
                    cloudfront_country_cdn_rtts[country_code] = []
                if len(total_dst_rtts):
                    cloudfront_country_cdn_rtts[country_code].append(min(total_dst_rtts))
                    country_total_msms +=1

                    
        print(f'{file_index} out of {num_files} files')
        file_index = file_index + 1
except Exception as e:
    print(e)
finally:
    with open(prb_map_filename, "w+") as json_file:
                json.dump(prb_map_dict, json_file, indent=4)

print(f"Traceroute filtering successfully completed")
print(f"{country_to_add} msms to be added = {country_total_msms}")   



In [ ]:
cloudfront_country_wise_cdn_rtts_stl['ZM'] = cloudfront_country_cdn_rtts['ZM']

In [ ]:
del cloudfront_acc_tracert_json

# Traceroute Measurements - Add Controlled Nodes measurements

In [ ]:
controlled_nodes_traceroute_files_path = f"{controlled_top_level_folder_path}/traceroute/08-27-2025-12-31-18_WebTrace_files_list"

cdns_list = ["cloudflare", "akamai", "cloudfront"]
countries_added_set = set()
hop_ip_match = 0
hop_ip_no_match = 0
for cdn_name in cdns_list:
    acc_tracert_filename = f"controlled_nodes_acc_trace_{cdn_name}_final_result_late.json"
    acc_tracert_filepath = os.path.join(controlled_nodes_traceroute_files_path,'acc_result',acc_tracert_filename)
    with open(acc_tracert_filepath, "r") as json_file:
        acc_tracert_json = json.load(json_file)

    idx = 0
    for each_msm in acc_tracert_json:
        country_code = each_msm['country_code']
        prb_id = each_msm['prb_id']
        dst_addr = each_msm['dst_addr']
        unix_timestamp = each_msm['timestamp']

        threshold_date = datetime.datetime(2025,8,11)
        timestamp_date = datetime.datetime.fromtimestamp(unix_timestamp)
        
        if timestamp_date < threshold_date:
            continue

        prefix = None
        if cdn_name == "cloudflare":
            prefix = "cf"
        else:
            prefix = cdn_name
        cdn_rtts_stl = eval(f"{prefix}_country_wise_cdn_rtts_stl")
        pop_rtts_stl = eval(f"{prefix}_country_wise_pop_rtts_stl")


        if country_code not in cdn_rtts_stl:
            cdn_rtts_stl[country_code] = []
        if country_code not in pop_rtts_stl:
            pop_rtts_stl[country_code] = []

        total_dst_rtts = []
        for idx, hop_ip in enumerate(each_msm['all_hops'][-1]['hop_ips']):
            if hop_ip == dst_addr:
                total_dst_rtts.append(each_msm['all_hops'][-1]['hop_rtts'][idx])
                hop_ip_match += 1
            else:
                print(f"Hop IP: {hop_ip}, DST IP: {dst_addr}")
                hop_ip_no_match += 1

        if len(total_dst_rtts):
            cdn_rtts_stl[country_code].append(min(total_dst_rtts))

        if len(total_dst_rtts) and len(each_msm['pop_rtts']):
            if statistics.mean(total_dst_rtts) > statistics.mean(each_msm['pop_rtts']):
                pop_rtts_stl[country_code].append(min(each_msm['pop_rtts']))
                countries_added_set.add(country_code)

In [ ]:
def adjust_box(plot):
    for i in range(2):
        plt.setp(plot['boxes'][i], facecolor=cols[i], linewidth=1)
        plt.setp(plot['medians'][i], color='yellow')

countries = ["US", "CA" ,"DE", "HU", "CL", "CO", "AU", "NZ" ,"PH", "GU", "BJ", "ZW" , "ZM", "MG"]

fig, ax = plt.subplots(figsize=(4.66, 2))

pos1 = np.arange(0, 4 * len(countries), 4)
pos2 = np.arange(1, 4 * len(countries) + 1, 4)
pos3 = np.arange(2, 4 * len(countries) + 2, 4)

cf_country_wise_cdn_rtts_stl_filtered = {}
akamai_country_wise_cdn_rtts_stl_filtered = {}
cloudfront_country_wise_cdn_rtts_stl_filtered = {}

# Filter out outliers less than 15 ms -- for Starlink msms 
for country in countries:
    cf_country_wise_cdn_rtts_stl_filtered[country] = [cf_rtt for cf_rtt in cf_country_wise_cdn_rtts_stl[country] if cf_rtt > 15]
    akamai_country_wise_cdn_rtts_stl_filtered[country] = [ak_rtt for ak_rtt in akamai_country_wise_cdn_rtts_stl[country] if ak_rtt > 15]
    cloudfront_country_wise_cdn_rtts_stl_filtered[country] = [cfront_rtt for cfront_rtt in cloudfront_country_wise_cdn_rtts_stl[country] if cfront_rtt > 15]

ax.boxplot([cf_country_wise_cdn_rtts_stl_filtered[country] for country in countries], positions=pos1, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[1], lw=1), medianprops=dict(color="yellow"))
ax.boxplot([akamai_country_wise_cdn_rtts_stl_filtered[country] for country in countries], positions=pos2, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[2], lw=1), medianprops=dict(color="yellow"))
ax.boxplot([cloudfront_country_wise_cdn_rtts_stl_filtered[country] for country in countries], positions=pos3, patch_artist=True,
           showfliers=False, boxprops=dict(facecolor=cols[4], lw=1), medianprops=dict(color="yellow"))
xspan = 4
[ax.axvspan(i * xspan - 1, i * xspan - 1 + xspan, facecolor="k", alpha=0.2)
    for i in range(len(countries))
    if i % 2 == 1]

ax.set_xticks(np.arange(1.0, 4 * len(countries), 4))
ax.set_xticklabels(countries)
ax.set_ylabel("RTT [ms]")
ax.set_yticks(np.arange(0, 460, 50))
ax.set_xlim(-1,55)
ax.xaxis.get_major_formatter()._usetex = False
ax.yaxis.get_major_formatter()._usetex = False

handles = [Patch(facecolor=cols[1]),Patch(facecolor=cols[2]),Patch(facecolor=cols[4])]
labels = ["Cloudflare", "Akamai", "Cloudfront"]

ax.legend(handles, labels, loc="upper left", fontsize="small", edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4)
plt.grid(True, axis="y", linestyle="--")
plt.ylim(0,450)
fig.tight_layout()

plt.savefig(f"{figs_top_level_folder_path}/cdn_comparison_rtt_selected_countries.pdf", bbox_inches = "tight", pad_inches = 0.1)
plt.savefig(f"{figs_top_level_folder_path}/cdn_comparison_rtt_selected_countries.svg", bbox_inches = "tight", pad_inches = 0.1)
plt.show()

# Overall Measurements - CDN vs DNS vs PoP latency

## Starlink - accumulated

In [ ]:
common_countries_trace_msms_stl = set(cf_country_wise_cdn_rtts_stl.keys()).intersection(set(akamai_country_wise_cdn_rtts_stl.keys())).intersection(set(cloudfront_country_wise_cdn_rtts_stl.keys()))
common_countries_dns_msms_stl = set(cf_country_wise_dns_CHAOS_response_time_dict.keys()).intersection(google_country_wise_dns_CHAOS_response_time_dict.keys()).intersection(quad9_country_wise_dns_CHAOS_response_time_dict.keys())
common_countries_all_msms_stl = common_countries_trace_msms_stl.intersection(common_countries_dns_msms_stl)
print(common_countries_all_msms_stl)

In [ ]:
cdn_country_wise_cdn_rtts_stl = {}
cdn_country_wise_pop_rtts_stl = {}
dns_country_wise_dns_CHAOS_response_time_dict_stl = {}

for country in common_countries_all_msms_stl:
    cdn_country_wise_cdn_rtts_stl[country] = cf_country_wise_cdn_rtts_stl[country] + akamai_country_wise_cdn_rtts_stl[country] + cloudfront_country_wise_cdn_rtts_stl[country]
    cdn_country_wise_pop_rtts_stl[country] = cf_country_wise_pop_rtts_stl[country] + akamai_country_wise_pop_rtts_stl[country] + cloudfront_country_wise_pop_rtts_stl[country]
    dns_country_wise_dns_CHAOS_response_time_dict_stl[country] = cf_country_wise_dns_CHAOS_response_time_dict[country] + google_country_wise_dns_CHAOS_response_time_dict[country] + quad9_country_wise_dns_CHAOS_response_time_dict[country]

In [ ]:
cf_all_rtt = []
akamai_all_rtt = []
cloudfront_all_rtt = []

for country in common_countries_all_msms_stl:
    cf_all_rtt.append(statistics.median(cf_country_wise_cdn_rtts_stl[country]))
    akamai_all_rtt.append(statistics.median(akamai_country_wise_cdn_rtts_stl[country]))
    cloudfront_all_rtt.append(statistics.median(cloudfront_country_wise_cdn_rtts_stl[country]))

cf_median_rtt = statistics.mean(cf_all_rtt)
akamai_median_rtt = statistics.mean(akamai_all_rtt)
cloudfront_median_rtt = statistics.mean(cloudfront_all_rtt)

print(f"Cloudflare Mean CDN RTT: {cf_median_rtt}")
print(f"Akamai Mean CDN RTT: {akamai_median_rtt}")
print(f"Cloudfront Mean CDN RTT: {cloudfront_median_rtt}")

In [ ]:
pop_cdn_percent_stl = []
pop_dns_percent_stl = []
pop_stl = []
e2e_stl = []
dns_stl = []

dns_bar_col = cols[0]
cdn_bar_col = cols[1]

chosen_countries = ["US","CA", "DE", "HU", "CL", "CO", "PH" ,"GU", "BJ", "ZM" ,"ZW"]

cdn_country_wise_cdn_rtts_stl_filtered = {}
cdn_country_wise_pop_rtts_stl_filtered = {}
dns_country_wise_dns_CHAOS_response_time_dict_stl_filtered = {}

# Filter out outliers less than 15 ms
for country in countries:
    cdn_country_wise_cdn_rtts_stl_filtered[country] = [cdn_rtt for cdn_rtt in cdn_country_wise_cdn_rtts_stl[country] if cdn_rtt > 15]
    cdn_country_wise_pop_rtts_stl_filtered[country] = [pop_rtt for pop_rtt in cdn_country_wise_pop_rtts_stl[country] if pop_rtt > 15]
    dns_country_wise_dns_CHAOS_response_time_dict_stl_filtered[country] = [dns_rt for dns_rt in dns_country_wise_dns_CHAOS_response_time_dict_stl[country] if dns_rt > 15]

for country in chosen_countries[::-1]:

    e2e_rtt_stl = np.percentile(cdn_country_wise_cdn_rtts_stl_filtered[country],50)
    pop_rtt_stl = np.percentile(cdn_country_wise_pop_rtts_stl_filtered[country],50)
    dns_rtt_stl = np.percentile(dns_country_wise_dns_CHAOS_response_time_dict_stl_filtered[country],50)


    e2e_stl.append(e2e_rtt_stl)
    pop_stl.append(pop_rtt_stl)
    dns_stl.append(dns_rtt_stl)
    pop_cdn_percent_stl.append(100 * pop_rtt_stl/e2e_rtt_stl)
    pop_dns_percent_stl.append(100 * pop_rtt_stl/dns_rtt_stl)


bar_width = 1.2
group_space = 0.5
bar_spacing = 0.2
y_pos = np.arange(len(chosen_countries)) * (2 * bar_width + bar_spacing + group_space)
dns_bar_positions = y_pos

cdn_bar_positions = y_pos + bar_width + bar_spacing


# Plot
fig, ax = plt.subplots(figsize=(4.66, 4))
bars_dns = ax.barh(dns_bar_positions, dns_stl, bar_width, color=dns_bar_col, edgecolor='black')
bars_e2e = ax.barh(cdn_bar_positions, e2e_stl, bar_width, color=cdn_bar_col, edgecolor='black')

# Add percentage labels at the end of bars
for bar, pop_rtt ,percent in zip(bars_e2e, pop_stl, pop_cdn_percent_stl):
    striped_rect = patches.Rectangle(
        (0,bar.get_y()),
        pop_rtt,
        bar.get_height(),
        hatch='\\\\\\\\',
        facecolor='none',
        edgecolor='black',
        linewidth=0.5
    )
    ax.add_patch(striped_rect)
    ax.text(bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,
            f"{percent:.1f}\%", va='center', fontsize="small", color =cdn_bar_col)

for bar, pop_rtt ,percent in zip(bars_dns, pop_stl, pop_dns_percent_stl):
    striped_rect = patches.Rectangle(
        (0,bar.get_y()),
        pop_rtt,
        bar.get_height(),
        hatch='\\\\\\\\',
        facecolor='none',
        edgecolor='black',
        linewidth=0.5
    )
    ax.add_patch(striped_rect)
    ax.text(bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,
            f"{percent:.1f}\%", va='center', fontsize="small", color =dns_bar_col)

# Labels and aesthetics
ax.set_xlabel("RTT to PoP vs DNS vs CDN [ms]")
ax.set_yticks(y_pos + bar_width / 2 + bar_spacing / 2)
ax.set_yticklabels(chosen_countries[::-1])

ax.set_xlim(0, 130)  # extra room for labels
ax.grid(axis='x', linestyle='--', alpha=0.6)

handles = [Patch(facecolor=cdn_bar_col,edgecolor='black'),Patch(facecolor=dns_bar_col,edgecolor='black'),Patch(facecolor='none',edgecolor='black',hatch='\\\\\\\\')]
labels = ["CDN", "DNS", "PoP"]
ax.legend(handles, labels, loc="upper right", fontsize="small", edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4)
plt.xlim(0,110)
plt.tight_layout()
plt.savefig(f"{figs_top_level_folder_path}/overall_pop_vs_dns_vs_cdn_rtt_stl.pdf", bbox_inches = "tight", pad_inches = 0)
plt.show()


## Newer PoPs - Before vs After

### Old measurements

In [ ]:
cdns_list = ["cloudflare", "akamai", "cloudfront"]

In [ ]:
cf_prb_wise_cdn_rtts_stl_old = {}
cf_prb_wise_pop_rtts_stl_old = {}
cf_prb_wise_pop_cdn_rtts_stl_old = {}

akamai_prb_wise_cdn_rtts_stl_old = {}
akamai_prb_wise_pop_rtts_stl_old = {}
akamai_prb_wise_pop_cdn_rtts_stl_old = {}

cloudfront_prb_wise_cdn_rtts_stl_old = {}
cloudfront_prb_wise_pop_rtts_stl_old = {}
cloudfront_prb_wise_pop_cdn_rtts_stl_old = {}

prb_pop_ips_old = {}

user_type = "Starlink"
msm_type = "traceroute"


for server in cdns_list:

    cdn_rtts_stl_old = None
    pop_rtts_stl_old = None
    cdn_name = server
    if server == 'cloudflare':
        cdn_name = "cf"
    cdn_rtts_stl_old = eval(f"{cdn_name}_prb_wise_cdn_rtts_stl_old")
    pop_rtts_stl_old = eval(f"{cdn_name}_prb_wise_pop_rtts_stl_old")
    pop_cdn_rtts_old = eval(f"{cdn_name}_prb_wise_pop_cdn_rtts_stl_old")

    full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)

    acc_tracert_filename = f"acc_trace_{server}_final_result.json"
    acc_tracert_filepath = os.path.join(full_msms_path,'acc_result',acc_tracert_filename)
    print(acc_tracert_filepath)
    with open(acc_tracert_filepath, "r") as json_file:
        acc_tracert_json = json.load(json_file)

    idx = 0
    for each_msm in acc_tracert_json:
        country_code = each_msm['country_code']
        prb_id = each_msm['prb_id']

        if prb_id not in prb_pop_ips_old:
            prb_pop_ips_old[prb_id] = []

        pop_ips = each_msm['pop_hop_ip']
        for ip in pop_ips:
            if ip not in prb_pop_ips_old[prb_id]:
                prb_pop_ips_old[prb_id].append(ip)

        if prb_id not in cdn_rtts_stl_old:
            cdn_rtts_stl_old[prb_id] = []
            pop_rtts_stl_old[prb_id] = []
            pop_cdn_rtts_old[prb_id] = []
        
        if len(each_msm['total_rtts']) and len(each_msm['pop_rtts']):
            if statistics.mean(each_msm['total_rtts']) > statistics.mean(each_msm['pop_rtts']):
                min_cdn_rtt = min(each_msm['total_rtts'])
                min_pop_rtt = min(each_msm['pop_rtts'])
                cdn_rtts_stl_old[prb_id].append(min_cdn_rtt)
                pop_rtts_stl_old[prb_id].append(min_pop_rtt)
                pop_cdn_rtt = min_cdn_rtt - min_pop_rtt

                if pop_cdn_rtt:
                    pop_cdn_rtts_old[prb_id].append(pop_cdn_rtt)

### New Measurements

In [ ]:
cf_prb_wise_cdn_rtts_stl_new = {}
cf_prb_wise_pop_rtts_stl_new = {}
cf_prb_wise_pop_cdn_rtts_stl_new = {}

akamai_prb_wise_cdn_rtts_stl_new = {}
akamai_prb_wise_pop_rtts_stl_new = {}
akamai_prb_wise_pop_cdn_rtts_stl_new = {}

cloudfront_prb_wise_cdn_rtts_stl_new = {}
cloudfront_prb_wise_pop_rtts_stl_new = {}
cloudfront_prb_wise_pop_cdn_rtts_stl_new = {}

prb_pop_ips_new = {}

user_type = "Starlink"
msm_type = "traceroute"


for server in cdns_list:

    cdn_rtts_stl_new = None
    pop_rtts_stl_new = None
    cdn_name = server
    if server == 'cloudflare':
        cdn_name = "cf"
    cdn_rtts_stl_new = eval(f"{cdn_name}_prb_wise_cdn_rtts_stl_new")
    pop_rtts_stl_new = eval(f"{cdn_name}_prb_wise_pop_rtts_stl_new")
    pop_cdn_rtts_new = eval(f"{cdn_name}_prb_wise_pop_cdn_rtts_stl_new")

    full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)

    acc_tracert_filename = f"acc_trace_{server}_final_result.json"
    acc_tracert_filepath = os.path.join(full_msms_path,'acc_result',acc_tracert_filename)
    print(acc_tracert_filepath)
    with open(acc_tracert_filepath, "r") as json_file:
        acc_tracert_json = json.load(json_file)

    idx = 0
    for each_msm in acc_tracert_json:
        country_code = each_msm['country_code']
        prb_id = each_msm['prb_id']

        if prb_id not in prb_pop_ips_new:
            prb_pop_ips_new[prb_id] = []

        pop_ips = each_msm['pop_hop_ip']
        for ip in pop_ips:
            if ip not in prb_pop_ips_new[prb_id]:
                prb_pop_ips_new[prb_id].append(ip)

        if prb_id not in cdn_rtts_stl_new:
            cdn_rtts_stl_new[prb_id] = []
            pop_rtts_stl_new[prb_id] = []
            pop_cdn_rtts_new[prb_id] = []
        
        if len(each_msm['total_rtts']) and len(each_msm['pop_rtts']):
            if statistics.mean(each_msm['total_rtts']) > statistics.mean(each_msm['pop_rtts']):
                min_cdn_rtt = min(each_msm['total_rtts'])
                min_pop_rtt = min(each_msm['pop_rtts'])
                cdn_rtts_stl_new[prb_id].append(min_cdn_rtt)
                pop_rtts_stl_new[prb_id].append(min_pop_rtt)
                pop_cdn_rtt = min_cdn_rtt - min_pop_rtt

                if pop_cdn_rtt:
                    pop_cdn_rtts_new[prb_id].append(pop_cdn_rtt)

In [ ]:
del acc_tracert_json

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
rcParams['font.size'] = 10

probe_id = 65337 ## RIPE Atlas probe for Guam
entity_colors = {
    "CDN" : cols[4],
    "Cloudflare" : cols[3],
    "Akamai" : cols[4],
    "Cloudfront" : cols[2],
    "PoP" : cols[1]
}

all_pop_rtts_stl_old = {}
all_pop_rtts_stl_new = {}

all_pop_rtts_stl_new[probe_id] = []
all_pop_rtts_stl_old[probe_id] = []

all_cdn_rtts_stl_old = {}
all_cdn_rtts_stl_new = {}

all_cdn_rtts_stl_new[probe_id] = []
all_cdn_rtts_stl_old[probe_id] = []

rtt_params = {
    "Cloudflare" : "cf_prb_wise_", 
    "Akamai" : "akamai_prb_wise_", 
    "Cloudfront" : "cloudfront_prb_wise_", 
    "PoP" : "all_pop_rtts_stl_"
}

all_rtt_params = {
    "CDN" : "all_cdn_rtts_stl_",
    "PoP" : "all_pop_rtts_stl_"
}

for cdn in list(rtt_params.keys()):
    if cdn != "PoP":
        pop_prb_wise_rtts_new = eval(rtt_params[cdn]+"pop_rtts_stl_new")
        all_pop_rtts_stl_new[probe_id] += pop_prb_wise_rtts_new[probe_id]

        pop_prb_wise_rtts_old = eval(rtt_params[cdn]+"pop_rtts_stl_old")
        all_pop_rtts_stl_old[probe_id] += pop_prb_wise_rtts_old[probe_id]

        cdn_prb_wise_rtts_new = eval(rtt_params[cdn]+"cdn_rtts_stl_new")
        all_cdn_rtts_stl_new[probe_id] += cdn_prb_wise_rtts_new[probe_id]

        cdn_prb_wise_rtts_old = eval(rtt_params[cdn]+"cdn_rtts_stl_old")
        all_cdn_rtts_stl_old[probe_id] += cdn_prb_wise_rtts_old[probe_id]


fig, ax = plt.subplots(figsize=(4.66, 2))


handles = []
labels = []
for entity in all_rtt_params.keys():
    if entity in ["CDN", "PoP"]:
        rtt_prb_wise_old = []
        rtt_prb_wise_new = []

        rtt_prb_wise_old = eval(all_rtt_params[entity]+"old")

        rtt_BEFORE_list = rtt_prb_wise_old[probe_id]
        if rtt_BEFORE_list:
 
            xs = np.sort(rtt_BEFORE_list)
            ys = np.arange(1, len(xs) + 1) / len(xs)
            indices = []
            current = xs[0]
            for i, x in enumerate(xs): # only take max y value at each x value to smoothen out the graph
                if x != current:
                    current = x
                    indices.append(i - 1)
            indices.append(len(ys) - 1)
            xs = sorted(set(xs))
            ys = [ys[i] for i in indices]
            ax.plot(xs, ys, label=f"{entity}", color=entity_colors[entity], linestyle="dotted")


        rtt_prb_wise_new = eval(all_rtt_params[entity]+"new")

        rtt_AFTER_list = rtt_prb_wise_new[probe_id]
        if rtt_AFTER_list:

            xs = np.sort(rtt_AFTER_list)
            ys = np.arange(1, len(xs) + 1) / len(xs)
            indices = []
            current = xs[0]
            for i, x in enumerate(xs): # only take max y value at each x value to smoothen out the graph
                if x != current:
                    current = x
                    indices.append(i - 1)
            indices.append(len(ys) - 1)
            xs = sorted(set(xs))
            ys = [ys[i] for i in indices]
            ax.plot(xs, ys, label=f"{entity}", color=entity_colors[entity], linestyle="solid")

        handles.append(Patch(facecolor=entity_colors[entity]))
        labels.append(entity)

ax.set_xlabel("RTT [ms]")
ax.set_ylabel("Percentile")
ax.set_yticks(np.arange(0, 1.20, 0.25))
ax.set_xticks(np.arange(0, 600, 50))
ax.xaxis.get_major_formatter()._usetex = False
ax.yaxis.get_major_formatter()._usetex = False


solid_line = Line2D([0], [0], color='black', linestyle='-', label='Starlink')
dashed_line = Line2D([0], [0], color='black', linestyle=':', label='Terrestrial')

net_handles = [solid_line, dashed_line]
net_labels = ['AFTER', 'BEFORE']

country_legend  = ax.legend(handles, labels, loc="lower right",  fontsize="small", edgecolor="k",  handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4)
ax.add_artist(country_legend)
ax.legend(net_handles, net_labels,loc="lower right", bbox_to_anchor=(0.65, 0.94), fontsize="x-small", edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3,frameon=False, fancybox=False, ncol=4) # bbox_to_anchor=(0.25, 0.91), 

plt.grid(True, axis='y', linestyle='--', alpha=0.7, linewidth=0.5)

ax.set_xlim(20,300)

fig.tight_layout()
plt.savefig(f"{figs_top_level_folder_path}/cf_cdn_Guam_BEFORE_vs_AFTER_cdf.pdf", bbox_inches = "tight", pad_inches = 0)
plt.savefig(f"{figs_top_level_folder_path}/cf_cdn_Guam_BEFORE_vs_AFTER_cdf.svg", bbox_inches = "tight", pad_inches = 0)
plt.show()

## Traceroute server selection

### Cloudflare - Late 2025

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "cloudflare"


acc_result_filename = f"acc_trace_{server}_final_result.json"
full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)

cf_acc_tracert_json = None
with open(filepath, "r") as json_file:
    cf_acc_tracert_json = json.load(json_file)
    print("Accumulated Traceroute JSON file loaded successfully")

In [ ]:
probes_cf_path = {}
all_public_ips = []
ip_to_country_dict = {}


shortlisted_countries = ["US", "CA" ,"DE", "HU", "CL", "CO", "AU", "NZ" ,"PH", "GU", "BJ", "ZW" , "ZM", "MG"]

class Hop:
    def __init__(self,ip,type,min_rtt,asn,hostname,country,location):
        self.ip = ip
        self.type = type
        self.min_rtt = min_rtt
        self.asn = asn
        self.hostname = hostname
        self.country = country
        self.location = location
        self.children = []

    def add_children(self,child_node):
        self.children.append(child_node)
    
    def has_children(self):
        return len(self.children) > 0
    
    def get_children(self):
        return self.children


for idx, each_msm in enumerate(cf_acc_tracert_json,start=0):
        print(f"Index: {idx} out of {len(cf_acc_tracert_json)}")

        pop_ip_set = set(each_msm['pop_hop_ip'])
        cdn_ip_set = set(each_msm['all_hops'][-1]['hop_ips'])

        if len(pop_ip_set) == 0:
            continue
        
        if len(pop_ip_set) != 1:
            print(f"More than 2 PoP IPs in the same hop. Probe id: {prb_id}, domain: {each_msm['domain']}")
            pop_ip_set = {list(pop_ip_set)[0],}
            print(f"Continuing with PoP IP: {list(pop_ip_set)[0]}")
            # continue

        if each_msm['country_code'] not in shortlisted_countries:
            continue

        ## Create new probe in probes if not done already
        prb_id = each_msm['prb_id']
        if prb_id not in probes_cf_path:
            probes_cf_path[prb_id] = {
                'id' : prb_id,
                'country_code' : each_msm['country_code'],
                'paths': {},
            }

        ## Create first hop for the probe as starting with the pop node if not done already
        pop_ip = next(iter(pop_ip_set))
        pop_asn = 14593
        domain = each_msm['domain']
        if domain not in probes_cf_path[prb_id]['paths']:
            probes_cf_path[prb_id]['paths'][domain] = {}
        if pop_ip not in probes_cf_path[prb_id]['paths'][domain]:
                country = None
                if pop_ip in ip_to_country_dict:
                    country = ip_to_country_dict[pop_ip]
                else:
                    geo_details = find_ip(pop_ip,ip_to_country_df)
                    if geo_details is not None:
                        if 'country_code' in geo_details:
                            country = geo_details['country_code']
                            ip_to_country_dict[pop_ip] = country
                    else:
                        ip_to_country_dict[pop_ip] = None
                probes_cf_path[prb_id]['paths'][domain][pop_ip] = Hop(
                    ip=pop_ip,
                    type='PoP',
                    min_rtt=min(each_msm['pop_rtts']),
                    asn=pop_asn,
                    hostname=None,
                    country=country,
                    location=None,
                )
        else:
            this_min_rtt = min(each_msm['pop_rtts'])
            prev_min_rtt = probes_cf_path[prb_id]['paths'][domain][pop_ip].min_rtt
            if this_min_rtt < prev_min_rtt:
                probes_cf_path[prb_id]['paths'][domain][pop_ip].min_rtt = this_min_rtt

        pop_reached = False
        hopNode = probes_cf_path[prb_id]['paths'][domain][pop_ip]
        
        ## Go through all the hops
        for idx, each_hop in enumerate(each_msm['all_hops']):
            hop_ip_set = set(each_hop['hop_ips'])
            hop_id = each_hop['hop_id']
            min_hop_rtt = min(each_hop['hop_rtts'])

            if pop_ip_set.intersection(hop_ip_set):
                pop_reached = True
                continue

            
            
            if pop_reached:
                if len(hop_ip_set) != 1:
                    print(f"More than 2 destinations in the same hop. Probe id: {prb_id}, domain: {each_msm['domain']}, hop id: {hop_id}")
                    
                    print("Check if the hop ips are part of the same network address")
                    hop_ip_network_count = {}
                    hop_ip_network_min_rtt = {}
                    for uni_idx, unique_hop_ip in enumerate(each_hop['hop_ips']):
                        unique_hop_ip_obj = ipaddress.ip_interface(unique_hop_ip + '/24')
                        unique_hop_network = str(unique_hop_ip_obj.network.network_address)

                        if unique_hop_network not in hop_ip_network_count:
                            hop_ip_network_count[unique_hop_network] = 1
                            hop_ip_network_min_rtt[unique_hop_network] = each_hop['hop_rtts'][uni_idx]
                        else:
                            continue
                        for each_idx, each_ip in enumerate(each_hop['hop_ips']):
                            if each_idx == uni_idx:
                                continue
                            each_ip_obj = ipaddress.ip_interface(each_ip + '/24')
                            each_ip_network = str(each_ip_obj.network.network_address)
                            if each_ip_network == unique_hop_network:
                                hop_ip_network_count[unique_hop_network] += 1
                                if each_hop['hop_rtts'][each_idx] <  hop_ip_network_min_rtt[unique_hop_network]:
                                    hop_ip_network_min_rtt[unique_hop_network] = each_hop['hop_rtts'][each_idx]
                    
                    max_count = max(hop_ip_network_count.values())
                    if max_count < 2:
                        print("Could not find a majority network address. Skipping measurement.")
                        break
                    max_key = max(hop_ip_network_count, key=hop_ip_network_count.get)
                    hop_ip_set = {max_key,}
                    min_hop_rtt = hop_ip_network_min_rtt[max_key]


                need_new_child_node = True
                hop_ip = next(iter(hop_ip_set))

                if not ipaddress.ip_address(hop_ip).is_private:
                    all_public_ips.append(hop_ip)

                hopChildNode = None
                if hopNode.has_children():
                    hopChildNodes = hopNode.get_children()
                    for node in hopChildNodes:
                        
                        if node.ip == hop_ip:
                            hopChildNode = node
                            need_new_child_node = False
                
                if need_new_child_node:
                    hop_type = 'Node'
                    if hop_ip in cdn_ip_set:
                        hop_type = 'CDN'
                    asn_info, _ = asndb.lookup(hop_ip)
                    country = None
                    if hop_ip in ip_to_country_dict:
                        country = ip_to_country_dict[hop_ip]
                    else:
                        geo_details = find_ip(hop_ip,ip_to_country_df)
                        if geo_details is not None:
                            if 'country_code' in geo_details:
                                country = geo_details['country_code']
                                ip_to_country_dict[hop_ip] = country
                        else:
                            ip_to_country_dict[hop_ip] = None

                    newChildNode = Hop(
                        ip=hop_ip,
                        type=hop_type,
                        min_rtt=min_hop_rtt,
                        asn=asn_info,
                        hostname=None,
                        country=country,
                        location=None,
                    )
                    hopNode.add_children(newChildNode)
                    hopChildNode = newChildNode
                else:
                    this_min_rtt = min_hop_rtt
                    prev_min_rtt = hopChildNode.min_rtt
                    if this_min_rtt < prev_min_rtt:
                        hopChildNode.min_rtt = this_min_rtt
                hopNode = hopChildNode

In [ ]:
ip_to_details_map_filename = "../dataset/ip_to_details.json"
ip_to_details = {}
with open(ip_to_details_map_filename,'r') as json_file:
    ip_to_details = json.load(json_file)

ip_to_hostname_map_filename = "../dataset/ip_to_hostname.json"
ip_to_hostname = {}
with open(ip_to_hostname_map_filename,'r') as json_file:
    ip_to_hostname = json.load(json_file)

In [ ]:
import graphviz

def go_through_all_nodes_cf(root,pop_ip_addr):
    pop_ips_unknown = set()
    last_unicast_ips = {}
    last_unicast_ips_within_rtt = set()
    pop_ips_known = []
    pop_ip_dict = {}
    loc_to_count = {}


    def visit_node(node):
        node_ip = node.ip

        if node_ip is not None:
            
            if node_ip not in ip_to_details:
                if node_ip not in ip_to_hostname:
                    _, hostname = reverse_dns_lookup(node_ip)
                    node.hostname = hostname
                    ip_to_hostname[node_ip] = hostname

                else:
                    node.hostname = ip_to_hostname[node_ip]
            else:
                node.location = ip_to_details[node_ip]['location']

            if node.type == 'PoP':

                loc = None
                if node_ip not in ip_to_details:
                    pop_ips_unknown.add(node_ip)

                else:
                    pop_ips_known.append(node_ip)
                    loc = ip_to_details[node_ip]['location']
                pop_ip_dict[node_ip] = {
                    'min_rtt' : node.min_rtt,
                    'location' : loc
                }
                
        for child_node in node.children:
            if child_node.type == 'CDN':

                loc = "unknown"
                if node.ip not in ip_to_loc or node.location is None:

                    if pop_ip_dict[pop_ip_addr]["location"]:

                        pop_min_rtt = pop_ip_dict[pop_ip_addr]["min_rtt"]
                        if child_node.min_rtt >= pop_min_rtt and child_node.min_rtt - pop_min_rtt <= 2:
                            loc = pop_ip_dict[pop_ip_addr]["location"]
                else:
                    if child_node.min_rtt >= node.min_rtt and child_node.min_rtt - node.min_rtt <= 2:
                        if node.ip in ip_to_loc:
                            loc = ip_to_loc[node.ip]
                        elif node.location:
                            loc = node.location

                if node.ip not in last_unicast_ips:
                    last_unicast_ips[node.ip] = loc
                if child_node.min_rtt >= node.min_rtt and child_node.min_rtt - node.min_rtt <= 2:
                    last_unicast_ips_within_rtt.add(node.ip)
                    
                if loc not in loc_to_count:
                    loc_to_count[loc] = 0
                loc_to_count[loc] += 1
            visit_node(child_node)

                    

    def save_ip_map_json(filename, dict_to_save, inital_len):
        if len(dict_to_save) > inital_len:
            with open(filename, "w+") as json_file:
                json.dump(dict_to_save,json_file)

    visit_node(root)
    

    return pop_ips_unknown, last_unicast_ips, last_unicast_ips_within_rtt, pop_ips_known, pop_ip_dict, loc_to_count

In [ ]:
def reverse_dns_lookup(ip_addr):
    try:
        hostname, _, _ = socket.gethostbyaddr(ip_addr)
        return ip_addr, hostname
    except socket.herror as e:
        return ip_addr, None

In [ ]:
domain_list = [
  "www.broadcom.com",
  "www.comodoca.com",
  "www.apnic.net",
  "www.riskified.com",
  "www.wiley.com",
  "www.vmware.com",
  "www.sportskeeda.com",
  "www.garmin.com", #IPV4 only
  "www.fao.org",
  "www.n-able.com",
    ]

glb_cf_loc_to_count = {}
glb_cf_loc_to_count_abs_num = {}
chosen_countries = ["US", "CA" ,"DE", "HU", "CL", "CO", "AU", "NZ" ,"PH", "GU", "BJ", "ZW" , "ZM", "MG"]

for chosen_country in chosen_countries:
  cdn_ip_count_rtt = {}
  cf_loc_to_count = {}
  print(f"Country: {chosen_country}")

  probes_list = []
  for probe_id, probe_details in probes_cf_path.items():
      if chosen_country == probe_details['country_code']:
          probes_list.append(probe_id)
  print(f"Probes: {probes_list}")
  probe_id = probes_list[0]
  print(f"Now checking for probe id: {probe_id}")
  print(f"Prb: {probe_id}")

  # FOR PROBE LIST
  pop_ip_addr_set = set()
  for probe_id in probes_list:
    for domain in domain_list:
      if domain in probes_cf_path[probe_id]['paths']:
          pop_ip_addr_set = pop_ip_addr_set.union(set(probes_cf_path[probe_id]['paths'][domain].keys()))
  pop_ip_addr_list = list(pop_ip_addr_set)
  print(pop_ip_addr_list)

  glb_pop_ips_unknown = set()
  glb_last_unicast_ips = set()
  glb_last_unicast_ips_within_rtt = set()
  glb_loc_to_count = {}
  glb_pop_ip_dict = {}

  for probe_id in probes_list:
    for domain in domain_list:
      for pop_ip_address in pop_ip_addr_list:
        if domain not in probes_cf_path[probe_id]['paths']:
          continue
        if pop_ip_address in probes_cf_path[probe_id]['paths'][domain]:
          rootNode = probes_cf_path[probe_id]['paths'][domain][pop_ip_address]
          pop_ips_unknown, last_unicast_ips, last_unicast_ips_within_rtt, pop_ips_known, pop_ip_dict, loc_to_count = go_through_all_nodes_cf(root=rootNode,pop_ip_addr=pop_ip_address)
          
          glb_pop_ips_unknown = glb_pop_ips_unknown.union(pop_ips_unknown)
          glb_last_unicast_ips = glb_last_unicast_ips.union(set(last_unicast_ips.keys()))
          glb_last_unicast_ips_within_rtt = glb_last_unicast_ips_within_rtt.union(last_unicast_ips_within_rtt)
          glb_pop_ip_dict.update(pop_ip_dict)

          for ip_location, count in loc_to_count.items():
            if ip_location not in glb_loc_to_count:
              glb_loc_to_count[ip_location] = 0
            glb_loc_to_count[ip_location] += count

  print(f"glb_pop_ip_dict={glb_pop_ip_dict}")  
  print(f"glb_pop_ips_unknown={glb_pop_ips_unknown}")
  print(f"glb_loc_to_count={glb_loc_to_count}")
  print(f"glb_last_unicast_ips_within_rtt={glb_last_unicast_ips_within_rtt}")

  known_unicast_ips = []
  for ip in list(glb_last_unicast_ips_within_rtt):
    known_unicast_ips.append(ip)
    ipinfo_details = find_ip(ip, ip_to_country_df)
    if ipinfo_details:
      if 'country' in ipinfo_details:
        print(f"IP: {ip} -> {ipinfo_details['country']}")
      else:
        print(f"IP: {ip} -> unknown")   
    else:
      print(f"IP: {ip} -> unknown")


  all_last_unicast_ips = set()
  for idx, each_msm in enumerate(cf_acc_tracert_json,start=0):

    prb_id = each_msm['prb_id']
    if prb_id in probes_list: 
        pop_ip_set = set(each_msm['pop_hop_ip'])
        if len(each_msm['all_hops']) >=2:
          cdn_ip_list = each_msm['all_hops'][-2]['hop_ips']

          if each_msm['domain'] in domain_list:
              for msm_idx, ip in enumerate(cdn_ip_list):

                  all_last_unicast_ips.add(ip)
                  if ip not in cdn_ip_count_rtt:
                      cdn_ip_count_rtt[ip] = []
                  
                  cdn_ip_count_rtt[ip].append(each_msm['all_hops'][-2]['hop_rtts'][msm_idx])



  for ip, cdn_rt_list in cdn_ip_count_rtt.items():
      loc = "unknown"

      if ip in glb_last_unicast_ips_within_rtt:
        if ip in ip_to_details and ip_to_details[ip]['location'] is not None:
            loc = ip_to_details[ip]['location']
            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
        else:
            ipinfo_details = find_ip(ip,ip_to_country_df)
            if ipinfo_details:
                loc = ipinfo_details['country']
                print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
            else:

                print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}]")
      else:
        print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}]")

      if loc not in cf_loc_to_count:
          cf_loc_to_count[loc] = 0
      cf_loc_to_count[loc] += len(cdn_rt_list)

  sum = 0
  for value in cf_loc_to_count.values():
      sum += value

  if chosen_country not in glb_cf_loc_to_count:
    glb_cf_loc_to_count[chosen_country] = {}
  if chosen_country not in glb_cf_loc_to_count_abs_num:
    glb_cf_loc_to_count_abs_num[chosen_country] = {}

  glb_cf_loc_to_count_abs_num[chosen_country] = copy.deepcopy(cf_loc_to_count)

  print("************** Final Count ****************")
  for loc, value in cf_loc_to_count.items():
      country_percent = round((value/sum) * 100,2)
      print(f"{loc} : {country_percent}%")

      if loc not in glb_cf_loc_to_count[chosen_country]:
        glb_cf_loc_to_count[chosen_country][loc] = country_percent

      

  print("********************************")
  print()

In [ ]:

loc_to_country = {
    "Germany" : "DE",
    "Bulgaria" : "BG",
    "Peru" : "PE",
    "Santiago" : "CL",
    "United States" : "US",
    "Canada" : "CA",
    "Colombia" : "CO",
    "Australia" : "AU",
    "New Zealand" : "NZ",
    "Singapore" : "SG",
    "United Kingdom" : "GB",
    "South Africa" : "ZA",
    "Nigeria" : "NG",
    "unknown" : "N/A"
}
cf_loc_to_count_filtered = {}
for chosen_country in chosen_countries:
    print(f"Country: {chosen_country}")
    cf_loc_to_count_filtered[chosen_country] = {}
    total_sum_locs = __builtins__.sum(glb_cf_loc_to_count[chosen_country].values())

    print("************** CF Final Count ****************")
    for loc, value in glb_cf_loc_to_count[chosen_country].items():
        loc_percent = round((value/total_sum_locs) * 100,2)
        if loc_percent > 5:
            loc_country_code = loc_to_country[loc]
            cf_loc_to_count_filtered[chosen_country][loc_country_code] = loc_percent
            print(f"{loc_country_code} : {loc_percent}%")
    print()

### Cloudflare - Early 2025

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "cloudflare"

acc_result_filename = f"acc_trace_{server}_final_result.json"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)
print(filepath)

cf_accumulated_trace_json_old = None
with open(filepath, "r") as json_file:
    cf_accumulated_trace_json_old = json.load(json_file)
    print("Accumulated Traceroute JSON file loaded successfully")

In [ ]:
probes_cf_path_old = {}
all_public_ips = []
ip_to_country_dict = {}


shortlisted_countries = ['GU']

class Hop:
    def __init__(self,ip,type,min_rtt,asn,hostname,country,location):
        self.ip = ip
        self.type = type
        self.min_rtt = min_rtt
        self.asn = asn
        self.hostname = hostname
        self.country = country
        self.location = location
        self.children = []

    def add_children(self,child_node):
        self.children.append(child_node)
    
    def has_children(self):
        return len(self.children) > 0
    
    def get_children(self):
        return self.children


for idx, each_msm in enumerate(cf_accumulated_trace_json_old,start=0):
        print(f"Index: {idx} out of {len(cf_accumulated_trace_json_old)}")

        pop_ip_set = set(each_msm['pop_hop_ip'])
        cdn_ip_set = set(each_msm['all_hops'][-1]['hop_ips'])

        if len(pop_ip_set) == 0:
            continue
        
        if len(pop_ip_set) != 1:
            print(f"More than 2 PoP IPs in the same hop. Probe id: {prb_id}, domain: {each_msm['domain']}")
            pop_ip_set = {list(pop_ip_set)[0],}
            print(f"Continuing with PoP IP: {list(pop_ip_set)[0]}")

        if each_msm['country_code'] not in shortlisted_countries:
            continue

        ## Create new probe in probes if not done already
        prb_id = each_msm['prb_id']
        if prb_id not in probes_cf_path_old:
            probes_cf_path_old[prb_id] = {
                'id' : prb_id,
                'country_code' : each_msm['country_code'],
                'paths': {},
            }

        ## Create first hop for the probe as starting with the pop node if not done already
        pop_ip = next(iter(pop_ip_set))
        pop_asn = 14593
        domain = each_msm['domain']
        if domain not in probes_cf_path_old[prb_id]['paths']:
            probes_cf_path_old[prb_id]['paths'][domain] = {}
        if pop_ip not in probes_cf_path_old[prb_id]['paths'][domain]:
                country = None
                if pop_ip in ip_to_country_dict:
                    country = ip_to_country_dict[pop_ip]
                else:
                    geo_details = find_ip(pop_ip,ip_to_country_df)
                    if geo_details is not None:
                        if 'country_code' in geo_details:
                            country = geo_details['country_code']
                            ip_to_country_dict[pop_ip] = country
                    else:
                        ip_to_country_dict[pop_ip] = None
                probes_cf_path_old[prb_id]['paths'][domain][pop_ip] = Hop(
                    ip=pop_ip,
                    type='PoP',
                    min_rtt=min(each_msm['pop_rtts']),
                    asn=pop_asn,
                    hostname=None,
                    country=country,
                    location=None,
                )
        else:
            this_min_rtt = min(each_msm['pop_rtts'])
            prev_min_rtt = probes_cf_path_old[prb_id]['paths'][domain][pop_ip].min_rtt
            if this_min_rtt < prev_min_rtt:
                probes_cf_path_old[prb_id]['paths'][domain][pop_ip].min_rtt = this_min_rtt

        pop_reached = False
        hopNode = probes_cf_path_old[prb_id]['paths'][domain][pop_ip]
        
        ## Go through all the hops
        for idx, each_hop in enumerate(each_msm['all_hops']):
            hop_ip_set = set(each_hop['hop_ips'])
            hop_id = each_hop['hop_id']
            min_hop_rtt = min(each_hop['hop_rtts'])

            if pop_ip_set.intersection(hop_ip_set):
                pop_reached = True
                continue

            
            
            if pop_reached:
                if len(hop_ip_set) != 1:
                    print(f"More than 2 destinations in the same hop. Probe id: {prb_id}, domain: {each_msm['domain']}, hop id: {hop_id}")
                    
                    print("Check if the hop ips are part of the same network address")
                    hop_ip_network_count = {}
                    hop_ip_network_min_rtt = {}
                    for uni_idx, unique_hop_ip in enumerate(each_hop['hop_ips']):
                        unique_hop_ip_obj = ipaddress.ip_interface(unique_hop_ip + '/24')
                        unique_hop_network = str(unique_hop_ip_obj.network.network_address)
                        
                        if unique_hop_network not in hop_ip_network_count:
                            hop_ip_network_count[unique_hop_network] = 1
                            hop_ip_network_min_rtt[unique_hop_network] = each_hop['hop_rtts'][uni_idx]
                        else:
                            continue
                        for each_idx, each_ip in enumerate(each_hop['hop_ips']):
                            if each_idx == uni_idx:
                                continue
                            each_ip_obj = ipaddress.ip_interface(each_ip + '/24')
                            each_ip_network = str(each_ip_obj.network.network_address)
                            if each_ip_network == unique_hop_network:
                                hop_ip_network_count[unique_hop_network] += 1
                                if each_hop['hop_rtts'][each_idx] <  hop_ip_network_min_rtt[unique_hop_network]:
                                    hop_ip_network_min_rtt[unique_hop_network] = each_hop['hop_rtts'][each_idx]
                    
                    max_count = max(hop_ip_network_count.values())
                    if max_count < 2:
                        print("Could not find a majority network address. Skipping measurement.")
                        break
                    max_key = max(hop_ip_network_count, key=hop_ip_network_count.get)
                    hop_ip_set = {max_key,}
                    min_hop_rtt = hop_ip_network_min_rtt[max_key]

              
                need_new_child_node = True
                hop_ip = next(iter(hop_ip_set))

                if not ipaddress.ip_address(hop_ip).is_private:
                    all_public_ips.append(hop_ip)

                hopChildNode = None
                if hopNode.has_children():
                    hopChildNodes = hopNode.get_children()
                    for node in hopChildNodes:
                        
                        if node.ip == hop_ip:
                            hopChildNode = node
                            need_new_child_node = False
                
                if need_new_child_node:
                    hop_type = 'Node'
                    if hop_ip in cdn_ip_set:
                        hop_type = 'CDN'
                    asn_info, _ = asndb.lookup(hop_ip)
                    country = None
                    if hop_ip in ip_to_country_dict:
                        country = ip_to_country_dict[hop_ip]
                    else:
                        geo_details = find_ip(hop_ip,ip_to_country_df)
                        if geo_details is not None:
                            if 'country_code' in geo_details:
                                country = geo_details['country_code']
                                ip_to_country_dict[hop_ip] = country
                        else:
                            ip_to_country_dict[hop_ip] = None

                    newChildNode = Hop(
                        ip=hop_ip,
                        type=hop_type,
                        min_rtt=min_hop_rtt,
                        asn=asn_info,
                        hostname=None,
                        country=country,
                        location=None,
                    )
                    hopNode.add_children(newChildNode)
                    hopChildNode = newChildNode
                else:
                    this_min_rtt = min_hop_rtt
                    prev_min_rtt = hopChildNode.min_rtt
                    if this_min_rtt < prev_min_rtt:
                        hopChildNode.min_rtt = this_min_rtt
                hopNode = hopChildNode

In [ ]:
cdn_ip_count_rtt_old = {}
cf_loc_to_count_old = {}

domain_list = [
  "www.broadcom.com",
  "www.comodoca.com",
  "www.apnic.net",
  "www.riskified.com",
  "www.wiley.com",
  "www.vmware.com",
  "www.sportskeeda.com",
  "www.garmin.com", #IPV4 only
  "www.fao.org",
  "www.n-able.com",
    ]


# FOR PROBE LIST
probes_list = [65337] # For GU probe
pop_ip_addr_set = set()
for probe_id in probes_list:
  for domain in domain_list:
    if domain in probes_cf_path_old[probe_id]['paths']:
        pop_ip_addr_set = pop_ip_addr_set.union(set(probes_cf_path_old[probe_id]['paths'][domain].keys()))
pop_ip_addr_list = list(pop_ip_addr_set)
print(pop_ip_addr_list)


glb_pop_ips_unknown = set()
glb_last_unicast_ips = set()
glb_last_unicast_ips_within_rtt = set()
glb_loc_to_count = {}
glb_pop_ip_dict = {}

for probe_id in probes_list:
  for domain in domain_list:
    for pop_ip_address in pop_ip_addr_list:
      if domain not in probes_cf_path_old[probe_id]['paths']:
        continue
      if pop_ip_address in probes_cf_path_old[probe_id]['paths'][domain]:
        rootNode = probes_cf_path_old[probe_id]['paths'][domain][pop_ip_address]
        pop_ips_unknown, last_unicast_ips, last_unicast_ips_within_rtt, pop_ips_known, pop_ip_dict, loc_to_count = go_through_all_nodes_cf(root=rootNode,pop_ip_addr=pop_ip_address)
        
        glb_pop_ips_unknown = glb_pop_ips_unknown.union(pop_ips_unknown)
        glb_last_unicast_ips = glb_last_unicast_ips.union(set(last_unicast_ips.keys()))
        glb_last_unicast_ips_within_rtt = glb_last_unicast_ips_within_rtt.union(last_unicast_ips_within_rtt)
        glb_pop_ip_dict.update(pop_ip_dict)

        for ip_location, count in loc_to_count.items():
          if ip_location not in glb_loc_to_count:
            glb_loc_to_count[ip_location] = 0
          glb_loc_to_count[ip_location] += count

print(f"glb_pop_ip_dict={glb_pop_ip_dict}")  
print(f"glb_pop_ips_unknown={glb_pop_ips_unknown}")
print(f"glb_loc_to_count={glb_loc_to_count}")
print(f"glb_last_unicast_ips_within_rtt={glb_last_unicast_ips_within_rtt}")

known_unicast_ips = []
for ip in list(glb_last_unicast_ips_within_rtt):
  known_unicast_ips.append(ip)
  ipinfo_details = find_ip(ip, ip_to_country_df)
  if ipinfo_details:
    if 'country' in ipinfo_details:
      print(f"IP: {ip} -> {ipinfo_details['country']}")
    else:
      print(f"IP: {ip} -> unknown")   
  else:
    print(f"IP: {ip} -> unknown")



all_last_unicast_ips = set()
for idx, each_msm in enumerate(cf_accumulated_trace_json_old,start=0):

  prb_id = each_msm['prb_id']
  if prb_id == probe_id: 
      pop_ip_set = set(each_msm['pop_hop_ip'])
      if len(each_msm['all_hops']) >=2:
        cdn_ip_list = each_msm['all_hops'][-2]['hop_ips']

        if each_msm['domain'] in domain_list:
            for msm_idx, ip in enumerate(cdn_ip_list):

                all_last_unicast_ips.add(ip)
                if ip not in cdn_ip_count_rtt_old:
                    cdn_ip_count_rtt_old[ip] = []
                
                cdn_ip_count_rtt_old[ip].append(each_msm['all_hops'][-2]['hop_rtts'][msm_idx])



for ip, cdn_rt_list in cdn_ip_count_rtt_old.items():
    loc = "unknown"

    if ip in glb_last_unicast_ips_within_rtt:
      if ip in ip_to_details and ip_to_details[ip]['location'] is not None:
          loc = ip_to_details[ip]['location']
          print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
      else:
          ipinfo_details = find_ip(ip,ip_to_country_df)
          if ipinfo_details:
              loc = ipinfo_details['country']
              print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
          else:

              print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}]")
    else:
       print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}]")

    if loc not in cf_loc_to_count_old:
        cf_loc_to_count_old[loc] = 0
    cf_loc_to_count_old[loc] += len(cdn_rt_list)

sum = 0
for value in cf_loc_to_count_old.values():
    sum += value

print("************** Final Count ****************")
for loc, value in cf_loc_to_count_old.items():
    print(f"{loc} : {round((value/sum) * 100,2)}%")



### Akamai - Late 2025

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "akamai"

acc_result_filename = "acc_trace_akamai_final_result.json"
full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)

akamai_accumulated_trace_json = None
with open(filepath, "r") as json_file:
    akamai_accumulated_trace_json = json.load(json_file)
    print("Accumulated Traceroute JSON file loaded successfully")

In [ ]:
akamai_controlled_nodes_traceroute_files_path = f"{controlled_top_level_folder_path}/traceroute/08-27-2025-12-31-18_WebTrace_files_list"
akamai_controlled_acc_filename = "controlled_nodes_acc_trace_akamai_final_result_late.json"
akamai_controlled_accumulated_trace_json = []
akamai_controlled_acc_filepath = os.path.join(akamai_controlled_nodes_traceroute_files_path,"acc_result",akamai_controlled_acc_filename)
with open(akamai_controlled_acc_filepath, "r") as json_file:
    akamai_controlled_accumulated_trace_json = json.load(json_file)


akamai_accumulated_trace_json += akamai_controlled_accumulated_trace_json
print(f"{len(akamai_controlled_accumulated_trace_json)} controlled node measurements added to accumulated_trace_json")

In [ ]:
cdn_ip_count_rtt = {}


chosen_country_list = ["US", "CA" ,"DE", "HU", "CL", "CO", "AU", "NZ" ,"PH", "GU", "BJ", "ZW" , "ZM", "MG"]

domain_list = [
    "www.microsoft.com", #IPV4 only
    "www.apple.com", #IPV4 only
    "www.bing.com", #IPV4 only
    "www.icloud.com", #IPV4 only
    "www.intuit.com",
    "www.unity3d.com",
    "www.samsung.com",
    "www.ebay.com",
    "www.webex.com", #IPV4 only
    "www.cisco.com", #IPV4 only
    ]

for idx, each_msm in enumerate(akamai_accumulated_trace_json,start=0):

        prb_id = each_msm['prb_id']
        country_code = each_msm['country_code']

        if country_code in chosen_country_list:
  
            if country_code not in cdn_ip_count_rtt:
                cdn_ip_count_rtt[country_code] = {}
            pop_ip_set = set(each_msm['pop_hop_ip'])
            cdn_ip_set = set(each_msm['all_hops'][-1]['hop_ips'])
            dst_addr = each_msm['dst_addr']
            if each_msm['domain'] in domain_list[:5]:
                for msm_idx, ip in enumerate(cdn_ip_set):

                    if ip == dst_addr:
                        if ip not in cdn_ip_count_rtt[country_code]:
                            cdn_ip_count_rtt[country_code][ip] = []
                        
                        cdn_ip_count_rtt[country_code][ip].append(each_msm['all_hops'][-1]['hop_rtts'][msm_idx])


In [ ]:
akamai_loc_to_count = {}

with open("../dataset/ip_to_country_dns_cdn_dict.json","r") as json_file:
    ip_to_country_dns_cdn_dict = json.load(json_file)


for country_code in list(cdn_ip_count_rtt.keys()):

    if country_code not in akamai_loc_to_count:
        akamai_loc_to_count[country_code] = {}
    for ip, cdn_rt_list in cdn_ip_count_rtt[country_code].items():
        loc = "unknown"
        if ip in ip_to_country_dns_cdn_dict and ip_to_country_dns_cdn_dict[ip] is not None:
            loc = ip_to_country_dns_cdn_dict[ip]

        else:
            ipinfo_details = find_ip(ip,ip_to_country_df)
            if ipinfo_details:
                loc = ipinfo_details['country_code']
                ip_to_country_dns_cdn_dict[ip] = loc
        
        
        if loc not in akamai_loc_to_count[country_code]:
            akamai_loc_to_count[country_code][loc] = 0
        akamai_loc_to_count[country_code][loc] += len(cdn_rt_list)

with open("../dataset/ip_to_country_dns_cdn_dict.json","w") as json_file:
    json.dump(ip_to_country_dns_cdn_dict,json_file)

In [ ]:
# del sum
akamai_loc_to_count_filtered = {}
for chosen_country in list(akamai_loc_to_count.keys()):
    print(f"Country: {chosen_country}")
    akamai_loc_to_count_filtered[chosen_country] = {}
    total_sum_locs = __builtins__.sum(akamai_loc_to_count[chosen_country].values())

    print("************** Akamai Final Count ****************")
    for loc, value in akamai_loc_to_count[chosen_country].items():
        loc_percent = round((value/total_sum_locs) * 100,2)
        if loc_percent > 5:
            akamai_loc_to_count_filtered[chosen_country][loc] = loc_percent
            print(f"{loc} : {loc_percent}%")
    print()

In [ ]:
probes_list = [65337] # for GU probe

cdn_ip_count_rtt = {}
akamai_loc_to_count_gu = {}

domain_list = [
    "www.microsoft.com", #IPV4 only
    "www.apple.com", #IPV4 only
    "www.bing.com", #IPV4 only
    "www.icloud.com", #IPV4 only
    "www.intuit.com",
    "www.unity3d.com",
    "www.samsung.com",
    "www.ebay.com",
    "www.webex.com", #IPV4 only
    "www.cisco.com", #IPV4 only
    ]
prbs_found_new = []
for idx, each_msm in enumerate(akamai_accumulated_trace_json,start=0):

        prb_id = each_msm['prb_id']
        if prb_id in probes_list: 
            if prb_id not in prbs_found_new:
                prbs_found_new.append(prb_id)
            pop_ip_set = set(each_msm['pop_hop_ip'])
            cdn_ip_set = set(each_msm['all_hops'][-1]['hop_ips'])
            if each_msm['domain'] in domain_list:
                for msm_idx, ip in enumerate(cdn_ip_set):

                        if ip not in cdn_ip_count_rtt:
                            cdn_ip_count_rtt[ip] = []
                        
                        cdn_ip_count_rtt[ip].append(each_msm['all_hops'][-1]['hop_rtts'][msm_idx])

for ip, cdn_rt_list in cdn_ip_count_rtt.items():
    loc = "unknown"
    if ip in ip_to_details and ip_to_details[ip]['location'] is not None:
        loc = ip_to_details[ip]['location']
        print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
    else:
        ipinfo_details = find_ip(ip,ip_to_country_df)
        if ipinfo_details:
            loc = ipinfo_details['country']
            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
        else:

            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}]")
    
    if loc not in akamai_loc_to_count_gu:
        akamai_loc_to_count_gu[loc] = 0
    akamai_loc_to_count_gu[loc] += len(cdn_rt_list)

sum = 0
for value in akamai_loc_to_count_gu.values():
    sum += value

print("************** Final Count ****************")
for loc, value in akamai_loc_to_count_gu.items():
    print(f"{loc} : {round((value/sum) * 100,2)}%")

### Akamai - Early 2025

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "akamai"

acc_result_filename = "acc_trace_akamai_final_result.json"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)

akamai_accumulated_trace_json_old = None
with open(filepath, "r") as json_file:
    akamai_accumulated_trace_json_old = json.load(json_file)
    print("Accumulated Traceroute JSON file loaded successfully")

In [ ]:
probes_list = [65337] # For GU probe

cdn_ip_count_rtt_old = {}
akamai_loc_to_count_old = {} # Holds only for GU

domain_list = [
    "www.microsoft.com", #IPV4 only
    "www.apple.com", #IPV4 only
    "www.bing.com", #IPV4 only
    "www.icloud.com", #IPV4 only
    "www.intuit.com",
    "www.unity3d.com",
    "www.samsung.com",
    "www.ebay.com",
    "www.webex.com", #IPV4 only
    "www.cisco.com", #IPV4 only
    ]
prbs_found_old = []
for idx, each_msm in enumerate(akamai_accumulated_trace_json_old,start=0):

        prb_id = each_msm['prb_id']
        if prb_id in probes_list: 
            if prb_id not in prbs_found_old:
                prbs_found_old.append(prb_id)
            pop_ip_set = set(each_msm['pop_hop_ip'])
            cdn_ip_set = set(each_msm['all_hops'][-1]['hop_ips'])
            if each_msm['domain'] in domain_list:
                for msm_idx, ip in enumerate(cdn_ip_set):

                        if ip not in cdn_ip_count_rtt_old:
                            cdn_ip_count_rtt_old[ip] = []
                        
                        cdn_ip_count_rtt_old[ip].append(each_msm['all_hops'][-1]['hop_rtts'][msm_idx])

for ip, cdn_rt_list in cdn_ip_count_rtt_old.items():
    loc = "unknown"
    if ip in ip_to_details and ip_to_details[ip]['location'] is not None:
        loc = ip_to_details[ip]['location']
        print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
    else:
        ipinfo_details = find_ip(ip,ip_to_country_df)
        if ipinfo_details:
            loc = ipinfo_details['country']
            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
        else:

            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}]")
    
    if loc not in akamai_loc_to_count_old:
        akamai_loc_to_count_old[loc] = 0
    akamai_loc_to_count_old[loc] += len(cdn_rt_list)

sum = 0
for value in akamai_loc_to_count_old.values():
    sum += value

print("************** Final Count ****************")
for loc, value in akamai_loc_to_count_old.items():
    print(f"{loc} : {round((value/sum) * 100,2)}%")

### Cloudfront - Late 2025

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "cloudfront"

acc_result_filename = "acc_trace_cloudfront_final_result.json"
full_msms_path = os.path.join(top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)

cloudfront_acc_tracert_json = None
with open(filepath, "r") as json_file:
    cloudfront_acc_tracert_json = json.load(json_file)
    print("Accumulated Cloudfront Traceroute JSON file loaded successfully")

In [ ]:
controlled_nodes_traceroute_files_path = f"{controlled_top_level_folder_path}/traceroute/08-27-2025-12-31-18_WebTrace_files_list"
cfront_controlled_acc_filename = "controlled_nodes_acc_trace_cloudfront_final_result_late.json"
cfront_controlled_accumulated_trace_json = []
cfront_controlled_acc_filepath = os.path.join(controlled_nodes_traceroute_files_path,"acc_result",cfront_controlled_acc_filename)
with open(cfront_controlled_acc_filepath, "r") as json_file:
    cloudfront_controlled_accumulated_trace_json = json.load(json_file)


cloudfront_acc_tracert_json += cloudfront_controlled_accumulated_trace_json
print(f"{len(cloudfront_controlled_accumulated_trace_json)} controlled node measurements added to accumulated_trace_json")

In [ ]:
cfront_cdn_ip_count_rtt = {}
all_cfront_countries = set()

chosen_country_list = ["US", "CA" ,"DE", "HU", "CL", "CO", "AU", "NZ" ,"PH", "GU", "BJ", "ZW" , "ZM", "MG"]

domain_list = [
    "www.soundcloud.com",
    "www.zynga.com", #IPV4 only
    "www.doi.org",
    "www.booking.com",
    "www.brave.com", #IPV4 only
    "www.tycsports.com", #IPV4 only
    "www.logitech.com", #IPV4 only
    "www.checkpoint.com",
    "www.goodreads.com",
    "www.surveymonkey.com"  
    ]

for idx, each_msm in enumerate(cloudfront_acc_tracert_json,start=0):
        prb_id = each_msm['prb_id']
        country_code = each_msm['country_code']
        all_cfront_countries.add(country_code)

        if country_code in chosen_country_list:
            if country_code not in cfront_cdn_ip_count_rtt:
                cfront_cdn_ip_count_rtt[country_code] = {}
            pop_ip_set = set(each_msm['pop_hop_ip'])
            cdn_ip_set = set(each_msm['all_hops'][-1]['hop_ips'])
            dst_addr = each_msm['dst_addr']
            if each_msm['domain'] in domain_list[:5]:
                for msm_idx, ip in enumerate(cdn_ip_set):
                    if ip == dst_addr:
                        if ip not in cfront_cdn_ip_count_rtt[country_code]:
                            cfront_cdn_ip_count_rtt[country_code][ip] = []
                        
                        cfront_cdn_ip_count_rtt[country_code][ip].append(each_msm['all_hops'][-1]['hop_rtts'][msm_idx])


In [ ]:
cloudfront_loc_to_count = {}

with open("../dataset/ip_to_country_dns_cdn_dict.json","r") as json_file:
    ip_to_country_dns_cdn_dict = json.load(json_file)


for country_code in list(cfront_cdn_ip_count_rtt.keys()):

    if country_code not in cloudfront_loc_to_count:
        cloudfront_loc_to_count[country_code] = {}
    for ip, cdn_rt_list in cfront_cdn_ip_count_rtt[country_code].items():
        loc = "unknown"
        if ip in ip_to_country_dns_cdn_dict and ip_to_country_dns_cdn_dict[ip] is not None:
            loc = ip_to_country_dns_cdn_dict[ip]

        else:
            ipinfo_details = find_ip(ip,ip_to_country_df)
            if ipinfo_details:
                loc = ipinfo_details['country_code']
                ip_to_country_dns_cdn_dict[ip] = loc
        
        
        if loc not in cloudfront_loc_to_count[country_code]:
            cloudfront_loc_to_count[country_code][loc] = 0
        cloudfront_loc_to_count[country_code][loc] += len(cdn_rt_list)

with open("../dataset/ip_to_country_dns_cdn_dict.json","w") as json_file:
    json.dump(ip_to_country_dns_cdn_dict,json_file)

In [ ]:
# del sum
cloudfront_loc_to_count_filtered = {}
for chosen_country in list(cfront_cdn_ip_count_rtt.keys()):
    print(f"Country: {chosen_country}")
    cloudfront_loc_to_count_filtered[chosen_country] = {}
    total_sum_locs = __builtins__.sum(cloudfront_loc_to_count[chosen_country].values())

    print("************** cloudfront Final Count ****************")
    for loc, value in cloudfront_loc_to_count[chosen_country].items():
        loc_percent = round((value/total_sum_locs) * 100,2)
        if loc_percent > 5:
            cloudfront_loc_to_count_filtered[chosen_country][loc] = loc_percent
            print(f"{loc} : {loc_percent}%")
    print()

In [ ]:
cdn_ip_count_rtt = {}
cfront_loc_to_count = {}


domain_list = [
    "www.soundcloud.com",
    "www.zynga.com", #IPV4 only
    "www.doi.org",
    "www.booking.com",
    "www.brave.com", #IPV4 only
    "www.tycsports.com", #IPV4 only
    "www.logitech.com", #IPV4 only
    "www.checkpoint.com",
    "www.goodreads.com",
    "www.surveymonkey.com"
    ]


probes_list = [65337]

for idx, each_msm in enumerate(cloudfront_acc_tracert_json,start=0):

        prb_id = each_msm['prb_id']
        if prb_id in probes_list: 
            pop_ip_set = set(each_msm['pop_hop_ip'])
            cdn_ip_list = each_msm['all_hops'][-1]['hop_ips']
            if each_msm['domain'] in domain_list:
                for msm_idx, ip in enumerate(cdn_ip_list):

                        if ip not in cdn_ip_count_rtt:
                            cdn_ip_count_rtt[ip] = []
                        
                        cdn_ip_count_rtt[ip].append(each_msm['all_hops'][-1]['hop_rtts'][msm_idx])

for ip, cdn_rt_list in cdn_ip_count_rtt.items():
    loc = "unknown"
    if ip in ip_to_details and ip_to_details[ip]['location'] is not None:
        loc = ip_to_details[ip]['location']
        print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
    else:
        ipinfo_details = find_ip(ip,ip_to_country_df)
        if ipinfo_details:
            loc = ipinfo_details['country']
            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
        else:

            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}]")
    
    if loc not in cfront_loc_to_count:
        cfront_loc_to_count[loc] = 0
    cfront_loc_to_count[loc] += len(cdn_rt_list)

sum = 0
for value in cfront_loc_to_count.values():
    sum += value

print("************** Final Count ****************")
for loc, value in cfront_loc_to_count.items():
    print(f"{loc} : {round((value/sum) * 100,2)}%")

### Cloudfront - Early 2025

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "cloudfront"

acc_result_filename = f"acc_trace_{server}_final_result.json"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)

cfront_accumulated_trace_json_old = None
with open(filepath, "r") as json_file:
    cfront_accumulated_trace_json_old = json.load(json_file)
    print("Accumulated Traceroute JSON file loaded successfully")

In [ ]:
cdn_ip_count_rtt_old = {}
cfront_loc_to_count_old = {}


domain_list = [
    "www.soundcloud.com",
    "www.zynga.com", #IPV4 only
    "www.doi.org",
    "www.booking.com",
    "www.brave.com", #IPV4 only
    "www.tycsports.com", #IPV4 only
    "www.logitech.com", #IPV4 only
    "www.checkpoint.com",
    "www.goodreads.com",
    "www.surveymonkey.com"
    ]


probes_list = [65337]

for idx, each_msm in enumerate(cfront_accumulated_trace_json_old,start=0):

        prb_id = each_msm['prb_id']
        if prb_id in probes_list: 
            pop_ip_set = set(each_msm['pop_hop_ip'])
            cdn_ip_list = each_msm['all_hops'][-1]['hop_ips']
            if each_msm['domain'] in domain_list:
                for msm_idx, ip in enumerate(cdn_ip_list):

                        if ip not in cdn_ip_count_rtt_old:
                            cdn_ip_count_rtt_old[ip] = []
                        
                        cdn_ip_count_rtt_old[ip].append(each_msm['all_hops'][-1]['hop_rtts'][msm_idx])

for ip, cdn_rt_list in cdn_ip_count_rtt_old.items():
    loc = "unknown"
    if ip in ip_to_details and ip_to_details[ip]['location'] is not None:
        loc = ip_to_details[ip]['location']
        print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
    else:
        ipinfo_details = find_ip(ip,ip_to_country_df)
        if ipinfo_details:
            loc = ipinfo_details['country']
            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}] -> ({loc})")
        else:

            print(f"{ip} : {statistics.median(cdn_rt_list)} ms [{len(cdn_rt_list)}]")
    
    if loc not in cfront_loc_to_count_old:
        cfront_loc_to_count_old[loc] = 0
    cfront_loc_to_count_old[loc] += len(cdn_rt_list)

sum = 0
for value in cfront_loc_to_count_old.values():
    sum += value

print("************** Final Count ****************")
for loc, value in cfront_loc_to_count_old.items():
    print(f"{loc} : {round((value/sum) * 100,2)}%")

### Compound bar - All CDN locations

In [ ]:

import operator
rcParams['font.size'] = 10
rcParams["font.family"] = "CMU Sans Serif"
width = 1.2

color_map = {}

cmap_26 = plt.get_cmap('hsv') 

# Sample N equally spaced colors from the map's range (0.0 to 1.0)
colors_26 = [cmap_26(i) for i in np.linspace(0, 1, 26)]

colors_tab20 = plt.get_cmap('tab20').colors 

# Get 6 additional, distinct colors
colors_set1 = plt.get_cmap('Set2').colors[:6] 

# Combine them for 26 unique colors
colors_26_combined = colors_tab20 + colors_set1

country_priority_dict = {
    "CA" : ["US", "CA"],
    "CO" : ["CO", "CL", "PE"],
    "PH" : ["SG", "HK"],
    "GU" : ["SG", "HK", "US"],
    "BJ" : ["ZA", "NG", "GB"],
    "MG" : ["ZA"]
}

# color_map = {
#     "AT": cols_tab20[0],
#     "AU": cols_tab20[1],
#     "US": cols_tab20[2],
#     "CL": cols_tab20[3],
#     "CA": cols_tab20[4],
#     "CO": cols_tab20[17],
#     "DE": cols_tab20[6],
#     "GB": cols_tab20[7],
#     "HK": cols_tab20[8],
#     "IT": cols_tab20[9],
#     "JP": cols_tab20[10],
#     "NZ" : cols_tab20[11],
#     "PE" : cols_tab20[12],
#     "SG" : cols_tab20[13],
#     "BR" : cols_tab20[19],
#     "ZA" : cols_tab20[18]
# }

unique_countries = set()
fig, ax = plt.subplots(figsize=(8.37, 4.37))

first_handles = [Patch(facecolor='none',edgecolor='darkorange'),Patch(facecolor='none',edgecolor='darkgreen'),Patch(facecolor='none',edgecolor='purple')]
first_labels = ["Cloudflare", "Akamai", "Cloudfront"]
first_legend = ax.legend(first_handles, first_labels, loc="lower left",bbox_to_anchor=(0.1, 1) , facecolor="white", framealpha=1.0, fontsize=12, edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4, frameon=False)
ax.add_artist(first_legend)
last_pos = 0
space = 0.3
alpha_param = 0.2

all_labels = []
x_A = np.arange(len(chosen_country_list)) * 5.5


x_ticks = x_A + width + space

color_idx = 0

for j_dx, chosen_country_code in enumerate(chosen_country_list):

    all_labels.append(chosen_country_code)
    bottom_accumulated = 0

    cf_loc_to_count_filtered_country = cf_loc_to_count_filtered[chosen_country_code]

    if chosen_country_code in country_priority_dict:
        cf_loc_to_count_filtered_country = {}
        for priority_cdn_loc in country_priority_dict[chosen_country_code]:
            if priority_cdn_loc in cf_loc_to_count_filtered[chosen_country_code]:
                cf_loc_to_count_filtered_country[priority_cdn_loc] = cf_loc_to_count_filtered[chosen_country_code][priority_cdn_loc]
        
        remaining_cdn_locs = set(cf_loc_to_count_filtered[chosen_country_code]) - set(cf_loc_to_count_filtered_country.keys())

        for other_cdn_loc in remaining_cdn_locs:
            cf_loc_to_count_filtered_country[other_cdn_loc] = cf_loc_to_count_filtered[chosen_country_code][other_cdn_loc]

    for idx, (cdn_loc, value) in enumerate(cf_loc_to_count_filtered_country.items()):
        alpha_param = 0.5
        cf_country_percent = value
        unique_countries.add(cdn_loc)
        chosen_color = None
        if cdn_loc not in color_map:
            color_map[cdn_loc] = colors_26_combined[color_idx]

            color_idx += 1

        chosen_color = color_map.get(cdn_loc,"gray")

        if cdn_loc == "N/A":
            chosen_color = "black"
            alpha_param = 1.0
        ax.bar(x_A[j_dx],cf_country_percent,bottom=bottom_accumulated,width=width,color=chosen_color,alpha=alpha_param, label=cdn_loc,edgecolor='black')
        if cdn_loc != "N/A":
            ax.text(x_A[j_dx]-0.25,(2*bottom_accumulated+cf_country_percent)/2,cdn_loc, va='center', fontsize="xx-small", color ='black',rotation=90)
 
        bottom_accumulated += cf_country_percent
    
    if bottom_accumulated != 100:
        ax.bar(x_A[j_dx],100-bottom_accumulated,bottom=bottom_accumulated,width=width,color="white", hatch='///',  alpha=alpha_param, label=cdn_loc, edgecolor='black')
    
    ax.bar(x_A[j_dx],100,bottom=0,width=width,color='none',alpha=1.0,edgecolor='darkorange',linewidth=1.8)

    bottom_accumulated = 0

    akamai_loc_to_count_filtered_country = akamai_loc_to_count_filtered[chosen_country_code]

    if chosen_country_code in country_priority_dict:
        akamai_loc_to_count_filtered_country = {}
        for priority_cdn_loc in country_priority_dict[chosen_country_code]:
            if priority_cdn_loc in akamai_loc_to_count_filtered[chosen_country_code]:
                akamai_loc_to_count_filtered_country[priority_cdn_loc] = akamai_loc_to_count_filtered[chosen_country_code][priority_cdn_loc]
        
        remaining_cdn_locs = set(akamai_loc_to_count_filtered[chosen_country_code]) - set(akamai_loc_to_count_filtered_country.keys())

        for other_cdn_loc in remaining_cdn_locs:
            akamai_loc_to_count_filtered_country[other_cdn_loc] = akamai_loc_to_count_filtered[chosen_country_code][other_cdn_loc]

    for idx, (cdn_loc, value) in enumerate(akamai_loc_to_count_filtered_country.items()):
        alpha_param = 0.5
        akamai_country_percent = value
        unique_countries.add(cdn_loc)
        chosen_color = None
        if cdn_loc not in color_map:
            color_map[cdn_loc] = colors_26_combined[color_idx]
         
            color_idx += 1
        chosen_color = color_map.get(cdn_loc,"gray")

        hatch_pattern = None
        if cdn_loc == "N/A":
            chosen_color = "black"
            alpha_param = 1.0
        ax.bar(x_A[j_dx]+width+space,akamai_country_percent,bottom=bottom_accumulated,width=width,color=chosen_color,alpha=alpha_param, label=cdn_loc, edgecolor='black')
        if cdn_loc != "N/A":
            ax.text(x_A[j_dx]+width+space-0.25,(2*bottom_accumulated+akamai_country_percent)/2,cdn_loc, va='center', fontsize="xx-small", color ='black', rotation=90)
        
        bottom_accumulated += akamai_country_percent

    if bottom_accumulated != 100:
        ax.bar(x_A[j_dx]+width+space,100-bottom_accumulated,bottom=bottom_accumulated,width=width,color="white", hatch='///', alpha=alpha_param, label=cdn_loc, edgecolor='black')

    ax.bar(x_A[j_dx]+width+space,100,bottom=0,width=width,color='none',alpha=1.0,edgecolor='darkgreen',linewidth=1.8)

    bottom_accumulated = 0

    cloudfront_loc_to_count_filtered_country = cloudfront_loc_to_count_filtered[chosen_country_code]

    if chosen_country_code in country_priority_dict:
        cloudfront_loc_to_count_filtered_country = {}
        for priority_cdn_loc in country_priority_dict[chosen_country_code]:
            if priority_cdn_loc in cloudfront_loc_to_count_filtered[chosen_country_code]:
                cloudfront_loc_to_count_filtered_country[priority_cdn_loc] = cloudfront_loc_to_count_filtered[chosen_country_code][priority_cdn_loc]
        
        remaining_cdn_locs = set(cloudfront_loc_to_count_filtered[chosen_country_code]) - set(cloudfront_loc_to_count_filtered_country.keys())

        for other_cdn_loc in remaining_cdn_locs:
            cloudfront_loc_to_count_filtered_country[other_cdn_loc] = cloudfront_loc_to_count_filtered[chosen_country_code][other_cdn_loc]

    for idx, (cdn_loc, value) in enumerate(cloudfront_loc_to_count_filtered_country.items()):
        alpha_param = 0.5
        cloudfront_country_percent = value
        unique_countries.add(cdn_loc)
        chosen_color = None
        if cdn_loc not in color_map:
            color_map[cdn_loc] = colors_26_combined[color_idx]
            color_idx += 1
        chosen_color = color_map.get(cdn_loc,"gray")

        if cdn_loc == "N/A":
            chosen_color = "black"
            alpha_param = 1.0
        ax.bar(x_A[j_dx]+2*(width+space),cloudfront_country_percent,bottom=bottom_accumulated,width=width,color=chosen_color,alpha=alpha_param, label=cdn_loc, edgecolor='black')
        if cdn_loc != "N/A":
            ax.text(x_A[j_dx]+2*(width+space)-0.25,(2*bottom_accumulated+cloudfront_country_percent)/2,cdn_loc, va='center', fontsize="xx-small", color ='black', rotation=90)

        bottom_accumulated += cloudfront_country_percent

    if bottom_accumulated != 100:
        ax.bar(x_A[j_dx]+2*(width+space),100-bottom_accumulated,bottom=bottom_accumulated,width=width,color="white", hatch='///', alpha=alpha_param, label=cdn_loc, edgecolor='black')

    ax.bar(x_A[j_dx]+2*(width+space),100,bottom=0,width=width,color='none',alpha=1.0,edgecolor='purple',linewidth=1.8)  


    last_pos = x_A[idx] + space

plt.ylim(0,102)
plt.xlim(-1,76)

ax.tick_params(axis='both', labelsize=16)
ax.set_yticks([0,10,20,30,40,50,60,70,80,90,100])

ax.set_xticks(x_ticks)
ax.set_xticklabels(all_labels, rotation=0, ha="center", fontsize=14)
ax.set_ylabel("Percentage",fontsize=16)

plt.grid(True, axis='y', linestyle='-', alpha=0.7, linewidth=0.5)

handles = [Patch(facecolor="black",alpha=1.0,edgecolor='black',hatch="o"),Patch(facecolor='none',edgecolor='black',hatch='///')]
labels = ["unknown", "other countries"]
ax.legend(handles, labels, loc="lower right", bbox_to_anchor=(0.84, 1) ,fontsize=12, facecolor="white", framealpha=1.0, edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4, frameon=False)
plt.tight_layout()
plt.savefig(f"{figs_top_level_folder_path}/selected_countries_all_cdn_locations_percent.pdf", bbox_inches = "tight", pad_inches = 0.1)
plt.savefig(f"{figs_top_level_folder_path}/selected_countries_all_cdn_locations_percent.svg", bbox_inches = "tight", pad_inches = 0.1)

plt.show()

In [ ]:
chosen_country_list = ["US", "CA" ,"DE", "HU", "CL", "CO", "AU", "NZ" ,"PH", "GU", "BJ", "ZW" , "ZM", "MG"]

data = {}
for chosen_country_code in chosen_country_list:
    cf_loc_to_count_filtered_country = cf_loc_to_count_filtered[chosen_country_code]
    cf_loc_to_count_filtered_country_sorted = dict(sorted(cf_loc_to_count_filtered_country.items(), key=lambda item: item[1], reverse=True))

    akamai_loc_to_count_filtered_country = akamai_loc_to_count_filtered[chosen_country_code]
    akamai_loc_to_count_filtered_country_sorted = dict(sorted(akamai_loc_to_count_filtered_country.items(), key=lambda item: item[1], reverse=True))

    cloudfront_loc_to_count_filtered_country = cloudfront_loc_to_count_filtered[chosen_country_code]
    cloudfront_loc_to_count_filtered_country_sorted = dict(sorted(cloudfront_loc_to_count_filtered_country.items(), key=lambda item: item[1], reverse=True))

    if chosen_country_code not in data:
        data[chosen_country_code] = {}
        data[chosen_country_code]['Cloudflare'] = []
        data[chosen_country_code]['Akamai'] = []
        data[chosen_country_code]['Cloudfront'] = []

    for cdn_loc, cf_country_percent in cf_loc_to_count_filtered_country_sorted.items():
        if cf_country_percent > 5:
            data[chosen_country_code]['Cloudflare'].append((cdn_loc,cf_country_percent))

    for cdn_loc, akamai_country_percent in akamai_loc_to_count_filtered_country_sorted.items():
        if akamai_country_percent > 5:
            data[chosen_country_code]['Akamai'].append((cdn_loc,akamai_country_percent))

    for cdn_loc, cloudfront_country_percent in cloudfront_loc_to_count_filtered_country_sorted.items():
        if cloudfront_country_percent > 5:
            data[chosen_country_code]['Cloudfront'].append((cdn_loc,cloudfront_country_percent))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

rcParams["font.size"] = 12
user_countries = list(data.keys())
providers = ['Cloudflare', 'Akamai', 'Cloudfront']


# 2. Setup Figure with Subplots (1 row, 3 columns)
# sharey=True ensures country labels only show on the left

gs_kw = dict(width_ratios=[3, 5, 4])
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(8.37, 4.37), 
                         gridspec_kw=gs_kw, constrained_layout=True)


cmap = plt.cm.YlOrRd.copy()
cmap.set_bad(color='white')

# 3. Iterate through each provider to build its specific subplot
for p_idx, provider in enumerate(providers):
    ax = axes[p_idx]
    if p_idx > 0:
        ax.tick_params(axis='y', which='both', left=False, labelleft=False)
    # Initialize a matrix for just this provider's sub-cells
    if provider == "Cloudflare":
        max_subcells = 3
    elif provider == "Akamai":
        max_subcells = 5
    else:
        max_subcells = 4
    provider_matrix = np.full((len(user_countries), max_subcells), np.nan)
    
    for r, u_country in enumerate(user_countries):
        cells = data[u_country][provider]
        
        for c_idx, (loc_code, percent) in enumerate(cells):
            if c_idx < max_subcells:
                provider_matrix[r, c_idx] = percent
                text_color = "white" if percent > 70 else "black"
                ax.text(c_idx, r, loc_code, ha="center", va="center", 
                        color=text_color, fontsize=12)

    # Plot the heatmap for this provider
    im = ax.imshow(provider_matrix, cmap=cmap, aspect='auto', vmin=0, vmax=100)
    
    # Formatting
    ax.set_title(provider, fontsize=14, fontweight='bold', pad=10)
    ax.set_xticks([]) # Remove sub-cell ticks
    for spine in ax.spines.values():
        spine.set_visible(True) # Hide the box border

# 4. Global labels and Colorbar
axes[0].set_yticks(np.arange(len(user_countries)))
axes[0].set_yticklabels(user_countries)

# Position a shared colorbar on the right
cbar_ax = fig.add_axes([1.01, 0.15, 0.02, 0.7])
cbar_ax.tick_params(labelsize=12)

fig.colorbar(im, cax=cbar_ax)
plt.savefig(f"{figs_top_level_folder_path}/selected_countries_all_cdn_locations_percent_heatmap.pdf", bbox_inches = "tight", pad_inches = 0.1)
plt.savefig(f"{figs_top_level_folder_path}/selected_countries_all_cdn_locations_percent_heatmap.svg", bbox_inches = "tight", pad_inches = 0.1)

plt.show()

### GU - old vs new CDN locations barplot

In [ ]:
loc_to_country = {
    'United States' : 'US',
    'Seattle' : 'US',
    'New York' : 'US',
    'Chicago' : 'US',
    'Toronto' : 'CA',
    'Washington' : 'US',
    'Minneapolis' : 'US',
    'Los Angeles' : 'US',
    'Boston' : 'US',
    'New York' : 'US',
    'San Jose' : 'US',
    'Atlanta' : 'US',
    'Canada' : 'CA',
    'Philadelphia' : 'US',
    'Detroit' : 'US,',
    'Germany' : 'DE',
    'United Kingdom' : 'GB',
    'Australia' : 'AU',
    'France' : 'FR',
    'Hong Kong' : 'HKG',
    'Mexico' : 'MX',
    'Singapore' : 'SG',
    'Philippines' : 'PH',
    'Taiwan' : 'TW',
    'Manila' : 'PH',
    'Japan' : 'JP',
    'Tokyo' : 'JP',
    'Italy' : 'IT',
    'Israel' : 'IL',
    'The Netherlands' : 'NL',
    'Finland' : 'FI',
    'Malaysia' : 'MY',
    'Vietnam' : 'VN',
    'India' : 'IN',
    'unknown' : 'N/A'
}
country_to_loc = {v:k for k,v in loc_to_country.items()}

In [ ]:
cc_loc_to_count_old = {}
cc_loc_to_count_new = {}

loc_to_count_dicts_old = [cf_loc_to_count_old,akamai_loc_to_count_old,cfront_loc_to_count_old] # these already are filtered for GU
loc_to_count_dicts_new = [glb_cf_loc_to_count_abs_num['GU'],akamai_loc_to_count_gu,cfront_loc_to_count] # only for GU

for count_dict in loc_to_count_dicts_old:
    for location, count in count_dict.items():
        cc_loc = loc_to_country[location]
        if cc_loc not in cc_loc_to_count_old:
            cc_loc_to_count_old[cc_loc] = 0

        cc_loc_to_count_old[cc_loc] += count

for count_dict in loc_to_count_dicts_new:
    for location, count in count_dict.items():
        if location not in loc_to_country.values():
            if location == 'HK':
                cc_loc = 'HKG'
            else:
                cc_loc = loc_to_country[location]
        else:
            cc_loc = location
        if cc_loc not in cc_loc_to_count_new:
            cc_loc_to_count_new[cc_loc] = 0

        cc_loc_to_count_new[cc_loc] += count

In [ ]:

from matplotlib import rcParams
import operator
rcParams['font.size'] = 10
rcParams["font.family"] = "CMU Sans Serif"
chosen_country_code = 'GU'
old_count = __builtins__.sum(cc_loc_to_count_old.values())
new_count = __builtins__.sum(cc_loc_to_count_new.values())

old_percent_dict = {k: round((v/old_count) * 100) for k, v in cc_loc_to_count_old.items() if round((v/old_count) * 100) >=2}
new_percent_dict = {k: round((v/new_count) * 100) for k, v in cc_loc_to_count_new.items() if round((v/new_count) * 100) >=2}

sorted_old_percent_dict = sorted(old_percent_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_new_percent_dict = sorted(new_percent_dict.items(), key=operator.itemgetter(1), reverse=True)

cols = plt.get_cmap("tab10").colors
color_map = {
    "PH": cols[0],
    "TW": cols[1],
    "HKG": cols[2],
    "SG": cols[3],
    "N/A": cols[4],
    "DE": cols[5],
    "US": cols[6],
    "CA": cols[7],
    "IL": cols[8],
    "GB": cols[9]
}

space = 0.8
x_A = np.arange(len(old_percent_dict))
x_B = np.arange(len(new_percent_dict)) + len(old_percent_dict) + space
width = 0.8
fig, ax = plt.subplots(figsize=(3.37, 3.37))

print(old_percent_dict)
print(new_percent_dict)

old_countries = []
new_countries = []
for i, (country, percent) in enumerate(sorted_old_percent_dict):
    old_countries.append(country)
    ax.bar(x_A[i],percent, width,color=color_map.get(country,"gray"), alpha=0.7, label=country)

for i, (country, percent) in enumerate(sorted_new_percent_dict):
    new_countries.append(country)
    ax.bar(x_B[i],percent, width,color=color_map.get(country,"gray"), alpha=0.7, label=country)


all_labels = old_countries + new_countries
x_ticks = np.concatenate([x_A, x_B])
ax.set_yticks([0,10,20,30,40,50,60,70,80,90])
ax.set_xticks(x_ticks)
ax.set_xticklabels(all_labels, rotation=0, ha="center", fontsize=10)
ax.set_ylabel("Percentage")

ax.text(-0.35,-15,"JP PoP")
ax.text(4.0,-15,"PH PoP")
plt.grid(True, axis='y', linestyle='-', alpha=0.7, linewidth=0.5)
plt.savefig(f"{figs_top_level_folder_path}/{chosen_country_code}_Probes_resolved_ip_loc_before_vs_after_PoP_change.pdf", bbox_inches = "tight", pad_inches = 0.1)
plt.savefig(f"{figs_top_level_folder_path}/{chosen_country_code}_Probes_resolved_ip_loc_before_vs_after_PoP_change.svg", bbox_inches = "tight", pad_inches = 0.1)
plt.tight_layout()
plt.show()


## Cloudflare AIM

In [ ]:
import jsonlines

In [ ]:
cloudflare_glob = f"../dataset/late-2025/cloudflare-AIM/starlink-bq-results-20251006-190426-1759777511222.json"

starlink_cdn_latency_countrycode = {}
starlink_cdn_latency_countryname = {}

starlink_cdn_loadedlatency_download_countryname = {}
starlink_cdn_loadedlatency_upload_countryname = {}

starlink_cdn_loadedlatency_download_city_to_cdn = {}
starlink_cdn_loadedlatency_upload_city_to_cdn = {}

starlink_country_to_city_map = {}
starlink_user_to_cdnloc_map ={}
starlink_user_to_cdnloc_minRTT_map ={}

code_country_json = json.load(open(f"../helper/code_to_country_name.json","r"))

with jsonlines.open(cloudflare_glob) as reader:
    for obj in reader:
        country = obj['clientCountry']
        city_name = obj['clientCity']
        server_loc = obj['serverPoP']
        
        if len(obj['latencyMs']) == 20:
            if country not in starlink_cdn_latency_countrycode:
                starlink_cdn_latency_countrycode[country] = []
            starlink_cdn_latency_countrycode[country] += obj['latencyMs']

            if country in code_country_json:
                country_name = code_country_json[country]
                if country_name not in starlink_cdn_latency_countryname:
                    starlink_cdn_latency_countryname[country_name] = []
                    starlink_cdn_loadedlatency_download_countryname[country_name] = []
                    starlink_cdn_loadedlatency_upload_countryname[country_name] = []
                starlink_cdn_latency_countryname[country_name] += obj['latencyMs']

                if city_name not in starlink_cdn_loadedlatency_download_city_to_cdn:
                    starlink_cdn_loadedlatency_download_city_to_cdn[city_name] = {}
                if city_name not in starlink_cdn_loadedlatency_upload_city_to_cdn:
                    starlink_cdn_loadedlatency_upload_city_to_cdn[city_name] = {}
                
                if server_loc not in starlink_cdn_loadedlatency_download_city_to_cdn[city_name]:
                    starlink_cdn_loadedlatency_download_city_to_cdn[city_name][server_loc] = []
                if server_loc not in starlink_cdn_loadedlatency_upload_city_to_cdn[city_name]:
                    starlink_cdn_loadedlatency_upload_city_to_cdn[city_name][server_loc] = []

                if 'download' in obj['loadedLatencyMs']:
                    if len(obj['loadedLatencyMs']['download']):
                        dw_inflationRTT = max(obj['loadedLatencyMs']['download']) - min(obj['loadedLatencyMs']['download'])
                        starlink_cdn_loadedlatency_download_countryname[country_name].append(dw_inflationRTT)
                        starlink_cdn_loadedlatency_download_city_to_cdn[city_name][server_loc].append(dw_inflationRTT)
                if 'upload' in obj['loadedLatencyMs']:
                    if len(obj['loadedLatencyMs']['upload']):
                        up_inflationRTT = max(obj['loadedLatencyMs']['upload']) - min(obj['loadedLatencyMs']['upload'])
                        starlink_cdn_loadedlatency_upload_countryname[country_name].append(up_inflationRTT)
                        starlink_cdn_loadedlatency_upload_city_to_cdn[city_name][server_loc].append(up_inflationRTT)


                if country_name not in starlink_country_to_city_map:
                    starlink_country_to_city_map[country_name] = []
                if city_name not in starlink_country_to_city_map[country_name]:
                    starlink_country_to_city_map[country_name].append(city_name)

                if city_name not in starlink_user_to_cdnloc_map:
                    starlink_user_to_cdnloc_map[city_name] = {}
                    starlink_user_to_cdnloc_minRTT_map[city_name] = {}
                
                if server_loc not in starlink_user_to_cdnloc_map[city_name]:
                    starlink_user_to_cdnloc_map[city_name][server_loc] = []
                    starlink_user_to_cdnloc_minRTT_map[city_name][server_loc] = []
                starlink_user_to_cdnloc_map[city_name][server_loc] += obj['latencyMs']
                starlink_user_to_cdnloc_minRTT_map[city_name][server_loc].append(min(obj['latencyMs']))

            else:
                pass

In [ ]:
cloudflare_glob_terrestrial_file = f"../dataset/late-2025/cloudflare-AIM/terrestrial-bq-results-20251006-194420-1759779889924.json"

terrestrial_cdn_latency_countrycode = {}
terrestrial_cdn_latency_countryname = {}


terrestrial_cdn_loadedlatency_download_countryname = {}
terrestrial_cdn_loadedlatency_upload_countryname = {}

terrestrial_cdn_loadedlatency_download_city_to_cdn = {}
terrestrial_cdn_loadedlatency_upload_city_to_cdn = {}

terrestrial_country_to_city_map ={}
terrestrial_user_to_cdnloc_map ={}
terrestrial_user_to_cdnloc_minRTT_map ={}

code_country_json = json.load(open(f"../helper/code_to_country_name.json","r"))
idx = 0

with jsonlines.open(cloudflare_glob_terrestrial_file) as reader:
    for obj in reader:
        idx += 1
        country = obj['clientCountry']
        city_name = obj['clientCity']
        server_loc = obj['serverPoP']

        if len(obj['latencyMs']) == 20:
            if country not in terrestrial_cdn_latency_countrycode:
                terrestrial_cdn_latency_countrycode[country] = []
            terrestrial_cdn_latency_countrycode[country] += obj['latencyMs']

            if country in code_country_json:
                country_name = code_country_json[country]
                if country_name not in terrestrial_cdn_latency_countryname:
                    terrestrial_cdn_latency_countryname[country_name] = []
                    terrestrial_cdn_loadedlatency_download_countryname[country_name] = []
                    terrestrial_cdn_loadedlatency_upload_countryname[country_name] = []
                terrestrial_cdn_latency_countryname[country_name] += obj['latencyMs']

                if city_name not in terrestrial_cdn_loadedlatency_download_city_to_cdn:
                        terrestrial_cdn_loadedlatency_download_city_to_cdn[city_name] = {}
                if city_name not in terrestrial_cdn_loadedlatency_upload_city_to_cdn:
                    terrestrial_cdn_loadedlatency_upload_city_to_cdn[city_name] = {}
                
                if server_loc not in terrestrial_cdn_loadedlatency_download_city_to_cdn[city_name]:
                    terrestrial_cdn_loadedlatency_download_city_to_cdn[city_name][server_loc] = []
                if server_loc not in terrestrial_cdn_loadedlatency_upload_city_to_cdn[city_name]:
                    terrestrial_cdn_loadedlatency_upload_city_to_cdn[city_name][server_loc] = []

                if 'download' in obj['loadedLatencyMs']:
                    if len(obj['loadedLatencyMs']['download']):
                        dw_inflationRTT = max(obj['loadedLatencyMs']['download']) - min(obj['loadedLatencyMs']['download'])
                        terrestrial_cdn_loadedlatency_download_countryname[country_name].append(dw_inflationRTT)
                        terrestrial_cdn_loadedlatency_download_city_to_cdn[city_name][server_loc].append(dw_inflationRTT)
                if 'upload' in obj['loadedLatencyMs']:
                    if len(obj['loadedLatencyMs']['upload']):
                        up_inflationRTT = max(obj['loadedLatencyMs']['upload']) - min(obj['loadedLatencyMs']['upload'])
                        terrestrial_cdn_loadedlatency_upload_countryname[country_name].append(up_inflationRTT)
                        terrestrial_cdn_loadedlatency_upload_city_to_cdn[city_name][server_loc].append(up_inflationRTT)

                if country_name not in terrestrial_country_to_city_map:
                    terrestrial_country_to_city_map[country_name] = []
                if city_name not in terrestrial_country_to_city_map[country_name]:
                    terrestrial_country_to_city_map[country_name].append(city_name)

                if city_name not in terrestrial_user_to_cdnloc_map:
                    terrestrial_user_to_cdnloc_map[city_name] = {}
                    terrestrial_user_to_cdnloc_minRTT_map[city_name] = {}
                
                if server_loc not in terrestrial_user_to_cdnloc_map[city_name]:
                    terrestrial_user_to_cdnloc_map[city_name][server_loc] = []
                    terrestrial_user_to_cdnloc_minRTT_map[city_name][server_loc] = []
                terrestrial_user_to_cdnloc_map[city_name][server_loc] += obj['latencyMs']
                terrestrial_user_to_cdnloc_minRTT_map[city_name][server_loc].append(min(obj['latencyMs']))
            else:
                pass


In [ ]:
terrestrial_user_to_bestcdn_map = {}
terrestrial_user_to_bestcdn_minRTT_map = {}

for user_city, cdn_latency_dict in terrestrial_user_to_cdnloc_map.items():
    temp_dict = {}
    for cdn_loc, latency_list in cdn_latency_dict.items():
        if len(latency_list) >= 100:
            temp_dict[cdn_loc] = statistics.median(latency_list)
    if temp_dict :
        key_of_min = min(temp_dict, key=temp_dict.get)
        terrestrial_user_to_bestcdn_map[user_city] = {
            key_of_min : cdn_latency_dict[key_of_min]
        }

        terrestrial_user_to_bestcdn_minRTT_map[user_city] = {
            key_of_min: terrestrial_user_to_cdnloc_minRTT_map[user_city][key_of_min]
        }

In [ ]:
starlink_user_to_bestcdn_map = {}
starlink_user_to_bestcdn_minRTT_map = {}

for user_city, cdn_latency_dict in starlink_user_to_cdnloc_map.items():
    temp_dict = {}
    for cdn_loc, latency_list in cdn_latency_dict.items():
        if len(latency_list) >= 100:
            temp_dict[cdn_loc] = statistics.median(latency_list)
    if temp_dict :
        key_of_min = min(temp_dict, key=temp_dict.get)
        starlink_user_to_bestcdn_map[user_city] = {
            key_of_min : cdn_latency_dict[key_of_min]
        }

        starlink_user_to_bestcdn_minRTT_map[user_city] = {
            key_of_min: starlink_user_to_cdnloc_minRTT_map[user_city][key_of_min]
        }

In [ ]:
terrestrial_cdn_latency_countryname_best_cdn_medianrtts = {}

for country, city_list in terrestrial_country_to_city_map.items():
    if country not in terrestrial_cdn_latency_countryname_best_cdn_medianrtts:
        terrestrial_cdn_latency_countryname_best_cdn_medianrtts[country] = []
    for city in city_list:
        if city in terrestrial_user_to_bestcdn_map:
            for cdn, rtts in terrestrial_user_to_bestcdn_map[city].items():
                terrestrial_cdn_latency_countryname_best_cdn_medianrtts[country] += rtts

In [ ]:
starlink_cdn_latency_countryname_best_cdn_medianrtts = {}

for country, city_list in starlink_country_to_city_map.items():
    if country not in starlink_cdn_latency_countryname_best_cdn_medianrtts:
        starlink_cdn_latency_countryname_best_cdn_medianrtts[country] = []
    for city in city_list:
        if city in starlink_user_to_bestcdn_map:
            for cdn, rtts in starlink_user_to_bestcdn_map[city].items():
                starlink_cdn_latency_countryname_best_cdn_medianrtts[country] += rtts

In [ ]:
cols = plt.get_cmap("tab10").colors
rcParams['font.size'] = 10
rcParams["font.family"] = "CMU Sans Serif"
popmarkerfacecolor = cols[2]

def plot_gsdata(gsdata_df, ax, label, marker, color, size=10, lw=0.6):
    latitude, longitude = gsdata_df["latitude"], gsdata_df["longitude"]
    ax.scatter(longitude, latitude,
               sizes = [size],
               color=color,
               zorder = 10,
               marker=marker,
               edgecolor="black",
               linewidth = lw,
               label=label)

pop_df = pd.read_csv(f'../dataset/pop_locations_Oct2025.csv') #pop_2023-10-10
pop_coords = geopandas.GeoDataFrame(pop_df, geometry=geopandas.points_from_xy(pop_df.longitude, pop_df.latitude))


world = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))
world = world[(world.name!="Antarctica")]

countries_plotted = set()
median_rtt_difference = []
all_values_list = []
for name, code in zip(world["name"], world["iso_a3"]):    
    if name in terrestrial_cdn_latency_countryname_best_cdn_medianrtts and name in starlink_cdn_latency_countryname_best_cdn_medianrtts:
        if len(terrestrial_cdn_latency_countryname_best_cdn_medianrtts[name]) > 25 and len(starlink_cdn_latency_countryname_best_cdn_medianrtts[name]) > 25:
            rtt_diff = statistics.median(starlink_cdn_latency_countryname_best_cdn_medianrtts[name]) - statistics.median(terrestrial_cdn_latency_countryname_best_cdn_medianrtts[name])
            print(f"Country: {name}, RTT delta: {rtt_diff}")
            median_rtt_difference.append(rtt_diff)
            countries_plotted.add(name)
            all_values_list.append(rtt_diff)
        else:
            median_rtt_difference.append(float("NaN"))
    elif code == "-99":
        median_rtt_difference.append(float("NaN"))
    else:
        median_rtt_difference.append(float("nan"))



world["rtt_diff"] = median_rtt_difference
fig, ax = plt.subplots(figsize=(4.66, 2), subplot_kw={'projection': ccrs.PlateCarree()})

total_bins = [-100,-80,-50,-20,0,20,50,80,100]
handles = []
pop_handles = [Line2D([0], [0], marker="o", color="w", markeredgecolor="black", markerfacecolor=popmarkerfacecolor, markeredgewidth=0.8)
           for i in range(1)]
labels = ["PoP"]
handles = pop_handles
ax.coastlines(linewidth=0.5)
cmap = 'seismic'

world.plot(column="rtt_diff", cmap=cmap, scheme='UserDefined', classification_kwds=dict(bins=total_bins),
           ax=ax,alpha=0.8,)

plot_gsdata(pop_df, ax, "PoP", "o", popmarkerfacecolor, size=10, lw=0.3)
ax.set_extent([-180, 180, -60, 90], ccrs.PlateCarree())
ax.get_xaxis().set_ticks([])
ax.get_yaxis().set_ticks([])

ax.legend(handles, labels,handlelength=1, labelspacing=0.06, columnspacing=0.5, handletextpad=0.3,
          loc="lower left", fancybox=False, edgecolor="k", fontsize="small")
 
# Create colorbar as a legend
sm = plt.cm.ScalarMappable(norm=pltColors.Normalize(vmin=-100, vmax=100), cmap=cmap)
# Empty array for the data range
sm._A = []
# Add the colorbar to the figure
cbaxes = fig.add_axes([0.015, 0.10, 0.02, 0.8])
cbar = fig.colorbar(sm, cax=cbaxes)
cbar.set_label("Median RTT Difference (ms)", labelpad = -50)
cbar.locator = ticker.MaxNLocator(nbins=6)
custom_ticks = [-100, -75 ,-50, -25, 0, 25, 50, 75, 100]
cbar.set_ticks(custom_ticks)

carrowupaxes = fig.add_axes([-0.070, 0.075, 0.02, 0.4])
carrowupaxes.arrow(0, 0, 0, -0.00005, length_includes_head=True,
          head_width=0.04, head_length=0.0000075, ec="black", fc="black")
carrowupaxes.set_axis_off()

carrowdownaxes = fig.add_axes([-0.070, 0.495, 0.02, 0.4])
carrowdownaxes.arrow(0, 0, 0, 0.00005, length_includes_head=True,
          head_width=0.04, head_length=0.0000075, ec="black", fc="black")
carrowdownaxes.set_axis_off()
plt.figtext(x=-0.095,y=0.120,s="Starlink faster", rotation='vertical', fontsize=8)
plt.figtext(x=-0.095,y=0.520,s="Terrestrial faster", rotation='vertical', fontsize=8)
plt.savefig(f"{figs_top_level_folder_path}/terrestrial_vs_starlink_rtt_difference_global_heatmap_bestCDN_Cloudflare_AIM.pdf", bbox_inches = "tight", pad_inches = 0.1)
plt.savefig(f"{figs_top_level_folder_path}/terrestrial_vs_starlink_rtt_difference_global_heatmap_bestCDN_Cloudflare_AIM.svg", bbox_inches = "tight", pad_inches = 0.1)
plt.show()

In [ ]:
iata_to_city_dict = {}
city_to_iata_dict = {}

### Load cloudflare traceroute data to infer PoP locations

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "cloudflare"

acc_result_filename = f"acc_trace_{server}_final_result.json"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)
print(filepath)

cf_accumulated_trace_json_old = None
with open(filepath, "r") as json_file:
    cf_accumulated_trace_json_old = json.load(json_file)
    print("Accumulated Traceroute JSON file loaded successfully")

In [ ]:
probes_cf_path_old = {}
all_public_ips = []
ip_to_country_dict = {}


class Hop:
    def __init__(self,ip,type,min_rtt,asn,hostname,country,location):
        self.ip = ip
        self.type = type
        self.min_rtt = min_rtt
        self.asn = asn
        self.hostname = hostname
        self.country = country
        self.location = location
        self.children = []

    def add_children(self,child_node):
        self.children.append(child_node)
    
    def has_children(self):
        return len(self.children) > 0
    
    def get_children(self):
        return self.children


for idx, each_msm in enumerate(cf_accumulated_trace_json_old,start=0):
        print(f"Index: {idx} out of {len(cf_accumulated_trace_json_old)}")

        pop_ip_set = set(each_msm['pop_hop_ip'])
        cdn_ip_set = set(each_msm['all_hops'][-1]['hop_ips'])

        if len(pop_ip_set) == 0:
            continue
        
        if len(pop_ip_set) != 1:
            print(f"More than 2 PoP IPs in the same hop. Probe id: {prb_id}, domain: {each_msm['domain']}")
            pop_ip_set = {list(pop_ip_set)[0],}
            print(f"Continuing with PoP IP: {list(pop_ip_set)[0]}")


        ## Create new probe in probes if not done already
        prb_id = each_msm['prb_id']
        if prb_id not in probes_cf_path_old:
            probes_cf_path_old[prb_id] = {
                'id' : prb_id,
                'country_code' : each_msm['country_code'],
                'paths': {},
            }

        ## Create first hop for the probe as starting with the pop node if not done already
        pop_ip = next(iter(pop_ip_set))
        pop_asn = 14593
        domain = each_msm['domain']
        if domain not in probes_cf_path_old[prb_id]['paths']:
            probes_cf_path_old[prb_id]['paths'][domain] = {}
        if pop_ip not in probes_cf_path_old[prb_id]['paths'][domain]:
                country = None
                if pop_ip in ip_to_country_dict:
                    country = ip_to_country_dict[pop_ip]
                else:
                    geo_details = find_ip(pop_ip,ip_to_country_df)
                    if geo_details is not None:
                        if 'country_code' in geo_details:
                            country = geo_details['country_code']
                            ip_to_country_dict[pop_ip] = country
                    else:
                        ip_to_country_dict[pop_ip] = None
                probes_cf_path_old[prb_id]['paths'][domain][pop_ip] = Hop(
                    ip=pop_ip,
                    type='PoP',
                    min_rtt=min(each_msm['pop_rtts']),
                    asn=pop_asn,
                    hostname=None,
                    country=country,
                    location=None,
                )
        else:
            this_min_rtt = min(each_msm['pop_rtts'])
            prev_min_rtt = probes_cf_path_old[prb_id]['paths'][domain][pop_ip].min_rtt
            if this_min_rtt < prev_min_rtt:
                probes_cf_path_old[prb_id]['paths'][domain][pop_ip].min_rtt = this_min_rtt

        pop_reached = False
        hopNode = probes_cf_path_old[prb_id]['paths'][domain][pop_ip]
        
        ## Go through all the hops
        for idx, each_hop in enumerate(each_msm['all_hops']):
            hop_ip_set = set(each_hop['hop_ips'])
            hop_id = each_hop['hop_id']
            min_hop_rtt = min(each_hop['hop_rtts'])

            if pop_ip_set.intersection(hop_ip_set):
                pop_reached = True
                continue

            
            
            if pop_reached:
                if len(hop_ip_set) != 1:
                    print(f"More than 2 destinations in the same hop. Probe id: {prb_id}, domain: {each_msm['domain']}, hop id: {hop_id}")
                    
                    print("Check if the hop ips are part of the same network address")
                    hop_ip_network_count = {}
                    hop_ip_network_min_rtt = {}
                    for uni_idx, unique_hop_ip in enumerate(each_hop['hop_ips']):
                        unique_hop_ip_obj = ipaddress.ip_interface(unique_hop_ip + '/24')
                        unique_hop_network = str(unique_hop_ip_obj.network.network_address)
                      
                        if unique_hop_network not in hop_ip_network_count:
                            hop_ip_network_count[unique_hop_network] = 1
                            hop_ip_network_min_rtt[unique_hop_network] = each_hop['hop_rtts'][uni_idx]
                        else:
                            continue
                        for each_idx, each_ip in enumerate(each_hop['hop_ips']):
                            if each_idx == uni_idx:
                                continue
                            each_ip_obj = ipaddress.ip_interface(each_ip + '/24')
                            each_ip_network = str(each_ip_obj.network.network_address)
                            if each_ip_network == unique_hop_network:
                                hop_ip_network_count[unique_hop_network] += 1
                                if each_hop['hop_rtts'][each_idx] <  hop_ip_network_min_rtt[unique_hop_network]:
                                    hop_ip_network_min_rtt[unique_hop_network] = each_hop['hop_rtts'][each_idx]
                    
                    max_count = max(hop_ip_network_count.values())
                    if max_count < 2:
                        print("Could not find a majority network address. Skipping measurement.")
                        break
                    max_key = max(hop_ip_network_count, key=hop_ip_network_count.get)
                    hop_ip_set = {max_key,}
                    min_hop_rtt = hop_ip_network_min_rtt[max_key]


                need_new_child_node = True
                hop_ip = next(iter(hop_ip_set))

                if not ipaddress.ip_address(hop_ip).is_private:
                    all_public_ips.append(hop_ip)

                hopChildNode = None
                if hopNode.has_children():
                    hopChildNodes = hopNode.get_children()
                    for node in hopChildNodes:
                        
                        if node.ip == hop_ip:
                            hopChildNode = node
                            need_new_child_node = False
                
                if need_new_child_node:
                    hop_type = 'Node'
                    if hop_ip in cdn_ip_set:
                        hop_type = 'CDN'
                    asn_info, _ = asndb.lookup(hop_ip)
                    country = None
                    if hop_ip in ip_to_country_dict:
                        country = ip_to_country_dict[hop_ip]
                    else:
                        geo_details = find_ip(hop_ip,ip_to_country_df)
                        if geo_details is not None:
                            if 'country_code' in geo_details:
                                country = geo_details['country_code']
                                ip_to_country_dict[hop_ip] = country
                        else:
                            ip_to_country_dict[hop_ip] = None

                    newChildNode = Hop(
                        ip=hop_ip,
                        type=hop_type,
                        min_rtt=min_hop_rtt,
                        asn=asn_info,
                        hostname=None,
                        country=country,
                        location=None,
                    )
                    hopNode.add_children(newChildNode)
                    hopChildNode = newChildNode
                else:
                    this_min_rtt = min_hop_rtt
                    prev_min_rtt = hopChildNode.min_rtt
                    if this_min_rtt < prev_min_rtt:
                        hopChildNode.min_rtt = this_min_rtt
                hopNode = hopChildNode

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"

cdn_country_wise_cdn_rtts_stl = {}
cdn_country_wise_pop_rtts_stl = {}

cdn_prb_wise_pop_rtts_stl = {}
cdn_prb_wise_cdn_rtts_stl = {}

cf_country_wise_cdn_rtts_stl = {}
cf_country_wise_pop_rtts_stl = {}

akamai_country_wise_cdn_rtts_stl = {}
akamai_country_wise_pop_rtts_stl = {}

cloudfront_country_wise_cdn_rtts_stl = {}
cloudfront_country_wise_pop_rtts_stl = {}


server = "cloudflare"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
acc_tracert_filename = f"acc_trace_{server}_final_result.json"
acc_tracert_filepath = os.path.join(full_msms_path,'acc_result',acc_tracert_filename)
with open(acc_tracert_filepath, "r") as json_file:
    acc_tracert_json = json.load(json_file)
print(f"Adding measurements from {acc_tracert_filepath}")

cf_msms_len = len(acc_tracert_json)
print(f"Total cloudflare measurements= {cf_msms_len}")

for each_msm in acc_tracert_json:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    pop_rtts = each_msm['pop_rtts']
    cdn_rtts = each_msm['total_rtts']

    if prb_id not in cdn_prb_wise_cdn_rtts_stl:
        cdn_prb_wise_cdn_rtts_stl[prb_id] = []
        cdn_prb_wise_pop_rtts_stl[prb_id] = []

    if country_code not in cdn_country_wise_cdn_rtts_stl:
        cdn_country_wise_cdn_rtts_stl[country_code] = []
        cdn_country_wise_pop_rtts_stl[country_code] = []

    if country_code not in cf_country_wise_cdn_rtts_stl:
        cf_country_wise_cdn_rtts_stl[country_code] = []
        cf_country_wise_pop_rtts_stl[country_code] = []
    
    if len(cdn_rtts) and len(pop_rtts):
        if statistics.mean(cdn_rtts) > statistics.mean(pop_rtts):
            min_cdn_rtt = min(cdn_rtts)
            min_pop_rtt = min(pop_rtts)
            if min_cdn_rtt > 15 and min_pop_rtt > 15: # removing outliers and false positives
                cf_country_wise_cdn_rtts_stl[country_code].append(min_cdn_rtt)
                cf_country_wise_pop_rtts_stl[country_code].append(min_pop_rtt)
                cdn_country_wise_cdn_rtts_stl[country_code].append(min_cdn_rtt)
                cdn_country_wise_pop_rtts_stl[country_code].append(min_pop_rtt)
                cdn_prb_wise_cdn_rtts_stl[prb_id].append(min_cdn_rtt)
                cdn_prb_wise_pop_rtts_stl[prb_id].append(min_pop_rtt)


server = "akamai"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
acc_tracert_filename = f"acc_trace_{server}_final_result.json"
acc_tracert_filepath = os.path.join(full_msms_path,'acc_result',acc_tracert_filename)
with open(acc_tracert_filepath, "r") as json_file:
    acc_tracert_json = json.load(json_file)
print(f"Adding measurements from {acc_tracert_filepath}")

akamai_msms_len = len(acc_tracert_json)
print(f"Total cloudflare measurements= {akamai_msms_len}")

for each_msm in acc_tracert_json:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    pop_rtts = each_msm['pop_rtts']
    cdn_rtts = each_msm['total_rtts']

    if prb_id not in cdn_prb_wise_cdn_rtts_stl:
        cdn_prb_wise_cdn_rtts_stl[prb_id] = []
        cdn_prb_wise_pop_rtts_stl[prb_id] = []

    if country_code not in cdn_country_wise_cdn_rtts_stl:
        cdn_country_wise_cdn_rtts_stl[country_code] = []
        cdn_country_wise_pop_rtts_stl[country_code] = []

    if country_code not in akamai_country_wise_cdn_rtts_stl:
        akamai_country_wise_cdn_rtts_stl[country_code] = []
        akamai_country_wise_pop_rtts_stl[country_code] = []
    
    if len(cdn_rtts) and len(pop_rtts):
        if statistics.mean(cdn_rtts) > statistics.mean(pop_rtts):
            min_cdn_rtt = min(cdn_rtts)
            min_pop_rtt = min(pop_rtts)
            if min_cdn_rtt > 15 and min_pop_rtt > 15: # removing outliers and false positives
                akamai_country_wise_cdn_rtts_stl[country_code].append(min_cdn_rtt)
                akamai_country_wise_pop_rtts_stl[country_code].append(min_pop_rtt)
                cdn_country_wise_cdn_rtts_stl[country_code].append(min_cdn_rtt)
                cdn_country_wise_pop_rtts_stl[country_code].append(min_pop_rtt)
                cdn_prb_wise_cdn_rtts_stl[prb_id].append(min_cdn_rtt)
                cdn_prb_wise_pop_rtts_stl[prb_id].append(min_pop_rtt)

server = "cloudfront"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
acc_tracert_filename = f"acc_trace_{server}_final_result.json"
acc_tracert_filepath = os.path.join(full_msms_path,'acc_result',acc_tracert_filename)
with open(acc_tracert_filepath, "r") as json_file:
    acc_tracert_json = json.load(json_file)
print(f"Adding measurements from {acc_tracert_filepath}")

cloudfront_msms_len = len(acc_tracert_json)
print(f"Total cloudront measurements= {cloudfront_msms_len}")

for each_msm in acc_tracert_json:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    pop_rtts = each_msm['pop_rtts']
    cdn_rtts = each_msm['total_rtts']

    if prb_id not in cdn_prb_wise_cdn_rtts_stl:
        cdn_prb_wise_cdn_rtts_stl[prb_id] = []
        cdn_prb_wise_pop_rtts_stl[prb_id] = []
        
    if country_code not in cdn_country_wise_cdn_rtts_stl:
        cdn_country_wise_cdn_rtts_stl[country_code] = []
        cdn_country_wise_pop_rtts_stl[country_code] = []

    if country_code not in cloudfront_country_wise_cdn_rtts_stl:
        cloudfront_country_wise_cdn_rtts_stl[country_code] = []
        cloudfront_country_wise_pop_rtts_stl[country_code] = []
    
    if len(cdn_rtts) and len(pop_rtts):
        if statistics.mean(cdn_rtts) > statistics.mean(pop_rtts):
            min_cdn_rtt = min(cdn_rtts)
            min_pop_rtt = min(pop_rtts)
            if min_cdn_rtt > 15 and min_pop_rtt > 15: # removing outliers and false positives
                cloudfront_country_wise_cdn_rtts_stl[country_code].append(min_cdn_rtt)
                cloudfront_country_wise_pop_rtts_stl[country_code].append(min_pop_rtt)
                cdn_country_wise_cdn_rtts_stl[country_code].append(min_cdn_rtt)
                cdn_country_wise_pop_rtts_stl[country_code].append(min_pop_rtt)
                cdn_prb_wise_cdn_rtts_stl[prb_id].append(min_cdn_rtt)
                cdn_prb_wise_pop_rtts_stl[prb_id].append(min_pop_rtt)

In [ ]:
network_type = "Starlink"
dns_server = "google"
msm_type = "dns"

acc_result_google_CHAOS_filepath_stl = os.path.join(old_top_level_folder_path,network_type,msm_type,dns_server,"CHAOS","acc_result",f"acc_dns_{dns_server}_CHAOS_final_result.json")
with open(acc_result_google_CHAOS_filepath_stl,"r") as file:
    try:
        google_CHAOS_data_stl = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_google_CHAOS_filepath_stl, "Error: ", err)

In [ ]:
google_dns_prb_resolver_location_rtt = {}
for prb_tuple_str, result_dict in google_CHAOS_data_stl.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    
    if prb_id not in google_dns_prb_resolver_location_rtt:
        google_dns_prb_resolver_location_rtt[prb_id] = {
            'country_code': country_code,
            'dns_resolver': {}
        }
    
    for idx, response_time in enumerate(result_dict['response_time']):
        
        if response_time > 15:
            
            dns_loc = result_dict['resolver_location'][idx]
            dns_iata = None
            if dns_loc.startswith('gpdns'):
                dns_iata_tuple = dns_loc.split('-')
                dns_iata = dns_iata_tuple[1]
            else:
                print(f"Probe: {prb_id}, Country: {country_code}, Resolver Location: {dns_loc} ")
            if dns_iata is not None:
                if dns_iata == 'mil':
                    dns_iata = 'mxp'
                if dns_iata.upper() not in google_dns_prb_resolver_location_rtt[prb_id]['dns_resolver']:
                    google_dns_prb_resolver_location_rtt[prb_id]['dns_resolver'][dns_iata.upper()] = []
                google_dns_prb_resolver_location_rtt[prb_id]['dns_resolver'][dns_iata.upper()].append(response_time)


In [ ]:
import airportsdata

def get_city_from_iata_airportsdata(iata_code):

  if iata_code == 'MIL':
    return 'Milan'

  airports = airportsdata.load('IATA')
  if iata_code in airports:
    city = airports[iata_code]['city']
    if city == 'Newark':
      return 'New York'
    return airports[iata_code]['city']
  else:
    return None

In [ ]:
pop_to_loc_map_filename = "../helper/pop_to_location.json"
pop_to_loc = {}
with open(pop_to_loc_map_filename,'r') as pop_loc_map_file:
    pop_to_loc = json.load(pop_loc_map_file)

def get_PoP_Location(prb_id):
    loc_set = set()
    pop_not_known_set = set()
    if prb_id in probes_cf_path_old:
        probe_details  = probes_cf_path_old[prb_id]
        if 'www.broadcom.com' in probe_details['paths']:
            pop_ip_set = list(probe_details['paths']['www.broadcom.com'].keys())
            for pop_ip in pop_ip_set:
                if pop_ip in pop_to_loc:
                        loc_set.add(pop_to_loc[pop_ip])
                else:
                    pop_not_known_set.add(pop_ip)
            return loc_set, pop_not_known_set
    else:
         print(f"{prb_id} not in probes_cf_path_old")

    return None, None

In [ ]:
pop_addtnl_distance_to_google_dns_rtts = {}
prb_with_unknwn_pop_ips = []
for prb_id in google_dns_prb_resolver_location_rtt:
    country_code = google_dns_prb_resolver_location_rtt[prb_id]['country_code']
    prb_dns_dict = google_dns_prb_resolver_location_rtt[prb_id]['dns_resolver']
    pop_loc, pop_not_known = get_PoP_Location(int(prb_id))

    if pop_loc:
        pop_list = list(pop_loc)
        pop_dns_median_rt = None
        if len(pop_list) == 1 and len(pop_not_known) == 0:
            dns_loc_far_set = set()
            for dns_iata in prb_dns_dict:

                dns_loc = None
                if dns_iata in iata_to_city_dict:
                    dns_loc = iata_to_city_dict[dns_iata]
                else:
                    dns_loc = get_city_from_iata_airportsdata(dns_iata)

                if dns_iata not in iata_to_city_dict:
                    iata_to_city_dict[dns_iata] = dns_loc
                if dns_loc not in city_to_iata_dict:
                    city_to_iata_dict[dns_loc] = dns_iata

                if dns_loc in pop_list[0] or pop_list[0] in dns_loc:
                    dns_rt = prb_dns_dict[dns_iata]
                    pop_median_dns_rt = statistics.median(dns_rt)
                else:
                    dns_loc_far_set.add(dns_loc)
            
            if int(prb_id) in cdn_prb_wise_pop_rtts_stl:
                pop_median_minRTT = statistics.median(cdn_prb_wise_pop_rtts_stl[int(prb_id)])
            else:
                pop_median_minRTT = 0

            if pop_median_minRTT:
                for other_dns_loc in dns_loc_far_set:
                    if pop_list[0] not in pop_addtnl_distance_to_google_dns_rtts:
                        pop_addtnl_distance_to_google_dns_rtts[pop_list[0]] = {}
                    if other_dns_loc not in pop_addtnl_distance_to_google_dns_rtts[pop_list[0]]:
                        pop_addtnl_distance_to_google_dns_rtts[pop_list[0]][other_dns_loc] = []
                    pop_addtnl_distance_to_google_dns_rtts[pop_list[0]][other_dns_loc] = [rt - pop_median_minRTT for rt in google_dns_prb_resolver_location_rtt[prb_id]['dns_resolver'][city_to_iata_dict[other_dns_loc]] if rt > pop_median_minRTT]
                  
                if len(dns_loc_far_set) > 0:
                    print(f"Probe: {prb_id}, Country: {country_code}, DNS: {dns_loc_far_set}, PoP Loc: {pop_loc}")
    else:
        prb_with_unknwn_pop_ips.append(prb_id)

In [ ]:
network_type = "Starlink"
dns_server = "cloudflare"
msm_type = "dns"

acc_result_cf_CHAOS_filepath_stl = os.path.join(old_top_level_folder_path,network_type,msm_type,dns_server,"CHAOS","acc_result",f"acc_dns_{dns_server}_CHAOS_final_result.json")
with open(acc_result_cf_CHAOS_filepath_stl,"r") as file:
    try:
        cf_CHAOS_data_stl = json.load(file)
        print(f"JSON Data loaded with {dns_server} DNS accumulated results!")
    except ValueError as err:
        print("Skipping invalid json:, ", acc_result_cf_CHAOS_filepath_stl, "Error: ", err)

In [ ]:
cf_dns_prb_resolver_location_rtt = {}
for prb_tuple_str, result_dict in cf_CHAOS_data_stl.items():
    prb_tuple = ast.literal_eval(prb_tuple_str)
    prb_id = prb_tuple[0]
    country_code = prb_tuple[1]
    
    if prb_id not in cf_dns_prb_resolver_location_rtt:
        cf_dns_prb_resolver_location_rtt[prb_id] = {
            'country_code': country_code,
            'dns_resolver': {}
        }
    
    for idx, response_time in enumerate(result_dict['response_time']):
        if response_time > 15:
            dns_loc = result_dict['resolver_location'][idx]
            if dns_loc not in cf_dns_prb_resolver_location_rtt[prb_id]['dns_resolver']:
                cf_dns_prb_resolver_location_rtt[prb_id]['dns_resolver'][dns_loc] = []
            cf_dns_prb_resolver_location_rtt[prb_id]['dns_resolver'][dns_loc].append(response_time)

In [ ]:
pop_addtnl_distance_to_cf_dns_rtts = {}
for prb_id in cf_dns_prb_resolver_location_rtt:
    country_code = cf_dns_prb_resolver_location_rtt[prb_id]['country_code']
    prb_dns_dict = cf_dns_prb_resolver_location_rtt[prb_id]['dns_resolver']
    pop_loc, pop_not_known = get_PoP_Location(int(prb_id))
    if pop_loc:
        pop_list = list(pop_loc)
        pop_median_dns_rt = None
        if len(pop_list) == 1 and len(pop_not_known) == 0:
            dns_loc_far_set = set()
            for dns_iata in prb_dns_dict:
                
                dns_loc = get_city_from_iata_airportsdata(dns_iata)
                if dns_iata not in iata_to_city_dict:
                    iata_to_city_dict[dns_iata] = dns_loc
                if dns_loc not in city_to_iata_dict:
                    city_to_iata_dict[dns_loc] = dns_iata
                if dns_loc in pop_list[0] or pop_list[0] in dns_loc:
                    dns_rt = prb_dns_dict[dns_iata]
                    pop_median_dns_rt = statistics.median(dns_rt)
                else:
                    dns_loc_far_set.add(dns_loc)
            


            if pop_median_dns_rt:
                for other_dns_loc in dns_loc_far_set:
                    if pop_list[0] not in pop_addtnl_distance_to_cf_dns_rtts:
                        pop_addtnl_distance_to_cf_dns_rtts[pop_list[0]] = {}
                    if other_dns_loc not in pop_addtnl_distance_to_cf_dns_rtts[pop_list[0]]:
                        pop_addtnl_distance_to_cf_dns_rtts[pop_list[0]][other_dns_loc] = []
                    pop_addtnl_distance_to_cf_dns_rtts[pop_list[0]][other_dns_loc] = [rt - pop_median_dns_rt for rt in cf_dns_prb_resolver_location_rtt[prb_id]['dns_resolver'][city_to_iata_dict[other_dns_loc]] if rt > pop_median_dns_rt]
                  
                if len(dns_loc_far_set) > 0:
                    print(f"Probe: {prb_id}, Country: {country_code}, DNS: {dns_loc_far_set}, PoP Loc: {pop_loc}")
        

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
cols = plt.get_cmap("tab10").colors

fig, ax = plt.subplots(figsize=(4.66, 2))
pop_locs = ['Miami', 'Chicago', 'Milan', 'Jakarta', 'Lagos', 'Madrid']

idx = 0
for popLoc in pop_locs:
    addnlDnsRTList = []
    dns_resolver_loc = []
    if popLoc in pop_addtnl_distance_to_google_dns_rtts:
        for dnsLoc, dnsRTList in pop_addtnl_distance_to_google_dns_rtts[popLoc].items():
            
            if len(dnsRTList) > 50:
                addnlDnsRTList += dnsRTList
                dns_resolver_loc.append(dnsLoc)
                print(f"{dnsLoc}: {len(dnsRTList)}")
    if popLoc in pop_addtnl_distance_to_cf_dns_rtts:
        for dnsLoc, dnsRTList in pop_addtnl_distance_to_cf_dns_rtts[popLoc].items():

            if len(dnsRTList) > 50:
                addnlDnsRTList += dnsRTList
                dns_resolver_loc.append(dnsLoc)
                print(f"{dnsLoc}: {len(dnsRTList)}")


    if len(addnlDnsRTList) > 50:
        print(f"PoP Loc: {popLoc}, DNS Locs: {dns_resolver_loc}")
    
        xs = np.sort(addnlDnsRTList)
        ys = np.arange(1, len(xs) + 1) / len(xs)
        indices = []
        current = xs[0]
        for i, x in enumerate(xs): # only take max y value at each x value to smoothen out the graph
            if x != current:
                current = x
                indices.append(i - 1)
        indices.append(len(ys) - 1)
        xs = sorted(set(xs))
        ys = [ys[i] for i in indices]
        ax.plot(xs, ys, label=popLoc, color=cols[idx], linestyle="solid")
        idx += 1

ax.set_xlabel("Additional DNS Response Time [ms]")
ax.set_ylabel("Percentile")
ax.set_yticks(np.arange(0, 1.20, 0.25))

ax.xaxis.get_major_formatter()._usetex = False
ax.yaxis.get_major_formatter()._usetex = False
ax.set_xlim(0,110)
ax.set_ylim(0,1)
ax.legend(loc="lower right", fontsize="small", ncol=2, edgecolor="k", handlelength=1, labelspacing=0.06,
          columnspacing=0.5, handletextpad=0.3, fancybox=False)
plt.grid(True, axis='y', linestyle='--', alpha=0.7, linewidth=0.5)

fig.tight_layout()

plt.show()

## Akamai - Africa - Starlink vs Terrestrial

In [ ]:
user_type = "Starlink"
msm_type = "traceroute"
server = "akamai"

acc_result_filename = f"acc_trace_{server}_final_result.json"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)
print(filepath)

akamai_accumulated_trace_json_old = None
with open(filepath, "r") as json_file:
    akamai_accumulated_trace_json_old = json.load(json_file)
    print("Accumulated Traceroute JSON file loaded successfully")

In [ ]:
from datetime import datetime
controlled_nodes_traceroute_files_path = f"{controlled_top_level_folder_path}/traceroute/08-27-2025-12-31-18_WebTrace_files_list"

cdns_list = ["akamai"]

for cdn_name in cdns_list:
    acc_tracert_filename = f"controlled_nodes_acc_trace_{cdn_name}_final_result_late.json"
    acc_tracert_filepath = os.path.join(controlled_nodes_traceroute_files_path,'acc_result',acc_tracert_filename)
    with open(acc_tracert_filepath, "r") as json_file:
        acc_tracert_json = json.load(json_file)
        akamai_accumulated_trace_json_old += acc_tracert_json

In [ ]:
user_type = "Terrestrial"
msm_type = "traceroute"
server = "akamai"

acc_result_filename = f"acc_trace_{server}_final_result.json"
full_msms_path = os.path.join(old_top_level_folder_path,user_type,msm_type,server)
filepath = os.path.join(full_msms_path,'acc_result',acc_result_filename)
print(filepath)

akamai_accumulated_trace_json_old_TERR = None
with open(filepath, "r") as json_file:
    akamai_accumulated_trace_json_old_TERR = json.load(json_file)
    print("Accumulated Traceroute JSON file loaded successfully")

In [ ]:
akamai_AF_country_wise_cdn_rtts_stl = {}
akamai_AF_country_wise_pop_rtts_stl = {}

idx = 0
for each_msm in akamai_accumulated_trace_json_old:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    dst_addr = each_msm['dst_addr']

    if country_code not in ['BJ','MG','ZM']:
        continue
    
    total_dst_rtts = []
    for idx, hop_ip in enumerate(each_msm['all_hops'][-1]['hop_ips']):
        if hop_ip == dst_addr:
            total_dst_rtts.append(each_msm['all_hops'][-1]['hop_rtts'][idx])
    
    if country_code not in akamai_AF_country_wise_cdn_rtts_stl:
        akamai_AF_country_wise_cdn_rtts_stl[country_code] = []
        akamai_AF_country_wise_pop_rtts_stl[country_code] = []

    if len(total_dst_rtts):
        akamai_AF_country_wise_cdn_rtts_stl[country_code].append(min(total_dst_rtts))
    
    if len(total_dst_rtts) and len(each_msm['pop_rtts']):
        if statistics.mean(total_dst_rtts) > statistics.mean(each_msm['pop_rtts']):
            akamai_AF_country_wise_pop_rtts_stl[country_code].append(min(each_msm['pop_rtts']))

In [ ]:
akamai_AF_country_wise_cdn_rtts_stl_TERR = {}
akamai_AF_country_wise_pop_rtts_stl_TERR = {}

idx = 0
for each_msm in akamai_accumulated_trace_json_old_TERR:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    dst_addr = each_msm['dst_addr']

    if country_code not in ['BJ','MG','ZM']:
        continue
    
    total_dst_rtts = []
    for idx, hop_ip in enumerate(each_msm['all_hops'][-1]['hop_ips']):
        if hop_ip == dst_addr:
            total_dst_rtts.append(each_msm['all_hops'][-1]['hop_rtts'][idx])
    
    if country_code not in akamai_AF_country_wise_cdn_rtts_stl_TERR:
        akamai_AF_country_wise_cdn_rtts_stl_TERR[country_code] = []
        akamai_AF_country_wise_pop_rtts_stl_TERR[country_code] = []

    if len(total_dst_rtts):
        akamai_AF_country_wise_cdn_rtts_stl_TERR[country_code].append(min(total_dst_rtts))
    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
rcParams['font.size'] = 10


country_colors = {
    "BJ" : cols[3],
    "MG" : cols[4],
    "ZM" : cols[2],
}

african_countries = ["MG", "ZM", "BJ"]
fig, ax = plt.subplots(figsize=(4.66, 2))


handles = []
labels = []
for country in african_countries:
    country_rt_TERR_list = []
    country_rt_TERR_list += akamai_AF_country_wise_cdn_rtts_stl_TERR[country]
    if country_rt_TERR_list:

        xs = np.sort(country_rt_TERR_list)
        ys = np.arange(1, len(xs) + 1) / len(xs)
        indices = []
        current = xs[0]
        for i, x in enumerate(xs): # only take max y value at each x value to smoothen out the graph
            if x != current:
                current = x
                indices.append(i - 1)
        indices.append(len(ys) - 1)
        xs = sorted(set(xs))
        ys = [ys[i] for i in indices]
        ax.plot(xs, ys, label=f"{country}-STL", color=country_colors[country], linestyle="dotted")

    country_rt_STL_list = []
    country_rt_STL_list += akamai_AF_country_wise_cdn_rtts_stl[country]
    if country_rt_STL_list:

        xs = np.sort(country_rt_STL_list)
        ys = np.arange(1, len(xs) + 1) / len(xs)
        indices = []
        current = xs[0]
        for i, x in enumerate(xs): # only take max y value at each x value to smoothen out the graph
            if x != current:
                current = x
                indices.append(i - 1)
        indices.append(len(ys) - 1)
        xs = sorted(set(xs))
        ys = [ys[i] for i in indices]
        ax.plot(xs, ys, label=f"{country}-STL", color=country_colors[country], linestyle="solid")


    handles.append(Patch(facecolor=country_colors[country]))
    labels.append(country)

ax.set_xlabel("CDN RTT [ms]")
ax.set_ylabel("Percentile")
ax.set_yticks(np.arange(0, 1.20, 0.25))
ax.set_xticks(np.arange(0, 600, 50))
ax.xaxis.get_major_formatter()._usetex = False
ax.yaxis.get_major_formatter()._usetex = False


solid_line = Line2D([0], [0], color='black', linestyle='-', label='Starlink')
dashed_line = Line2D([0], [0], color='black', linestyle=':', label='Terrestrial')

net_handles = [solid_line, dashed_line]
net_labels = ['Starlink', 'Terrestrial']

country_legend  = ax.legend(handles, labels, loc="lower right",  fontsize="small", edgecolor="k",  handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4)
ax.add_artist(country_legend)
ax.legend(net_handles, net_labels,loc="lower right", bbox_to_anchor=(0.65, 0.94), fontsize="x-small", edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3,frameon=False, fancybox=False, ncol=4) # bbox_to_anchor=(0.25, 0.91), 

plt.grid(True, axis='y', linestyle='--', alpha=0.7, linewidth=0.5)


ax.set_xlim(0,350)
ax.set_ylim(0,1.0)
fig.tight_layout()
plt.savefig(f"{figs_top_level_folder_path}/cf_cdn_africa_STAR_vs_TERR_cdf.pdf", bbox_inches = "tight", pad_inches = 0)
plt.show()

In [ ]:
ip_to_country_dict = {}

In [ ]:
AF_country_to_cdn_dest_stl = {
    'BJ' : {},
    'MG' : {}
}

idx_outer = 0
for each_msm in akamai_accumulated_trace_json_old:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    dst_addr = each_msm['dst_addr']
    idx_outer += 1
    print(f"{idx_outer+1} of {len(akamai_accumulated_trace_json_old)}")
    if country_code not in ['BJ','MG']:
        continue
    
    for idx, hop_ip in enumerate(each_msm['all_hops'][-1]['hop_ips']):
        if hop_ip == dst_addr:
            if dst_addr in ip_to_country_dict:
                cdn_country = ip_to_country_dict[dst_addr]
            else:
                ip_details = find_ip(dst_addr,ip_to_country_df)
                if 'country_code' in ip_details:
                    cdn_country = ip_details['country_code']
                else:
                    cdn_country = 'None'
                ip_to_country_dict[dst_addr] = cdn_country

            if cdn_country not in AF_country_to_cdn_dest_stl[country_code]:
                AF_country_to_cdn_dest_stl[country_code][cdn_country] = 0
            AF_country_to_cdn_dest_stl[country_code][cdn_country] += 1
    

In [ ]:
AF_country_to_cdn_dest_TERR = {
    'BJ' : {},
    'MG' : {}
}

idx_outer = 0
for each_msm in akamai_accumulated_trace_json_old_TERR:
    country_code = each_msm['country_code']
    prb_id = each_msm['prb_id']
    dst_addr = each_msm['dst_addr']
    idx_outer += 1
    print(f"{idx_outer+1} of {len(akamai_accumulated_trace_json_old_TERR)}")
    if country_code not in ['BJ','MG']:
        continue
    
    for idx, hop_ip in enumerate(each_msm['all_hops'][-1]['hop_ips']):
        if hop_ip == dst_addr:
            if dst_addr in ip_to_country_dict:
                cdn_country = ip_to_country_dict[dst_addr]
            else:
                ip_details = find_ip(dst_addr,ip_to_country_df)
                if 'country_code' in ip_details:
                    cdn_country = ip_details['country_code']
                else:
                    cdn_country = 'None'
                ip_to_country_dict[dst_addr] = cdn_country

            if cdn_country not in AF_country_to_cdn_dest_TERR[country_code]:
                AF_country_to_cdn_dest_TERR[country_code][cdn_country] = 0
            AF_country_to_cdn_dest_TERR[country_code][cdn_country] += 1

In [ ]:
rcParams['font.size'] = 12
chosen_country_code = 'MG'
starlink_count = __builtins__.sum(AF_country_to_cdn_dest_stl[chosen_country_code].values())
terrestrial_count = __builtins__.sum(AF_country_to_cdn_dest_TERR[chosen_country_code].values())

starlink_percent_dict = {k: round((v/starlink_count) * 100) for k, v in AF_country_to_cdn_dest_stl[chosen_country_code].items() if round((v/starlink_count) * 100) >=1}
terrestrial_percent_dict = {k: round((v/terrestrial_count) * 100) for k, v in AF_country_to_cdn_dest_TERR[chosen_country_code].items() if round((v/terrestrial_count) * 100) >=4}

sorted_starlink_percent_dict = sorted(starlink_percent_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_terrestrial_percent_dict = sorted(terrestrial_percent_dict.items(), key=operator.itemgetter(1), reverse=True)


color_map = {
    "FR": cols[0],
    "PL": cols[1],
    "ZA": cols[2],
    "ES": cols[3],
    "IT": cols[4],
    "DE": cols[5],
    "US": cols[6],
    "NL": cols[7],
    "TR": cols[8],
    "GB": cols[9]
}


space = 0.8
x_A = np.arange(len(starlink_percent_dict))
x_B = np.arange(len(terrestrial_percent_dict)) + len(starlink_percent_dict) + space
width = 0.8
fig, ax = plt.subplots(figsize=(3.37, 3.37))

print(starlink_percent_dict)
print(terrestrial_percent_dict)

stl_countries = []
terr_countries = []
for i, (country, percent) in enumerate(sorted_starlink_percent_dict):
    stl_countries.append(country)
    ax.bar(x_A[i],percent, width,color=color_map.get(country,"gray"), alpha=0.7, label=country)

for i, (country, percent) in enumerate(sorted_terrestrial_percent_dict):
    terr_countries.append(country)
    ax.bar(x_B[i],percent, width,color=color_map.get(country,"gray"), alpha=0.7, label=country)


all_labels = stl_countries + terr_countries
x_ticks = np.concatenate([x_A, x_B])
ax.set_xticks(x_ticks)
ax.set_xticklabels(all_labels, rotation=0, ha="center", fontsize=10)
ax.set_ylabel("Percentage")

ax.text(-0.5,-15,"Starlink")
ax.text(4.1,-15,"Terrestrial")
plt.grid(True, axis='y', linestyle='-', alpha=0.7, linewidth=0.5)
plt.savefig(f"{figs_top_level_folder_path}/{chosen_country_code}_resolved_ip_loc_akamai_domains.pdf", bbox_inches = "tight", pad_inches = 0.1)

plt.tight_layout()
plt.show()


In [ ]:
rcParams['font.size'] = 12
chosen_country_code = 'BJ'
starlink_count = __builtins__.sum(AF_country_to_cdn_dest_stl[chosen_country_code].values())
terrestrial_count = __builtins__.sum(AF_country_to_cdn_dest_TERR[chosen_country_code].values())

starlink_percent_dict = {k: round((v/starlink_count) * 100) for k, v in AF_country_to_cdn_dest_stl[chosen_country_code].items() if round((v/starlink_count) * 100) >=1}
terrestrial_percent_dict = {k: round((v/terrestrial_count) * 100) for k, v in AF_country_to_cdn_dest_TERR[chosen_country_code].items() if round((v/terrestrial_count) * 100) >=4}

sorted_starlink_percent_dict = sorted(starlink_percent_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_terrestrial_percent_dict = sorted(terrestrial_percent_dict.items(), key=operator.itemgetter(1), reverse=True)


color_map = {
    "FR": cols[0],
    "PL": cols[1],
    "ZA": cols[2],
    "ES": cols[3],
    "IT": cols[4],
    "DE": cols[5],
    "US": cols[6],
    "NL": cols[7],
    "TR": cols[8],
    "GB": cols[9]
}


space = 0.8
x_A = np.arange(len(starlink_percent_dict))
x_B = np.arange(len(terrestrial_percent_dict)) + len(starlink_percent_dict) + space
width = 0.8
fig, ax = plt.subplots(figsize=(3.37, 3.37))

print(starlink_percent_dict)
print(terrestrial_percent_dict)

stl_countries = []
terr_countries = []
for i, (country, percent) in enumerate(sorted_starlink_percent_dict):
    stl_countries.append(country)
    ax.bar(x_A[i],percent, width,color=color_map.get(country,"gray"), alpha=0.7, label=country)

for i, (country, percent) in enumerate(sorted_terrestrial_percent_dict):
    terr_countries.append(country)
    ax.bar(x_B[i],percent, width,color=color_map.get(country,"gray"), alpha=0.7, label=country)


all_labels = stl_countries + terr_countries
x_ticks = np.concatenate([x_A, x_B])
ax.set_xticks(x_ticks)
ax.set_xticklabels(all_labels, rotation=0, ha="center", fontsize=10)
ax.set_ylabel("Percentage")

ax.text(-0.5,-15,"Starlink")
ax.text(2.4,-15,"Terrestrial")
plt.grid(True, axis='y', linestyle='-', alpha=0.7, linewidth=0.5)
plt.savefig(f"{figs_top_level_folder_path}/{chosen_country_code}_resolved_ip_loc_akamai_domains.pdf", bbox_inches = "tight", pad_inches = 0.1)

plt.tight_layout()
plt.show()


## Cloudflare AIM Africa analysis

In [ ]:
africa_raw_jsonfile = "africa-raw-bq-results-20260121-224019-1769035354623.ndjson"
dataset_af_raw = os.path.join(cloudflare_aim_path, "all_starlink_africa.ndjson")
raw_af_df = pd.DataFrame()
# one shard
raw_af_df = pd.read_json(
    dataset_af_raw,   # or full/path/*.json.gz
    lines=True                            # NDJSON ➜ one JSON object per line
)

# 👉  verify
print(len(raw_af_df))          # row‑count
raw_af_df.head()  

In [ ]:
african_pops = {'LOS', 'CPT', 'NBO', 'JNB', 'CPT', 'MRS', 'JIB', 'DUR'}

In [ ]:
raw_af_df['year_month'] = pd.to_datetime(raw_af_df['measurementTime']).dt.to_period('M')

# 3. Compute each test’s mean latency
raw_af_df['test_latency_median'] = raw_af_df['latencyMs'].apply(np.median)

In [ ]:
# Some cleanup tasks

## only keep if download.bps and upload.bps are non-empty lists
mask_complete = (
    raw_af_df['download']
      .apply(lambda d: bool(d.get('bps')))  & 
    raw_af_df['upload']
      .apply(lambda u: bool(u.get('bps')))
)
raw_af_df = raw_af_df[mask_complete]

## only keep data where test_latency_median is greater than 10
raw_af_df = raw_af_df[raw_af_df['test_latency_median'] > 10.0]

raw_af_df.head(2)

In [ ]:
# 4. Compute the minimum test_latency_mean per (country, month, PoP)
filtered = raw_af_df.groupby(
    ['clientCountry', 'year_month', 'serverPoP']
).filter(lambda grp: len(grp) >= 5)

pop_min = (
    filtered
    .groupby(['clientCountry','year_month','serverPoP'])['test_latency_median']
    .min()
    .reset_index(name='pop_min_latency')
)

pop_median = (
    filtered
    .groupby(['clientCountry','year_month','serverPoP'])['test_latency_median']
    .median()
    .reset_index(name='pop_median_latency')
)

In [ ]:

## print the number of unique serverPoP
print(pop_min['serverPoP'].unique())

In [ ]:
print(pop_min)
print(pop_median)

In [ ]:
# 4. Restrict to PoPs *outside* Africa
pop_min_non = pop_min[~pop_min['serverPoP'].isin(african_pops)]

# 5. Pick the best (lowest‐latency) non‐African PoP per country×month
idx_non = (
    pop_min_non
    .groupby(['clientCountry','year_month'])['pop_min_latency']
    .idxmin()
)

best_nonaf = (
    pop_min_non
    .loc[idx_non, ['clientCountry','year_month','serverPoP','pop_min_latency']]
    .rename(columns={'serverPoP':'best_nonafrican_pop',
                     'pop_min_latency':'best_nonafrican_latency'})
    .reset_index(drop=True)
)

# 6. Preview that mapping
print(best_nonaf.to_string(index=False))

In [ ]:
# 5. Select the PoP with the absolute lowest latency per (country, month)
idx = pop_min.groupby(['clientCountry','year_month'])['pop_min_latency'].idxmin()
best_pops = (
    pop_min
    .loc[idx, ['clientCountry','year_month','serverPoP','pop_min_latency']]
    .rename(columns={'serverPoP':'best_pop'})
)

In [ ]:
idx = pop_median.groupby(['clientCountry','year_month'])['pop_median_latency'].idxmin()
best_pops_median = (
    pop_median
    .loc[idx, ['clientCountry','year_month','serverPoP','pop_median_latency']]
    .rename(columns={'serverPoP':'best_pop_median'})
)

Possible nice countries to analyze:
1. KE (latency reduces from 140 to 90)
2. MZ (goes to 40 over time)
3. ZM (quite significant decrease)
4. ZW (quite significant decrease)

Create loaded latency values for each country

In [ ]:
raw_af_df['loaded_latency_down'] = raw_af_df['loadedLatencyMs'].apply(
    lambda d: np.median(d.get('download', [])) 
              if isinstance(d, dict) and d.get('download') else np.nan
)

raw_af_df['loaded_latency_up'] = raw_af_df['loadedLatencyMs'].apply(
    lambda d: np.median(d.get('upload', [])) 
              if isinstance(d, dict) and d.get('upload') else np.nan
)

### Filter only for African Servers

In [ ]:
# 1. Keep all rows where serverPoP is in your African‐PoP set
mask_af = raw_af_df['serverPoP'].isin(african_pops)

# 2. Keep all rows matching the best non-African PoP mapping
mask_nonaf = raw_af_df.merge(
    best_nonaf,
    how='left',
    left_on=['clientCountry','year_month','serverPoP'],
    right_on=['clientCountry','year_month','best_nonafrican_pop']
)['best_nonafrican_pop'].notna()

# 3. Combine masks
filtered2 = raw_af_df[ mask_af | mask_nonaf ].copy()

# 4. Inspect
print(f"Kept {len(filtered2)} rows ({mask_af.sum()} via African PoPs + {mask_nonaf.sum()} via best non-Af PoPs)")
filtered2.head()

### Filter based on best_pop_median location

In [ ]:
# 2) Merge the best_pop_median mapping onto your raw data
merged = raw_af_df.merge(
    best_pops_median,   # cols: clientCountry, year_month, best_pop_median
    how='left',
    left_on=['clientCountry','year_month'],
    right_on=['clientCountry','year_month']
)

# 3) Keep only rows where serverPoP == best_pop_median
filtered_by_median = merged[
    merged['serverPoP'] == merged['best_pop_median']
].copy()

# 4) (Optional) Drop the extra columns from the merge if you like
filtered_by_median = filtered_by_median.drop(columns=['best_pop_median','pop_median_latency'])

# 5) Inspect
print(f"Kept {len(filtered_by_median)} rows out of {len(raw_af_df)}")
filtered_by_median.head()

In [ ]:
# --- 1. Base DataFrame prep
df_base = filtered_by_median.copy()

# Normalize year_month → Timestamp
if pd.api.types.is_period_dtype(df_base['year_month'].dtype):
    df_base['year_month'] = df_base['year_month'].dt.to_timestamp()
else:
    df_base['year_month'] = pd.to_datetime(
        df_base['year_month'].astype(str), format='%Y-%m'
    )

# Restrict to Nov 2024 – Apr 2025
months = pd.date_range('2024-11-01', '2025-04-01', freq='MS')
df_base = df_base[df_base['year_month'].isin(months)]
df_base['month_str'] = df_base['year_month'].dt.strftime('%Y-%m')

countries = ['KE','MZ','ZM','ZW']
colors    = ['C0','C1','C2','C3']

In [ ]:
from matplotlib.patches import Patch
from joypy import joyplot

# 1. Prepare & filter your DataFrame
df = filtered_by_median.copy()

# Normalize year_month → Timestamp
if pd.api.types.is_period_dtype(df['year_month'].dtype):
    df['year_month'] = df['year_month'].dt.to_timestamp()
else:
    df['year_month'] = pd.to_datetime(df['year_month'].astype(str), format='%Y-%m')

# Restrict to Nov 2024 – Apr 2025
months = pd.date_range('2024-11-01', '2025-04-01', freq='MS')
df = df[df['year_month'].isin(months)]

# Explode latency lists into individual rows
# df = df.explode('loaded_latency_down').dropna(subset=['loaded_latency_down'])
# df['latencyMs'] = df['loaded_latency_down'].astype(float)

# df = df[df['latencyMs'] >= 20]

# 2. Pivot into “wide” form: one column per country
countries = ['KE', 'MZ',  'ZM', 'ZW']
for c in countries:
    df[c] = df['loaded_latency_down'].where(df['clientCountry'] == c)

# Create a string‐label for the month (so joyplot uses that as the y‐axis)
df['month_str'] = df['year_month'].dt.strftime('%Y-%m')

# 3. Plot
plt.figure(figsize=(12, 8))
# 1) Disable JoyPy’s per-axis legend
fig, axes = joyplot(
    data=df[['month_str','KE','MZ','ZM','ZW']],
    by='month_str',
    column=['KE','ZM','ZW'],
    kind='normalized_counts',
    overlap=4,
    fade=False,
    alpha=0.5,
    color=['C0','C2','C3'],
    x_range=[0,200],
    legend=False,      # ← turn off its legend
    linewidth=1.2,
    bw_method=1,
    normalize=True
)

# 2) Build custom legend handles
countries = ['KE','ZM','ZW']
colors    = ['C0','C2','C3']
handles = [Patch(facecolor=col, alpha=1, label=ctr)
           for ctr, col in zip(countries, colors)]

# 3) Place a single horizontal legend above the plot

plt.legend(
    handles=handles,
    loc='upper left',
    ncol=len(handles),
    # bbox_to_anchor=(0.5, 1),
    labelspacing=0.06, 
    columnspacing=0.5, 
    handletextpad=0.3, 
    fancybox=False,
    # columnspacing=0.5,
    # handlelength=1,
    # handletextpad=0.3
)

# plt.title('Latency Distributions by Country (Nov 2024 – Apr 2025)')
plt.xlabel('Downlink latency under load (ms)')
plt.tight_layout()
# plt.savefig(
#     os.path.join(PLOT_DIR, "joyplot_down_loadedlatency_by_af_country_counts.pdf"),
#     dpi=300,
#     bbox_inches='tight',
#     pad_inches = 0.1
# )
# plt.savefig(
#     os.path.join(PLOT_DIR, "joyplot_down_loadedlatency_by_af_country_counts.svg")
# )
plt.show()

In [ ]:
# 1. Prepare your DataFrame
df = filtered_by_median.copy()
if pd.api.types.is_period_dtype(df['year_month'].dtype):
    df['year_month'] = df['year_month'].dt.to_timestamp()
else:
    df['year_month'] = pd.to_datetime(df['year_month'].astype(str), format='%Y-%m')

# explode raw latencies
df = df.explode('latencyMs').dropna(subset=['latencyMs'])
df['latencyMs'] = df['latencyMs'].astype(float)

df = df[df['latencyMs'] >= 20]

# 2. Define early and late periods by month strings
df['ym_str'] = df['year_month'].dt.strftime('%Y-%m')
early_months = ['2024-09','2024-10','2024-11']
late_months  = ['2025-03','2025-04','2025-05']

# 3. Single-axes CDF plot
plt.figure(figsize=(10,6))
countries = ['BW','MZ','ZM','SZ','MG']
colors    = {'BW':'C0','MZ':'C1','ZM':'C2','SZ':'C3', 'MG':'C4'}

for country in countries:
    # early period (dashed)
    data_e = df.loc[
        df['clientCountry'].eq(country) & df['ym_str'].isin(early_months),
        'latencyMs'
    ].values
    if data_e.size:
        d_e = np.sort(data_e)
        cdf_e = np.arange(1, len(d_e)+1) / len(d_e)
        plt.plot(d_e, cdf_e,
                 linestyle='--',
                 color=colors[country],
                 label=f'{country} Sep–Nov 2024')

    # late period (solid)
    data_l = df.loc[
        df['clientCountry'].eq(country) & df['ym_str'].isin(late_months),
        'latencyMs'
    ].values
    if data_l.size:
        d_l = np.sort(data_l)
        cdf_l = np.arange(1, len(d_l)+1) / len(d_l)
        plt.plot(d_l, cdf_l,
                 linestyle='-',
                 color=colors[country],
                 label=f'{country} Mar–May 2025')

# 4. Final formatting
plt.xlim(0, 500)
plt.ylim(0, 1)
plt.xlabel('Latency (ms)')
plt.ylabel('Cumulative Probability')
plt.title('Latency CDF: Sep–Nov 2024 (dashed) vs Mar–May 2025 (solid)')
plt.legend(loc='lower right', fontsize='small', ncol=2)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
early_months = ['2024-09','2024-10','2024-11']
late_months  = ['2025-03','2025-04','2025-05']
countries    = ['BW', 'GH','KE','MG','MW','MZ','NG','SS','SZ','ZM','ZW']
cols         = ['C0','C1']  # colors for early and late

# Gather the latency arrays
early_data = [ df.loc[(df['clientCountry']==c) & df['ym_str'].isin(early_months), 'latencyMs'].values
               for c in countries ]
late_data  = [ df.loc[(df['clientCountry']==c) & df['ym_str'].isin(late_months),  'latencyMs'].values
               for c in countries ]

# Positions for boxplots
pos1 = np.arange(0, 3*len(countries), 3)
pos2 = pos1 + 1

fig, ax = plt.subplots(figsize=(4.6,2))

# Early
ax.boxplot(
    early_data, positions=pos1, widths=0.8, patch_artist=True, showfliers=False,
    boxprops=dict(facecolor=cols[0], lw=1), medianprops=dict(color='yellow')
)
# Late
ax.boxplot(
    late_data, positions=pos2, widths=0.8, patch_artist=True, showfliers=False,
    boxprops=dict(facecolor=cols[1], lw=1), medianprops=dict(color='yellow')
)

# Alternate background shading
for i in range(len(countries)):
    if i % 2 == 1:
        ax.axvspan(pos1[i]-1, pos2[i]+1, facecolor='k', alpha=0.1)


ymin, ymax = ax.get_ylim()
yticks = np.arange(0, ymax + 1, 50)
ax.set_yticks(yticks)
ax.set_yticklabels([f"{int(y)}" for y in yticks])


# X-axis labels
ax.set_xticks((pos1 + pos2)/2)
ax.set_xticklabels(countries)
ax.set_ylabel('Latency (ms)')

ax.xaxis.get_major_formatter()._usetex = False
ax.yaxis.get_major_formatter()._usetex = False


# Legend
handles = [Patch(facecolor=cols[i], label=label) 
           for i, label in enumerate(['Sep–Nov 2024', 'Mar–May 2025'])]

ax.legend(handles, [h.get_label() for h in handles], loc="upper left", fontsize="small", edgecolor="k", handlelength=1, 
          labelspacing=0.06, columnspacing=0.5, handletextpad=0.3, fancybox=False, ncol=4)
plt.grid(True, axis="y", linestyle="--")

ax.grid(True, axis='y', linestyle='--', alpha=0.3)

plt.savefig(
    os.path.join(figs_top_level_folder_path, "cdf_af_cloudflare_all_countries.pdf"),
    dpi=300,
    bbox_inches='tight',
    pad_inches = 0.1
)
plt.tight_layout()
plt.show()

## Web Measurements

In [ ]:
import os
import json

from pathlib import Path
from datetime import datetime

import pytz
import matplotlib.pyplot as plt
from matplotlib import rcParams
from collections import defaultdict
import numpy as np

webMsms_dir = "../dataset/webMsms"



In [ ]:
plt.rc("text", usetex=True)

rcParams["font.family"] = "CMU Sans Serif"
rcParams["font.size"] = 10

cols = plt.get_cmap("tab10").colors

country_mapping = {
    "Lusaka-NBO": "ZM (NBO PoP)",
    "Lusaka-JNB": "ZM (JNB PoP)",
    "Accra": "GH (LOS PoP)",
    "Berlin": "DE (FRA PoP)",
    "Vancouver": "CA (SEA PoP)",
    "Calgary": "CA (YYC PoP)",
    "Toronto": "CA (YUL PoP)",
}
unique_cities = ["Lusaka", "Accra", "Berlin", "Vancouver", "Calgary", "Toronto"]
hit_statuses = {"HIT", "REVALIDATED"}
miss_statuses = {"MISS", "DYNAMIC", "EXPIRED", "BYPASS"}

In [ ]:
def parse_allinone(filename: str):
    data = json.load(open(filename, "r"))
    timestamps = filename.split("_")[-1].split(".")[0]
    timestamps = datetime.strptime(timestamps, "%Y%m%dT%H%M%SZ").replace(
        tzinfo=pytz.UTC
    )
    user_details = data["user_details"]
    city = user_details["City"]
    country = user_details["Country"]
    cache_result = []
    for domain, result in data["web_measurements"].items():
        curl_results = result["curl"]
        for ip, curl_resp in curl_results.items():
            initconnect_time = curl_resp.get("initconnect_time")
            appconnect_time = curl_resp.get("appconnect_time")
            pretransfer_time = curl_resp.get("pretransfer_time")
            redirect_time = curl_resp.get("redirect_time")
            starttransfer_time = curl_resp.get("starttransfer_time")
            total_time = curl_resp.get("total_time")
            cdn_server_id_key = curl_resp.get("cdn_server_id_key")
            cdn_server_id_value = curl_resp.get("cdn_server_id_value")
            cf_cache_status = curl_resp.get("cf_cache_status")
            x_cache = curl_resp.get("x_cache")
            x_cache_remote = curl_resp.get("x_cache_remote")

            r = {
                "domain": domain,
                "timestamp": timestamps,
                "ip": ip,
                "city": city,
                "country": country,
                "initconnect_time": initconnect_time,
                "appconnect_time": appconnect_time,
                "pretransfer_time": pretransfer_time,
                "redirect_time": redirect_time,
                "starttransfer_time": starttransfer_time,
                "total_time": total_time,
                "cdn_server_id_key": cdn_server_id_key,
                "cdn_server_id_value": cdn_server_id_value,
                "cf_cache_status": cf_cache_status,
                "x_cache": x_cache,
                "x_cache_remote": x_cache_remote,
            }
            cache_result.append(r)
    return cache_result, timestamps

In [ ]:
def plot(daily_city_hitrates):
    data_mean = {}
    data_std = {}

    ordered_cities = [
        "Vancouver",
        "Lusaka-NBO",
        "Lusaka-JNB",
        "Accra",
        "Berlin",
        "Calgary",
        "Toronto",
    ]
    for city in ordered_cities:
        daily_rates = daily_city_hitrates.get(city, [])
        if len(daily_rates) == 0:
            mean_hit = 0
            std_hit = 0
        else:
            mean_hit = __builtins__.sum(daily_rates) / len(daily_rates)
            std_hit = np.std(daily_rates) if len(daily_rates) > 1 else 0
        data_mean[city] = mean_hit
        data_std[city] = std_hit

    for _, city in enumerate(ordered_cities):
        mean_hit = data_mean[city]
        std_hit = data_std[city]
        print(f"{country_mapping[city]} mean={mean_hit:.3f} std={std_hit:.3f}")

### Cache Hit rates

In [ ]:
filename = []
path = Path(webMsms_dir)
for dirpath, dirnames, files in os.walk(path):
    if len(files) != 0:
        for f in files:
            if f.endswith(".json"):
                filename.append(Path(dirpath).joinpath(f))

per_day_counts = {
    "Vancouver": defaultdict(lambda: {"hit": 0, "miss": 0}),
    "Lusaka-NBO": defaultdict(lambda: {"hit": 0, "miss": 0}),
    "Lusaka-JNB": defaultdict(lambda: {"hit": 0, "miss": 0}),
    "Accra": defaultdict(lambda: {"hit": 0, "miss": 0}),
    "Berlin": defaultdict(lambda: {"hit": 0, "miss": 0}),
    "Calgary": defaultdict(lambda: {"hit": 0, "miss": 0}),
    "Toronto": defaultdict(lambda: {"hit": 0, "miss": 0}),
}

for i in range(len(filename)):
    per_file_results, timestamp = parse_allinone(str(filename[i]))
    if per_file_results is None:
        continue

    for j in range(len(per_file_results)):
        domain = per_file_results[j]["domain"]
        city = per_file_results[j]["city"]

        if city not in unique_cities:
            continue

        linestyle = "-"

        if city == "Lusaka":
            if timestamp < datetime(2025, 4, 1, tzinfo=pytz.UTC):
                city = "Lusaka-NBO"
            else:
                city = "Lusaka-JNB"
                linestyle = "dotted"

        cdn_server_id_key = per_file_results[j]["cdn_server_id_key"]
        cdn_server_id_value = per_file_results[j]["cdn_server_id_value"]

        if cdn_server_id_key == "cf-ray":
            if city == "Calgary" and "YYC" not in cdn_server_id_value:
                continue
            if city == "Toronto" and "YUL" not in cdn_server_id_value:
                continue
            cache_status = per_file_results[j]["cf_cache_status"]

            date_key = per_file_results[j]["timestamp"].date()
            if cache_status in hit_statuses:
                per_day_counts[city][date_key]["hit"] += 1
            elif cache_status in miss_statuses:
                per_day_counts[city][date_key]["miss"] += 1

daily_city_hitrates = {
    "Vancouver": [],
    "Lusaka-NBO": [],
    "Lusaka-JNB": [],
    "Accra": [],
    "Berlin": [],
    "Calgary": [],
    "Toronto": [],
}
for city, day_map in per_day_counts.items():
    for day, counts in day_map.items():
        total = counts["hit"] + counts["miss"]
        if total > 0:
            daily_city_hitrates[city].append(counts["hit"] / total)

plot(daily_city_hitrates)

### Time-to-first-byte

In [ ]:

from datetime import datetime



plt.rc("text", usetex=True)
rcParams["font.family"] = "CMU Sans Serif"
rcParams["font.size"] = 10

cols = plt.get_cmap("tab10").colors

country_mapping = {
    "Lusaka-NBO": "ZM (NBO PoP)",
    "Lusaka-JNB": "ZM (JNB PoP)",
    "Accra": "GH (LOS PoP)",
    "Berlin": "DE (FRA PoP)",
    "Vancouver": "CA (SEA PoP)",
    "Calgary": "CA (YYC PoP)",
    "Toronto": "CA (YUL PoP)",
}
unique_cities = ["Lusaka", "Accra", "Berlin", "Vancouver", "Calgary", "Toronto"]



In [ ]:
def parse_allinone(filename: str):
    data = json.load(open(filename, "r"))
    timestamps = filename.split("_")[-1].split(".")[0]
    timestamps = datetime.strptime(timestamps, "%Y%m%dT%H%M%SZ").replace(
        tzinfo=pytz.UTC
    )
    user_details = data["user_details"]
    city = user_details["City"]
    country = user_details["Country"]
    cache_result = []
    for domain, result in data["web_measurements"].items():
        dns_results = result["dns"]
        dns_result_q_default = float(dns_results["Default"]["query_time"]) / 1000

        if "Google" in dns_results:
            dns_result_q_google = float(dns_results["Google"]["query_time"]) / 1000
        else:
            dns_result_q_google = None

        if "Quad9" in dns_results:
            dns_result_q_quad9 = float(dns_results["Quad9"]["query_time"]) / 1000
        else:
            dns_result_q_quad9 = None

        if "Cloudflare" in dns_results:
            dns_result_q_cloudflare = (
                float(dns_results["Cloudflare"]["query_time"]) / 1000
            )

        else:
            dns_result_q_cloudflare = None
        curl_results = result["curl"]
        for ip, curl_resp in curl_results.items():
            initconnect_time = curl_resp.get("initconnect_time")
            appconnect_time = curl_resp.get("appconnect_time")
            pretransfer_time = curl_resp.get("pretransfer_time")
            redirect_time = curl_resp.get("redirect_time")
            starttransfer_time = curl_resp.get("starttransfer_time")
            total_time = curl_resp.get("total_time")
            cdn_server_id_key = curl_resp.get("cdn_server_id_key")
            cdn_server_id_value = curl_resp.get("cdn_server_id_value")
            cf_cache_status = curl_resp.get("cf_cache_status")
            x_cache = curl_resp.get("x_cache")
            x_cache_remote = curl_resp.get("x_cache_remote")

            r = {
                "domain": domain,
                "timestamp": timestamps,
                "ip": ip,
                "city": city,
                "country": country,
                "initconnect_time": initconnect_time,
                "appconnect_time": appconnect_time,
                "pretransfer_time": pretransfer_time,
                "redirect_time": redirect_time,
                "starttransfer_time": starttransfer_time,
                "total_time": total_time,
                "cdn_server_id_key": cdn_server_id_key,
                "cdn_server_id_value": cdn_server_id_value,
                "cf_cache_status": cf_cache_status,
                "x_cache": x_cache,
                "x_cache_remote": x_cache_remote,
                "dns_query_time_default": dns_result_q_default,
                "dns_query_time_google": dns_result_q_google,
                "dns_query_time_quad9": dns_result_q_quad9,
                "dns_query_time_cloudflare": dns_result_q_cloudflare,
                "overall_total_time": total_time + dns_result_q_default,
            }
            cache_result.append(r)
            break
    return cache_result, timestamps

In [ ]:
def parse_to_df() -> pd.DataFrame:
    filename = []
    path = Path(webMsms_dir)
    for dirpath, _, files in os.walk(path):
        if len(files) != 0:
            for f in files:
                if f.endswith(".json"):
                    filename.append(Path(dirpath).joinpath(f))

    results = []
    for i in range(len(filename)):
        per_file_results, timestamp = parse_allinone(str(filename[i]))
        if per_file_results is None:
            continue

        for j in range(len(per_file_results)):
            country = per_file_results[j]["country"]
            city = per_file_results[j]["city"]
            if country in ["US", "CH"]:
                continue
            if city not in unique_cities:
                continue

            if city == "Lusaka":
                if timestamp < datetime(2025, 4, 1, tzinfo=pytz.UTC):
                    city = "Lusaka-NBO"
                else:
                    city = "Lusaka-JNB"
                per_file_results[j]["city"] = city

            results.append(per_file_results[j])

    df = pd.DataFrame(results)
    df = df.sort_values(
        by=["city", "country", "timestamp", "cdn_server_id_key", "domain"]
    )
    return df

In [ ]:
def component_plot(df: pd.DataFrame):
    df = df[df["cdn_server_id_key"] == "cf-ray"].copy()

    for col in ["total_time", "starttransfer_time"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df = df.dropna(subset=["total_time", "starttransfer_time"])

    summary = (
        df.groupby("city")[["total_time", "starttransfer_time"]]
        .mean()
        .rename(
            columns={
                "total_time": "mean_total_time",
                "starttransfer_time": "mean_starttransfer_time",
            }
        )
        .reset_index()
    )

    summary.to_csv("cf_ray_city_ttfb_summary.csv", index=False)
    print("Cloudflare (cf-ray) mean total_time and starttransfer_time by country:")
    print(summary)

    ordered = [
        "Vancouver",
        "Lusaka-NBO",
        "Lusaka-JNB",
        "Accra",
        "Berlin",
        "Calgary",
        "Toronto",
    ]
    summary = summary[summary["city"].isin(ordered)].copy()
    summary["city"] = pd.Categorical(summary["city"], categories=ordered, ordered=True)
    summary = summary.sort_values("city")

    if summary.empty:
        return

    start_vals = summary["mean_starttransfer_time"].values
    total_vals = summary["mean_total_time"].values
    remaining = (total_vals - start_vals).clip(min=0)

    fig, ax = plt.subplots(figsize=(4.66, 2))
    y = summary["city"].values

    ax.barh(
        y, start_vals, color=cols[1], edgecolor="black", label="Start Transfer (TTFB)"
    )
    ax.barh(
        y,
        remaining,
        left=start_vals,
        color=cols[0],
        edgecolor="black",
        label="Remaining Transfer",
    )

    for yt, s, r in zip(y, start_vals, remaining):
        ax.text(s + r + 0.01, yt, f"{s + r:.2f}s", va="center", fontsize=8)

    ax.set_yticks(y)
    ax.set_yticklabels([country_mapping.get(c, c) for c in y])
    ax.set_xlabel("Total Page Fetch Time [s]")
    ax.grid(axis="x", linestyle="--", alpha=0.5)
    ax.set_xlim(0, max(total_vals) * 1.15)

    ax.legend(
        loc="best",
        fontsize="small",
        edgecolor="k",
        handlelength=1,
        labelspacing=0.06,
        columnspacing=0.5,
        handletextpad=0.3,
        fancybox=False,
    )
    plt.tight_layout()
    plt.savefig(f"{figs_top_level_folder_path}/cf_ray_city_ttfb_bar.png", dpi=300)
    plt.savefig(f"{figs_top_level_folder_path}/cf_ray_city_ttfb_bar.pdf", dpi=300)
    plt.show()

In [ ]:
df = parse_to_df()
df.head()

In [ ]:
component_plot(df)